### https://github.com/VectifyAI/PageIndex/tree/main

### https://github.com/VectifyAI/PageIndex/blob/main/cookbook/pageindex_RAG_simple.ipynb

In [ ]:

curl -fsSL https://ollama.com/install.sh | sh
nohup ollama serve &
ollama pull llama3:8b
ollama list

In [1]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [2]:
!nohup ollama serve &


!ollama pull llama3

nohup: appending output to 'nohup.out'



In [1]:
!nohup ollama serve &

nohup: appending output to 'nohup.out'


In [3]:
!ollama list

NAME             ID              SIZE      MODIFIED       
llama3:latest    365c0bd3c000    4.7 GB    13 seconds ago    


In [4]:
# Install PageIndex (if not already installed)
%pip install -q --upgrade pageindex
%pip install -q --upgrade openai  # For potential future use or local OpenAI-compatible servers
%pip install -q requests beautifulsoup4  # For document downloading and parsing

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 812.0/812.0 kB 14.2 MB/s eta 0:00:00


In [ ]:
# --- START OF COLAB CELL ---

# 0.0 Setup Environment and Install Dependencies
# This cell will install necessary packages and prepare the environment.

# Install PageIndex (if not already installed)
%pip install -q --upgrade pageindex
%pip install -q --upgrade openai  # For potential future use or local OpenAI-compatible servers
%pip install -q requests beautifulsoup4  # For document downloading and parsing

In [5]:


import os
import requests
import json
from bs4 import BeautifulSoup

# Dummy PageIndexClient for local execution (no actual API calls)
class LocalPageIndexClient:
    def __init__(self, api_key=None):
        print("Using dummy LocalPageIndexClient for local execution.")
        self.documents = {} # Store document content
        self.trees = {} # Store generated tree structures

    def submit_document(self, file_path):
        doc_id = os.path.basename(file_path).replace('.', '_').replace('-', '_') # Simple doc_id generation
        with open(file_path, 'r', encoding='utf-8') as f:
            self.documents[doc_id] = f.read()
        print(f"Document submitted locally: {doc_id}")
        # In a real scenario, you'd process the document here to generate a tree.
        # For this example, we'll manually create a simplified tree for demonstration.
        self.trees[doc_id] = self._generate_simple_tree_from_text(self.documents[doc_id], doc_id)
        return {"doc_id": doc_id}

    def is_retrieval_ready(self, doc_id):
        return doc_id in self.trees

    def get_tree(self, doc_id, node_summary=True):
        return {"result": self.trees.get(doc_id, [])}

    def _generate_simple_tree_from_text(self, text_content, doc_id):
        # This is a very basic way to simulate a tree from raw text for demonstration.
        # In a real PageIndex setup, this is a sophisticated process.
        # Here, we'll just make the entire document a single node for simplicity
        # or split by major headings if available (e.g., from PDF parsing).

        # For the provided academic paper structure, we'll try to mimic it slightly.
        # This is a placeholder and would ideally use a more robust parsing.

        # Example: Try to split by common section headers for academic papers
        sections = []
        lines = text_content.split('\n')
        current_section = []
        section_title = "Document Overview"
        node_counter = 0

        # Attempt to find common academic paper headings
        # This is a heuristic and might not work perfectly for all PDFs
        section_patterns = [
            "Abstract", "Contents", "1. Introduction", "2. Approach", "3. Experiment",
            "4. Discussion", "5. Conclusion, Limitations, and Future Work",
            "References", "Appendix"
        ]

        # Use a more robust (though still simple) parsing for structure
        parsed_sections = []
        current_section_title = None
        current_section_content = []
        current_page_index = 1 # Simplified page index

        for line in lines:
            stripped_line = line.strip()
            is_new_section = False
            for pattern in section_patterns:
                if stripped_line.startswith(pattern) and len(stripped_line) < 100: # Heuristic for title
                    if current_section_title and current_section_content:
                        parsed_sections.append({
                            "title": current_section_title,
                            "text": "\n".join(current_section_content).strip(),
                            "node_id": f"{doc_id}_{node_counter:04d}",
                            "page_index": current_page_index
                        })
                        node_counter += 1
                    current_section_title = pattern
                    current_section_content = [stripped_line]
                    is_new_section = True
                    # A very rough way to simulate page breaks for a PDF
                    if "Conclusion" in pattern or "References" in pattern:
                        current_page_index += 5 # Simulate a jump
                    else:
                        current_page_index += 1
                    break
            if not is_new_section:
                current_section_content.append(line)

        if current_section_title and current_section_content:
             parsed_sections.append({
                "title": current_section_title,
                "text": "\n".join(current_section_content).strip(),
                "node_id": f"{doc_id}_{node_counter:04d}",
                "page_index": current_page_index
            })

        # Convert simple list of sections into a nested tree structure
        # For simplicity, we'll just make it a flat list under a root for this dummy.
        tree_nodes = []
        node_map_for_subnodes = {} # To easily find parents

        root_node = {
            "title": "DeepSeek-R1: Incentivizing Reasoning Cap...",
            "node_id": f"{doc_id}_0000",
            "prefix_summary": "# DeepSeek-R1: Incentivizing Reasoning C...",
            "nodes": [],
            "page_index": 1 # Root starts at page 1
        }

        for section in parsed_sections:
            # Generate a simple summary (first few words)
            summary = section["text"].split(' ', 20)
            summary = ' '.join(summary[:20]) + "..." if len(summary) > 20 else ' '.join(summary)

            # Assign a more structured node_id for consistency with original notebook
            # This is still simplified. Real PageIndex would use more robust IDing.
            original_node_id_map = {
                "Abstract": "0001", "Contents": "0002", "1. Introduction": "0003",
                "1.1. Contributions": "0004", "1.2. Summary of Evaluation Results": "0005",
                "2. Approach": "0006", "2.1. Overview": "0007", "2.2. DeepSeek-R1-Zero: Reinforcement Lea...": "0008",
                "2.2.1. Reinforcement Learning Algorithm": "0009", "2.2.2. Reward Modeling": "0010",
                "2.2.3. Training Template": "0011", "2.2.4. Performance, Self-evolution Proce...": "0012",
                "2.3. DeepSeek-R1: Reinforcement Learning...": "0013", "2.4. Distillation: Empower Small Models ...": "0014",
                "3. Experiment": "0015", "3.1. DeepSeek-R1 Evaluation": "0016", "3.2. Distilled Model Evaluation": "0017",
                "4. Discussion": "0018", "5. Conclusion, Limitations, and Future Work": "0019",
                "References": "0020", "Appendix": "0021", "A. Contributions and Acknowledgments": "0022"
            }
            node_id_suffix = original_node_id_map.get(section["title"], f"{node_counter:04d}")

            node_entry = {
                "title": section["title"],
                "node_id": node_id_suffix,
                "summary": summary,
                "text": section["text"],
                "page_index": section["page_index"]
            }

            # Simple nesting logic based on common patterns
            if section["title"].startswith("1.") and len(section["title"]) > 10 and not section["title"].startswith("1.1.") and not section["title"].startswith("1.2."): # This is a bit weak, just for demo
                 node_map_for_subnodes["1. Introduction"] = node_entry
                 root_node["nodes"].append(node_entry)
            elif section["title"].startswith("1.1.") or section["title"].startswith("1.2."):
                if "1. Introduction" in node_map_for_subnodes and "nodes" not in node_map_for_subnodes["1. Introduction"]:
                    node_map_for_subnodes["1. Introduction"]["nodes"] = []
                if "1. Introduction" in node_map_for_subnodes:
                    node_map_for_subnodes["1. Introduction"]["nodes"].append(node_entry)
            elif section["title"].startswith("2.") and len(section["title"]) > 10 and not section["title"].startswith("2.1.") and not section["title"].startswith("2.2.") and not section["title"].startswith("2.3.") and not section["title"].startswith("2.4."):
                node_map_for_subnodes["2. Approach"] = node_entry
                root_node["nodes"].append(node_entry)
            elif section["title"].startswith("2.1.") or section["title"].startswith("2.2.") or section["title"].startswith("2.3.") or section["title"].startswith("2.4."):
                if "2. Approach" in node_map_for_subnodes and "nodes" not in node_map_for_subnodes["2. Approach"]:
                    node_map_for_subnodes["2. Approach"]["nodes"] = []
                if "2. Approach" in node_map_for_subnodes:
                    node_map_for_subnodes["2. Approach"]["nodes"].append(node_entry)
            elif section["title"].startswith("3.") and len(section["title"]) > 10 and not section["title"].startswith("3.1.") and not section["title"].startswith("3.2."):
                 node_map_for_subnodes["3. Experiment"] = node_entry
                 root_node["nodes"].append(node_entry)
            elif section["title"].startswith("3.1.") or section["title"].startswith("3.2."):
                if "3. Experiment" in node_map_for_subnodes and "nodes" not in node_map_for_subnodes["3. Experiment"]:
                    node_map_for_subnodes["3. Experiment"]["nodes"] = []
                if "3. Experiment" in node_map_for_subnodes:
                    node_map_for_subnodes["3. Experiment"]["nodes"].append(node_entry)
            elif section["title"].startswith("A."):
                if "Appendix" in node_map_for_subnodes and "nodes" not in node_map_for_subnodes["Appendix"]:
                    node_map_for_subnodes["Appendix"]["nodes"] = []
                if "Appendix" in node_map_for_subnodes:
                    node_map_for_subnodes["Appendix"]["nodes"].append(node_entry)
            else:
                root_node["nodes"].append(node_entry) # Add as top-level if no specific parent

        return [root_node] # Return as a list with the root node

# Dummy utils for local execution (mimicking pageindex.utils)
class LocalUtils:
    @staticmethod
    def print_tree(tree, indent=0):
        for node in tree:
            prefix = "  " * indent
            print(f"{prefix}{node.get('title', 'Untitled')} (ID: {node.get('node_id', 'N/A')})")
            if "nodes" in node:
                LocalUtils.print_tree(node["nodes"], indent + 1)

    @staticmethod
    def remove_fields(obj, fields):
        if isinstance(obj, dict):
            return {k: LocalUtils.remove_fields(v, fields) for k, v in obj.items() if k not in fields}
        elif isinstance(obj, list):
            return [LocalUtils.remove_fields(elem, fields) for elem in obj]
        else:
            return obj

    @staticmethod
    def create_node_mapping(tree, node_map=None):
        if node_map is None:
            node_map = {}
        for node in tree:
            node_map[node.get('node_id')] = node
            if "nodes" in node:
                LocalUtils.create_node_mapping(node["nodes"], node_map)
        return node_map

    @staticmethod
    def print_wrapped(text, width=80):
        import textwrap
        print(textwrap.fill(text, width=width))

pi_client = LocalPageIndexClient()
utils = LocalUtils()

# Ollama LLM client for local inference
import httpx

async def call_llm(prompt, model="llama3:latest", temperature=0):
    # Ensure Ollama server is running and model is pulled
    # Example: ollama run llama3
    client = httpx.AsyncClient(base_url="http://localhost:11434")
    try:
        response = await client.post(
            "/api/chat",
            json={
                "model": model,
                "messages": [{"role": "user", "content": prompt}],
                "stream": False,
                "options": {"temperature": temperature}
            },
            timeout=None # Disable timeout for potentially long responses
        )
        response.raise_for_status() # Raise an exception for HTTP errors
        return response.json()["message"]["content"].strip()
    except httpx.ConnectError as e:
        print(f"Error connecting to Ollama: {e}")
        print("Please ensure Ollama is running (e.g., 'ollama serve') and you have pulled the model (e.g., 'ollama pull llama3').")
        return "ERROR: Could not connect to Ollama. Please check your Ollama setup."
    except httpx.HTTPStatusError as e:
        print(f"Error from Ollama server: {e.response.status_code} - {e.response.text}")
        return f"ERROR: Ollama server responded with an error: {e.response.status_code}"
    except json.JSONDecodeError:
        print(f"Error decoding JSON from Ollama: {response.text}")
        return "ERROR: Invalid JSON response from Ollama."
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return f"ERROR: An unexpected error occurred: {e}"

# 1.1 Submit a document for generating PageIndex tree (locally)
pdf_url = "https://arxiv.org/pdf/2501.12948.pdf"
pdf_filename = pdf_url.split('/')[-1]
pdf_path = os.path.join("data", pdf_filename) # Use a 'data' directory

os.makedirs(os.path.dirname(pdf_path), exist_ok=True)

# Download the PDF content
print(f"Downloading {pdf_url}...")
response = requests.get(pdf_url)
response.raise_for_status() # Check for request errors

# Convert PDF content to text (simplified for local demo)
# In a real PageIndex client, this would involve robust PDF parsing.
# For this example, we'll try to get text from an existing HTML version if possible
# or just note that a real PDF parser is needed.
# Since the original notebook downloaded a PDF and then submitted it, we'll simulate
# having the text content. A full PDF-to-text conversion is outside this script's scope,
# but for demonstration, we'll assume we have the text content extracted.

# For a true local PDF text extraction, you'd use libraries like PyPDF2, pdfminer.six, or pypopper.
# Since we are focusing on *how* to integrate with Ollama once text is available,
# let's use a placeholder.
# For the purpose of making this runnable, let's create a dummy text file
# representing the content of the PDF.

# Fetch the abstract and intro from arXiv for a more realistic text
arxiv_id = pdf_filename.replace('.pdf', '')
arxiv_html_url = f"https://arxiv.org/html/{arxiv_id}v1.html" # Assuming v1 exists

print(f"Attempting to fetch text from {arxiv_html_url} for content simulation...")
try:
    html_response = requests.get(arxiv_html_url)
    html_response.raise_for_status()
    soup = BeautifulSoup(html_response.text, 'html.parser')

    # Extract main content - this will be highly dependent on arXiv's HTML structure
    # A more robust solution would be to use a PDF parser.
    # For now, let's grab all paragraphs and headings.
    extracted_text_parts = []
    for tag in soup.find_all(['h1', 'h2', 'h3', 'p', 'li']):
        text = tag.get_text(separator=' ', strip=True)
        if text:
            extracted_text_parts.append(text)

    dummy_text_content = "\n\n".join(extracted_text_parts)
    if not dummy_text_content.strip():
        raise ValueError("Could not extract significant text from arXiv HTML.")

    text_path = pdf_path.replace('.pdf', '.txt')
    with open(text_path, "w", encoding="utf-8") as f:
        f.write(dummy_text_content)
    print(f"Extracted content to {text_path}")
    doc_id = pi_client.submit_document(text_path)["doc_id"]
    print('Document Submitted:', doc_id)

except Exception as e:
    print(f"Failed to extract text from arXiv HTML or save to file: {e}")
    print("Falling back to a very minimal dummy text for tree generation.")
    dummy_text_content = """
    # DeepSeek-R1: Incentivizing Reasoning Capability of Large Language Models
    ## Abstract
    This paper introduces DeepSeek-R1-Zero and DeepSeek-R1, two reinforcement learning (RL) based approaches to enhance reasoning abilities in Large Language Models (LLMs). DeepSeek-R1-Zero uses a pure RL approach without cold-start data, achieving strong performance. DeepSeek-R1 further improves by leveraging cold-start data and iterative RL fine-tuning, reaching performance comparable to OpenAI-o1-1217 on various tasks.
    ## 1. Introduction
    Recent advancements in LLMs have shown impressive reasoning capabilities. This work explores methods to further improve these by directly optimizing for reasoning with RL.
    ### 1.1. Contributions
    We propose novel RL algorithms and a distillation process.
    ## 5. Conclusion, Limitations, and Future Work
    We conclude that RL is effective for enhancing reasoning. DeepSeek-R1 shows state-of-the-art results. Distillation to smaller models is promising. Future work involves scaling and exploring new reward functions.
    """
    text_path = pdf_path.replace('.pdf', '.txt')
    with open(text_path, "w", encoding="utf-8") as f:
        f.write(dummy_text_content)
    print(f"Used minimal dummy text and saved to {text_path}")
    doc_id = pi_client.submit_document(text_path)["doc_id"]
    print('Document Submitted:', doc_id)


# 1.2 Get the generated PageIndex tree structure
# This will use the simplified tree generated by our LocalPageIndexClient
if pi_client.is_retrieval_ready(doc_id):
    tree = pi_client.get_tree(doc_id, node_summary=True)['result']
    print('\nSimplified Tree Structure of the Document:')
    utils.print_tree(tree)
else:
    print("Processing document, please try again later...")

# 2.1 Use LLM for tree search and identify nodes that might contain relevant context
query = "What are the conclusions in this document?"

# Remove the 'text' field to avoid sending too much data to the LLM for tree search
tree_without_text = utils.remove_fields(tree.copy(), fields=['text'])

search_prompt = f"""
You are given a question and a tree structure of a document.
Each node contains a node id, node title, and a corresponding summary.
Your task is to find all nodes that are likely to contain the answer to the question.

Question: {query}

Document tree structure:
{json.dumps(tree_without_text, indent=2)}

Please reply in the following JSON format:
{{
    "thinking": "<Your thinking process on which nodes are relevant to the question>",
    "node_list": ["node_id_1", "node_id_2", ..., "node_id_n"]
}}
Directly return the final JSON structure. Do not output anything else.
"""

print("\nCalling local LLM for tree search (this might take a moment)...")
tree_search_result = await call_llm(search_prompt, model="llama3") # Use llama3 or your preferred local model

# 2.2 Print retrieved nodes and reasoning process
try:
    node_map = utils.create_node_mapping(tree)
    tree_search_result_json = json.loads(tree_search_result)

    print('\nReasoning Process:')
    utils.print_wrapped(tree_search_result_json.get('thinking', 'No thinking process provided by LLM.'))

    print('\nRetrieved Nodes:')
    retrieved_node_ids = tree_search_result_json.get("node_list", [])
    if not retrieved_node_ids:
        print("No nodes retrieved by LLM.")
    for node_id_key in retrieved_node_ids:
        # The node_id from the LLM might be just '0019', need to prepend doc_id if that's how we store
        # For this dummy client, the node_ids are like "doc_id_0019"
        # Let's adjust for the dummy structure if LLM outputs just the number
        actual_node_id = f"{doc_id}_{node_id_key}" if '_' not in node_id_key else node_id_key

        node = node_map.get(actual_node_id)
        if node:
            print(f"Node ID: {node['node_id']}\t Page: {node.get('page_index', 'N/A')}\t Title: {node['title']}")
        else:
            print(f"Node ID: {actual_node_id} (Not found in map - LLM might have hallucinated or ID format mismatch)")
except json.JSONDecodeError:
    print(f"\nError: LLM did not return valid JSON for tree search result:\n{tree_search_result}")
except Exception as e:
    print(f"\nAn error occurred during tree search result processing: {e}")


# 3.1 Extract relevant context from retrieved nodes
# Use the node_map to get the full text of the identified nodes
# Re-parse LLM result in case of error in previous block
try:
    retrieved_node_ids = json.loads(tree_search_result).get("node_list", [])
except json.JSONDecodeError:
    retrieved_node_ids = []

relevant_content = []
for node_id_key in retrieved_node_ids:
    actual_node_id = f"{doc_id}_{node_id_key}" if '_' not in node_id_key else node_id_key
    node = node_map.get(actual_node_id)
    if node and 'text' in node:
        relevant_content.append(node['text'])

relevant_content_str = "\n\n".join(relevant_content)

print('\nRetrieved Context:\n')
if relevant_content_str:
    utils.print_wrapped(relevant_content_str[:1000] + '...' if len(relevant_content_str) > 1000 else relevant_content_str)
else:
    print("No relevant context found based on LLM's node list.")


# 3.2 Generate answer based on retrieved context
if relevant_content_str:
    answer_prompt = f"""
    Answer the question based on the context:\n\nQuestion: {query}\nContext: {relevant_content_str}\n\nProvide a clear, concise answer based only on the context provided.
    """
    print('\nCalling local LLM for answer generation...')
    answer = await call_llm(answer_prompt, model="llama3:latest") # Use llama3 or your preferred local model
    print('\nGenerated Answer:\n')
    utils.print_wrapped(answer)
else:
    print("\nCannot generate an answer as no relevant context was retrieved.")

# --- END OF COLAB CELL ---

Using dummy LocalPageIndexClient for local execution.
Attempting to fetch text from https://arxiv.org/html/2501.12948v1.html for content simulation...
Failed to extract text from arXiv HTML or save to file: 404 Client Error: Not Found for url: https://arxiv.org/html/2501.12948v1.html
Falling back to a very minimal dummy text for tree generation.
Used minimal dummy text and saved to data/2501.12948.txt
Document submitted locally: 2501_12948_txt
Document Submitted: 2501_12948_txt

Simplified Tree Structure of the Document:
DeepSeek-R1: Incentivizing Reasoning Cap... (ID: 2501_12948_txt_0000)

Calling local LLM for tree search (this might take a moment)...

Reasoning Process:
The question is asking about conclusions in this document, so I will look for
nodes that have 'conclusion' or 'summary' in their title or summary.

Retrieved Nodes:
Node ID: 2501_12948_txt_0000	 Page: 1	 Title: DeepSeek-R1: Incentivizing Reasoning Cap...

Retrieved Context:

No relevant context found based on LLM's n

In [ ]:
https://arxiv.org/pdf/2501.12948

In [2]:


import os
import requests
import json
from bs4 import BeautifulSoup

# Dummy PageIndexClient for local execution (no actual API calls)
class LocalPageIndexClient:
    def __init__(self, api_key=None):
        print("Using dummy LocalPageIndexClient for local execution.")
        self.documents = {} # Store document content
        self.trees = {} # Store generated tree structures

    def submit_document(self, file_path):
        doc_id = os.path.basename(file_path).replace('.', '_').replace('-', '_') # Simple doc_id generation
        with open(file_path, 'r', encoding='utf-8') as f:
            self.documents[doc_id] = f.read()
        print(f"Document submitted locally: {doc_id}")
        # In a real scenario, you'd process the document here to generate a tree.
        # For this example, we'll manually create a simplified tree for demonstration.
        self.trees[doc_id] = self._generate_simple_tree_from_text(self.documents[doc_id], doc_id)
        return {"doc_id": doc_id}

    def is_retrieval_ready(self, doc_id):
        return doc_id in self.trees

    def get_tree(self, doc_id, node_summary=True):
        return {"result": self.trees.get(doc_id, [])}

    def _generate_simple_tree_from_text(self, text_content, doc_id):
        # This is a very basic way to simulate a tree from raw text for demonstration.
        # In a real PageIndex setup, this is a sophisticated process.
        # Here, we'll just make the entire document a single node for simplicity
        # or split by major headings if available (e.g., from PDF parsing).

        # For the provided academic paper structure, we'll try to mimic it slightly.
        # This is a placeholder and would ideally use a more robust parsing.

        # Example: Try to split by common section headers for academic papers
        sections = []
        lines = text_content.split('\n')
        current_section = []
        section_title = "Document Overview"
        node_counter = 0

        # Attempt to find common academic paper headings
        # This is a heuristic and might not work perfectly for all PDFs
        section_patterns = [
            "Abstract", "Contents", "1. Introduction", "2. Approach", "3. Experiment",
            "4. Discussion", "5. Conclusion, Limitations, and Future Work",
            "References", "Appendix"
        ]

        # Use a more robust (though still simple) parsing for structure
        parsed_sections = []
        current_section_title = None
        current_section_content = []
        current_page_index = 1 # Simplified page index

        for line in lines:
            stripped_line = line.strip()
            is_new_section = False
            for pattern in section_patterns:
                if stripped_line.startswith(pattern) and len(stripped_line) < 100: # Heuristic for title
                    if current_section_title and current_section_content:
                        parsed_sections.append({
                            "title": current_section_title,
                            "text": "\n".join(current_section_content).strip(),
                            "node_id": f"{doc_id}_{node_counter:04d}",
                            "page_index": current_page_index
                        })
                        node_counter += 1
                    current_section_title = pattern
                    current_section_content = [stripped_line]
                    is_new_section = True
                    # A very rough way to simulate page breaks for a PDF
                    if "Conclusion" in pattern or "References" in pattern:
                        current_page_index += 5 # Simulate a jump
                    else:
                        current_page_index += 1
                    break
            if not is_new_section:
                current_section_content.append(line)

        if current_section_title and current_section_content:
             parsed_sections.append({
                "title": current_section_title,
                "text": "\n".join(current_section_content).strip(),
                "node_id": f"{doc_id}_{node_counter:04d}",
                "page_index": current_page_index
            })

        # Convert simple list of sections into a nested tree structure
        # For simplicity, we'll just make it a flat list under a root for this dummy.
        tree_nodes = []
        node_map_for_subnodes = {} # To easily find parents

        root_node = {
            "title": "DeepSeek-R1: Incentivizing Reasoning Cap...",
            "node_id": f"{doc_id}_0000",
            "prefix_summary": "# DeepSeek-R1: Incentivizing Reasoning C...",
            "nodes": [],
            "page_index": 1 # Root starts at page 1
        }

        for section in parsed_sections:
            # Generate a simple summary (first few words)
            summary = section["text"].split(' ', 20)
            summary = ' '.join(summary[:20]) + "..." if len(summary) > 20 else ' '.join(summary)

            # Assign a more structured node_id for consistency with original notebook
            # This is still simplified. Real PageIndex would use more robust IDing.
            original_node_id_map = {
                "Abstract": "0001", "Contents": "0002", "1. Introduction": "0003",
                "1.1. Contributions": "0004", "1.2. Summary of Evaluation Results": "0005",
                "2. Approach": "0006", "2.1. Overview": "0007", "2.2. DeepSeek-R1-Zero: Reinforcement Lea...": "0008",
                "2.2.1. Reinforcement Learning Algorithm": "0009", "2.2.2. Reward Modeling": "0010",
                "2.2.3. Training Template": "0011", "2.2.4. Performance, Self-evolution Proce...": "0012",
                "2.3. DeepSeek-R1: Reinforcement Learning...": "0013", "2.4. Distillation: Empower Small Models ...": "0014",
                "3. Experiment": "0015", "3.1. DeepSeek-R1 Evaluation": "0016", "3.2. Distilled Model Evaluation": "0017",
                "4. Discussion": "0018", "5. Conclusion, Limitations, and Future Work": "0019",
                "References": "0020", "Appendix": "0021", "A. Contributions and Acknowledgments": "0022"
            }
            node_id_suffix = original_node_id_map.get(section["title"], f"{node_counter:04d}")

            node_entry = {
                "title": section["title"],
                "node_id": node_id_suffix,
                "summary": summary,
                "text": section["text"],
                "page_index": section["page_index"]
            }

            # Simple nesting logic based on common patterns
            if section["title"].startswith("1.") and len(section["title"]) > 10 and not section["title"].startswith("1.1.") and not section["title"].startswith("1.2."): # This is a bit weak, just for demo
                 node_map_for_subnodes["1. Introduction"] = node_entry
                 root_node["nodes"].append(node_entry)
            elif section["title"].startswith("1.1.") or section["title"].startswith("1.2."):
                if "1. Introduction" in node_map_for_subnodes and "nodes" not in node_map_for_subnodes["1. Introduction"]:
                    node_map_for_subnodes["1. Introduction"]["nodes"] = []
                if "1. Introduction" in node_map_for_subnodes:
                    node_map_for_subnodes["1. Introduction"]["nodes"].append(node_entry)
            elif section["title"].startswith("2.") and len(section["title"]) > 10 and not section["title"].startswith("2.1.") and not section["title"].startswith("2.2.") and not section["title"].startswith("2.3.") and not section["title"].startswith("2.4."):
                node_map_for_subnodes["2. Approach"] = node_entry
                root_node["nodes"].append(node_entry)
            elif section["title"].startswith("2.1.") or section["title"].startswith("2.2.") or section["title"].startswith("2.3.") or section["title"].startswith("2.4."):
                if "2. Approach" in node_map_for_subnodes and "nodes" not in node_map_for_subnodes["2. Approach"]:
                    node_map_for_subnodes["2. Approach"]["nodes"] = []
                if "2. Approach" in node_map_for_subnodes:
                    node_map_for_subnodes["2. Approach"]["nodes"].append(node_entry)
            elif section["title"].startswith("3.") and len(section["title"]) > 10 and not section["title"].startswith("3.1.") and not section["title"].startswith("3.2."):
                 node_map_for_subnodes["3. Experiment"] = node_entry
                 root_node["nodes"].append(node_entry)
            elif section["title"].startswith("3.1.") or section["title"].startswith("3.2."):
                if "3. Experiment" in node_map_for_subnodes and "nodes" not in node_map_for_subnodes["3. Experiment"]:
                    node_map_for_subnodes["3. Experiment"]["nodes"] = []
                if "3. Experiment" in node_map_for_subnodes:
                    node_map_for_subnodes["3. Experiment"]["nodes"].append(node_entry)
            elif section["title"].startswith("A."):
                if "Appendix" in node_map_for_subnodes and "nodes" not in node_map_for_subnodes["Appendix"]:
                    node_map_for_subnodes["Appendix"]["nodes"] = []
                if "Appendix" in node_map_for_subnodes:
                    node_map_for_subnodes["Appendix"]["nodes"].append(node_entry)
            else:
                root_node["nodes"].append(node_entry) # Add as top-level if no specific parent

        return [root_node] # Return as a list with the root node

# Dummy utils for local execution (mimicking pageindex.utils)
class LocalUtils:
    @staticmethod
    def print_tree(tree, indent=0):
        for node in tree:
            prefix = "  " * indent
            print(f"{prefix}{node.get('title', 'Untitled')} (ID: {node.get('node_id', 'N/A')})")
            if "nodes" in node:
                LocalUtils.print_tree(node["nodes"], indent + 1)

    @staticmethod
    def remove_fields(obj, fields):
        if isinstance(obj, dict):
            return {k: LocalUtils.remove_fields(v, fields) for k, v in obj.items() if k not in fields}
        elif isinstance(obj, list):
            return [LocalUtils.remove_fields(elem, fields) for elem in obj]
        else:
            return obj

    @staticmethod
    def create_node_mapping(tree, node_map=None):
        if node_map is None:
            node_map = {}
        for node in tree:
            node_map[node.get('node_id')] = node
            if "nodes" in node:
                LocalUtils.create_node_mapping(node["nodes"], node_map)
        return node_map

    @staticmethod
    def print_wrapped(text, width=80):
        import textwrap
        print(textwrap.fill(text, width=width))

pi_client = LocalPageIndexClient()
utils = LocalUtils()

# Ollama LLM client for local inference
import httpx

async def call_llm(prompt, model="llama3:latest", temperature=0):
    # Ensure Ollama server is running and model is pulled
    # Example: ollama run llama3
    client = httpx.AsyncClient(base_url="http://localhost:11434")
    try:
        response = await client.post(
            "/api/chat",
            json={
                "model": model,
                "messages": [{"role": "user", "content": prompt}],
                "stream": False,
                "options": {"temperature": temperature}
            },
            timeout=None # Disable timeout for potentially long responses
        )
        response.raise_for_status() # Raise an exception for HTTP errors
        return response.json()["message"]["content"].strip()
    except httpx.ConnectError as e:
        print(f"Error connecting to Ollama: {e}")
        print("Please ensure Ollama is running (e.g., 'ollama serve') and you have pulled the model (e.g., 'ollama pull llama3').")
        return "ERROR: Could not connect to Ollama. Please check your Ollama setup."
    except httpx.HTTPStatusError as e:
        print(f"Error from Ollama server: {e.response.status_code} - {e.response.text}")
        return f"ERROR: Ollama server responded with an error: {e.response.status_code}"
    except json.JSONDecodeError:
        print(f"Error decoding JSON from Ollama: {response.text}")
        return "ERROR: Invalid JSON response from Ollama."
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return f"ERROR: An unexpected error occurred: {e}"

# 1.1 Submit a document for generating PageIndex tree (locally)
pdf_url = "https://arxiv.org/pdf/2501.12948"
pdf_filename = pdf_url.split('/')[-1]
pdf_path = os.path.join("data", pdf_filename) # Use a 'data' directory

os.makedirs(os.path.dirname(pdf_path), exist_ok=True)

# Download the PDF content
print(f"Downloading {pdf_url}...")
response = requests.get(pdf_url)
response.raise_for_status() # Check for request errors

# Convert PDF content to text (simplified for local demo)
# In a real PageIndex client, this would involve robust PDF parsing.
# For this example, we'll try to get text from an existing HTML version if possible
# or just note that a real PDF parser is needed.
# Since the original notebook downloaded a PDF and then submitted it, we'll simulate
# having the text content. A full PDF-to-text conversion is outside this script's scope,
# but for demonstration, we'll assume we have the text content extracted.

# For a true local PDF text extraction, you'd use libraries like PyPDF2, pdfminer.six, or pypopper.
# Since we are focusing on *how* to integrate with Ollama once text is available,
# let's use a placeholder.
# For the purpose of making this runnable, let's create a dummy text file
# representing the content of the PDF.

# Fetch the abstract and intro from arXiv for a more realistic text
arxiv_id = pdf_filename.replace('.pdf', '')
arxiv_html_url = f"https://arxiv.org/html/{arxiv_id}v1.html" # Assuming v1 exists

print(f"Attempting to fetch text from {arxiv_html_url} for content simulation...")
try:
    html_response = requests.get(arxiv_html_url)
    html_response.raise_for_status()
    soup = BeautifulSoup(html_response.text, 'html.parser')

    # Extract main content - this will be highly dependent on arXiv's HTML structure
    # A more robust solution would be to use a PDF parser.
    # For now, let's grab all paragraphs and headings.
    extracted_text_parts = []
    for tag in soup.find_all(['h1', 'h2', 'h3', 'p', 'li']):
        text = tag.get_text(separator=' ', strip=True)
        if text:
            extracted_text_parts.append(text)

    dummy_text_content = "\n\n".join(extracted_text_parts)
    if not dummy_text_content.strip():
        raise ValueError("Could not extract significant text from arXiv HTML.")

    text_path = pdf_path.replace('.pdf', '.txt')
    with open(text_path, "w", encoding="utf-8") as f:
        f.write(dummy_text_content)
    print(f"Extracted content to {text_path}")
    doc_id = pi_client.submit_document(text_path)["doc_id"]
    print('Document Submitted:', doc_id)

except Exception as e:
    print(f"Failed to extract text from arXiv HTML or save to file: {e}")
    print("Falling back to a very minimal dummy text for tree generation.")
    dummy_text_content = """
    # DeepSeek-R1: Incentivizing Reasoning Capability of Large Language Models
    ## Abstract
    This paper introduces DeepSeek-R1-Zero and DeepSeek-R1, two reinforcement learning (RL) based approaches to enhance reasoning abilities in Large Language Models (LLMs). DeepSeek-R1-Zero uses a pure RL approach without cold-start data, achieving strong performance. DeepSeek-R1 further improves by leveraging cold-start data and iterative RL fine-tuning, reaching performance comparable to OpenAI-o1-1217 on various tasks.
    ## 1. Introduction
    Recent advancements in LLMs have shown impressive reasoning capabilities. This work explores methods to further improve these by directly optimizing for reasoning with RL.
    ### 1.1. Contributions
    We propose novel RL algorithms and a distillation process.
    ## 5. Conclusion, Limitations, and Future Work
    We conclude that RL is effective for enhancing reasoning. DeepSeek-R1 shows state-of-the-art results. Distillation to smaller models is promising. Future work involves scaling and exploring new reward functions.
    """
    text_path = pdf_path.replace('.pdf', '.txt')
    with open(text_path, "w", encoding="utf-8") as f:
        f.write(dummy_text_content)
    print(f"Used minimal dummy text and saved to {text_path}")
    doc_id = pi_client.submit_document(text_path)["doc_id"]
    print('Document Submitted:', doc_id)


# 1.2 Get the generated PageIndex tree structure
# This will use the simplified tree generated by our LocalPageIndexClient
if pi_client.is_retrieval_ready(doc_id):
    tree = pi_client.get_tree(doc_id, node_summary=True)['result']
    print('\nSimplified Tree Structure of the Document:')
    utils.print_tree(tree)
else:
    print("Processing document, please try again later...")

# 2.1 Use LLM for tree search and identify nodes that might contain relevant context
query = "What are the conclusions in this document?"

# Remove the 'text' field to avoid sending too much data to the LLM for tree search
tree_without_text = utils.remove_fields(tree.copy(), fields=['text'])

search_prompt = f"""
You are given a question and a tree structure of a document.
Each node contains a node id, node title, and a corresponding summary.
Your task is to find all nodes that are likely to contain the answer to the question.

Question: {query}

Document tree structure:
{json.dumps(tree_without_text, indent=2)}

Please reply in the following JSON format:
{{
    "thinking": "<Your thinking process on which nodes are relevant to the question>",
    "node_list": ["node_id_1", "node_id_2", ..., "node_id_n"]
}}
Directly return the final JSON structure. Do not output anything else.
"""

print("\nCalling local LLM for tree search (this might take a moment)...")
tree_search_result = await call_llm(search_prompt, model="llama3") # Use llama3 or your preferred local model

# 2.2 Print retrieved nodes and reasoning process
try:
    node_map = utils.create_node_mapping(tree)
    tree_search_result_json = json.loads(tree_search_result)

    print('\nReasoning Process:')
    utils.print_wrapped(tree_search_result_json.get('thinking', 'No thinking process provided by LLM.'))

    print('\nRetrieved Nodes:')
    retrieved_node_ids = tree_search_result_json.get("node_list", [])
    if not retrieved_node_ids:
        print("No nodes retrieved by LLM.")
    for node_id_key in retrieved_node_ids:
        # The node_id from the LLM might be just '0019', need to prepend doc_id if that's how we store
        # For this dummy client, the node_ids are like "doc_id_0019"
        # Let's adjust for the dummy structure if LLM outputs just the number
        actual_node_id = f"{doc_id}_{node_id_key}" if '_' not in node_id_key else node_id_key

        node = node_map.get(actual_node_id)
        if node:
            print(f"Node ID: {node['node_id']}\t Page: {node.get('page_index', 'N/A')}\t Title: {node['title']}")
        else:
            print(f"Node ID: {actual_node_id} (Not found in map - LLM might have hallucinated or ID format mismatch)")
except json.JSONDecodeError:
    print(f"\nError: LLM did not return valid JSON for tree search result:\n{tree_search_result}")
except Exception as e:
    print(f"\nAn error occurred during tree search result processing: {e}")


# 3.1 Extract relevant context from retrieved nodes
# Use the node_map to get the full text of the identified nodes
# Re-parse LLM result in case of error in previous block
try:
    retrieved_node_ids = json.loads(tree_search_result).get("node_list", [])
except json.JSONDecodeError:
    retrieved_node_ids = []

relevant_content = []
for node_id_key in retrieved_node_ids:
    actual_node_id = f"{doc_id}_{node_id_key}" if '_' not in node_id_key else node_id_key
    node = node_map.get(actual_node_id)
    if node and 'text' in node:
        relevant_content.append(node['text'])

relevant_content_str = "\n\n".join(relevant_content)

print('\nRetrieved Context:\n')
if relevant_content_str:
    utils.print_wrapped(relevant_content_str[:1000] + '...' if len(relevant_content_str) > 1000 else relevant_content_str)
else:
    print("No relevant context found based on LLM's node list.")


# 3.2 Generate answer based on retrieved context
if relevant_content_str:
    answer_prompt = f"""
    Answer the question based on the context:\n\nQuestion: {query}\nContext: {relevant_content_str}\n\nProvide a clear, concise answer based only on the context provided.
    """
    print('\nCalling local LLM for answer generation...')
    answer = await call_llm(answer_prompt, model="llama3:latest") # Use llama3 or your preferred local model
    print('\nGenerated Answer:\n')
    utils.print_wrapped(answer)
else:
    print("\nCannot generate an answer as no relevant context was retrieved.")

# --- END OF COLAB CELL ---

Using dummy LocalPageIndexClient for local execution.
Attempting to fetch text from https://arxiv.org/html/2501.12948v1.html for content simulation...
Failed to extract text from arXiv HTML or save to file: 404 Client Error: Not Found for url: https://arxiv.org/html/2501.12948v1.html
Falling back to a very minimal dummy text for tree generation.
Used minimal dummy text and saved to data/2501.12948
Document submitted locally: 2501_12948
Document Submitted: 2501_12948

Simplified Tree Structure of the Document:
DeepSeek-R1: Incentivizing Reasoning Cap... (ID: 2501_12948_0000)

Calling local LLM for tree search (this might take a moment)...

Reasoning Process:
The question is asking about conclusions in this document, so I will look for
nodes that have 'conclusion' or 'summary' in their title or summary.

Retrieved Nodes:
Node ID: 2501_12948_0000	 Page: 1	 Title: DeepSeek-R1: Incentivizing Reasoning Cap...

Retrieved Context:

No relevant context found based on LLM's node list.

Cannot ge

In [ ]:
    def _generate_simple_tree_from_text(self, text_content, doc_id):
        # This is a very basic way to simulate a tree from raw text for demonstration.
        # In a real PageIndex setup, this is a sophisticated process.

        tree_nodes = []
        root_node = {
            "title": "DeepSeek-R1: Incentivizing Reasoning Cap...",
            "node_id": f"{doc_id}_0000",
            "prefix_summary": "# DeepSeek-R1: Incentivizing Reasoning C...",
            "nodes": [],
            "page_index": 1 # Root starts at page 1
        }

        # Explicitly parse the minimal dummy text for distinct sections
        sections = {
            "Abstract": [],
            "1. Introduction": [],
            "1.1. Contributions": [],
            "5. Conclusion, Limitations, and Future Work": [],
        }

        current_section = None
        lines = text_content.split('\n')

        # Simple state machine to parse the dummy text
        for line in lines:
            stripped_line = line.strip()
            if stripped_line.startswith("## Abstract"):
                current_section = "Abstract"
                continue
            elif stripped_line.startswith("## 1. Introduction"):
                current_section = "1. Introduction"
                continue
            elif stripped_line.startswith("### 1.1. Contributions"):
                current_section = "1.1. Contributions"
                continue
            elif stripped_line.startswith("## 5. Conclusion, Limitations, and Future Work"):
                current_section = "5. Conclusion, Limitations, and Future Work"
                continue

            if current_section and stripped_line:
                sections[current_section].append(stripped_line)

        node_counter = 0
        node_map_for_nesting = {} # To hold references for nesting

        # Create nodes for the explicitly parsed sections
        for title_key, content_lines in sections.items():
            if not content_lines:
                continue

            node_counter += 1
            node_id = f"{doc_id}_{node_counter:04d}"
            text = "\n".join(content_lines)
            summary = text.split(' ', 20)
            summary = ' '.join(summary[:20]) + "..." if len(summary) > 20 else ' '.join(summary)

            node = {
                "title": title_key,
                "node_id": node_id,
                "summary": summary,
                "text": text,
                "page_index": 1 # Simplified page index for dummy text
            }

            # Simple nesting logic based on title prefixes
            if title_key == "Abstract":
                root_node["nodes"].append(node)
            elif title_key == "1. Introduction":
                root_node["nodes"].append(node)
                node_map_for_nesting["1. Introduction"] = node
            elif title_key == "1.1. Contributions":
                if "1. Introduction" in node_map_for_nesting:
                    if "nodes" not in node_map_for_nesting["1. Introduction"]:
                        node_map_for_nesting["1. Introduction"]["nodes"] = []
                    node_map_for_nesting["1. Introduction"]["nodes"].append(node)
                else: # Fallback to root if parent not found
                    root_node["nodes"].append(node)
            elif title_key == "5. Conclusion, Limitations, and Future Work":
                root_node["nodes"].append(node)
            else:
                root_node["nodes"].append(node)

        return [root_node] # Return as a list with the root node

In [4]:
import os
import requests
import json
from bs4 import BeautifulSoup

# Dummy PageIndexClient for local execution (no actual API calls)
class LocalPageIndexClient:
    def __init__(self, api_key=None):
        print("Using dummy LocalPageIndexClient for local execution.")
        self.documents = {} # Store document content
        self.trees = {} # Store generated tree structures

    def submit_document(self, file_path):
        doc_id = os.path.basename(file_path).replace('.', '_').replace('-', '_') # Simple doc_id generation
        with open(file_path, 'r', encoding='utf-8') as f:
            self.documents[doc_id] = f.read()
        print(f"Document submitted locally: {doc_id}")
        # In a real scenario, you'd process the document here to generate a tree.
        # For this example, we'll manually create a simplified tree for demonstration.
        self.trees[doc_id] = self._generate_simple_tree_from_text(self.documents[doc_id], doc_id)
        return {"doc_id": doc_id}

    def is_retrieval_ready(self, doc_id):
        return doc_id in self.trees

    def get_tree(self, doc_id, node_summary=True):
        return {"result": self.trees.get(doc_id, [])}


    def _generate_simple_tree_from_text(self, text_content, doc_id):
        # This is a very basic way to simulate a tree from raw text for demonstration.
        # In a real PageIndex setup, this is a sophisticated process.

        tree_nodes = []
        root_node = {
            "title": "DeepSeek-R1: Incentivizing Reasoning Cap...",
            "node_id": f"{doc_id}_0000",
            "prefix_summary": "# DeepSeek-R1: Incentivizing Reasoning C...",
            "nodes": [],
            "page_index": 1 # Root starts at page 1
        }

        # Explicitly parse the minimal dummy text for distinct sections
        sections = {
            "Abstract": [],
            "1. Introduction": [],
            "1.1. Contributions": [],
            "5. Conclusion, Limitations, and Future Work": [],
        }

        current_section = None
        lines = text_content.split('\n')

        # Simple state machine to parse the dummy text
        for line in lines:
            stripped_line = line.strip()
            if stripped_line.startswith("## Abstract"):
                current_section = "Abstract"
                continue
            elif stripped_line.startswith("## 1. Introduction"):
                current_section = "1. Introduction"
                continue
            elif stripped_line.startswith("### 1.1. Contributions"):
                current_section = "1.1. Contributions"
                continue
            elif stripped_line.startswith("## 5. Conclusion, Limitations, and Future Work"):
                current_section = "5. Conclusion, Limitations, and Future Work"
                continue

            if current_section and stripped_line:
                sections[current_section].append(stripped_line)

        node_counter = 0
        node_map_for_nesting = {} # To hold references for nesting

        # Create nodes for the explicitly parsed sections
        for title_key, content_lines in sections.items():
            if not content_lines:
                continue

            node_counter += 1
            node_id = f"{doc_id}_{node_counter:04d}"
            text = "\n".join(content_lines)
            summary = text.split(' ', 20)
            summary = ' '.join(summary[:20]) + "..." if len(summary) > 20 else ' '.join(summary)

            node = {
                "title": title_key,
                "node_id": node_id,
                "summary": summary,
                "text": text,
                "page_index": 1 # Simplified page index for dummy text
            }

            # Simple nesting logic based on title prefixes
            if title_key == "Abstract":
                root_node["nodes"].append(node)
            elif title_key == "1. Introduction":
                root_node["nodes"].append(node)
                node_map_for_nesting["1. Introduction"] = node
            elif title_key == "1.1. Contributions":
                if "1. Introduction" in node_map_for_nesting:
                    if "nodes" not in node_map_for_nesting["1. Introduction"]:
                        node_map_for_nesting["1. Introduction"]["nodes"] = []
                    node_map_for_nesting["1. Introduction"]["nodes"].append(node)
                else: # Fallback to root if parent not found
                    root_node["nodes"].append(node)
            elif title_key == "5. Conclusion, Limitations, and Future Work":
                root_node["nodes"].append(node)
            else:
                root_node["nodes"].append(node)

        return [root_node] # Return as a list with the root node

# Dummy utils for local execution (mimicking pageindex.utils)
class LocalUtils:
    @staticmethod
    def print_tree(tree, indent=0):
        for node in tree:
            prefix = "  " * indent
            print(f"{prefix}{node.get('title', 'Untitled')} (ID: {node.get('node_id', 'N/A')})")
            if "nodes" in node:
                LocalUtils.print_tree(node["nodes"], indent + 1)

    @staticmethod
    def remove_fields(obj, fields):
        if isinstance(obj, dict):
            return {k: LocalUtils.remove_fields(v, fields) for k, v in obj.items() if k not in fields}
        elif isinstance(obj, list):
            return [LocalUtils.remove_fields(elem, fields) for elem in obj]
        else:
            return obj

    @staticmethod
    def create_node_mapping(tree, node_map=None):
        if node_map is None:
            node_map = {}
        for node in tree:
            node_map[node.get('node_id')] = node
            if "nodes" in node:
                LocalUtils.create_node_mapping(node["nodes"], node_map)
        return node_map

    @staticmethod
    def print_wrapped(text, width=80):
        import textwrap
        print(textwrap.fill(text, width=width))

pi_client = LocalPageIndexClient()
utils = LocalUtils()

# Ollama LLM client for local inference
import httpx

async def call_llm(prompt, model="llama3:latest", temperature=0):
    # Ensure Ollama server is running and model is pulled
    # Example: ollama run llama3
    client = httpx.AsyncClient(base_url="http://localhost:11434")
    try:
        response = await client.post(
            "/api/chat",
            json={
                "model": model,
                "messages": [{"role": "user", "content": prompt}],
                "stream": False,
                "options": {"temperature": temperature}
            },
            timeout=None # Disable timeout for potentially long responses
        )
        response.raise_for_status() # Raise an exception for HTTP errors
        return response.json()["message"]["content"].strip()
    except httpx.ConnectError as e:
        print(f"Error connecting to Ollama: {e}")
        print("Please ensure Ollama is running (e.g., 'ollama serve') and you have pulled the model (e.g., 'ollama pull llama3').")
        return "ERROR: Could not connect to Ollama. Please check your Ollama setup."
    except httpx.HTTPStatusError as e:
        print(f"Error from Ollama server: {e.response.status_code} - {e.response.text}")
        return f"ERROR: Ollama server responded with an error: {e.response.status_code}"
    except json.JSONDecodeError:
        print(f"Error decoding JSON from Ollama: {response.text}")
        return "ERROR: Invalid JSON response from Ollama."
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return f"ERROR: An unexpected error occurred: {e}"

# 1.1 Submit a document for generating PageIndex tree (locally)
pdf_url = "https://arxiv.org/pdf/2501.12948"
pdf_filename = pdf_url.split('/')[-1]
pdf_path = os.path.join("data", pdf_filename) # Use a 'data' directory

os.makedirs(os.path.dirname(pdf_path), exist_ok=True)

# Download the PDF content
print(f"Downloading {pdf_url}...")
response = requests.get(pdf_url)
response.raise_for_status() # Check for request errors

# Convert PDF content to text (simplified for local demo)
# In a real PageIndex client, this would involve robust PDF parsing.
# For this example, we'll try to get text from an existing HTML version if possible
# or just note that a real PDF parser is needed.
# Since the original notebook downloaded a PDF and then submitted it, we'll simulate
# having the text content. A full PDF-to-text conversion is outside this script's scope,
# but for demonstration, we'll assume we have the text content extracted.

# For a true local PDF text extraction, you'd use libraries like PyPDF2, pdfminer.six, or pypopper.
# Since we are focusing on *how* to integrate with Ollama once text is available,
# let's use a placeholder.
# For the purpose of making this runnable, let's create a dummy text file
# representing the content of the PDF.

# Fetch the abstract and intro from arXiv for a more realistic text
arxiv_id = pdf_filename.replace('.pdf', '')
arxiv_html_url = f"https://arxiv.org/html/{arxiv_id}v1.html" # Assuming v1 exists

print(f"Attempting to fetch text from {arxiv_html_url} for content simulation...")
try:
    html_response = requests.get(arxiv_html_url)
    html_response.raise_for_status()
    soup = BeautifulSoup(html_response.text, 'html.parser')

    # Extract main content - this will be highly dependent on arXiv's HTML structure
    # A more robust solution would be to use a PDF parser.
    # For now, let's grab all paragraphs and headings.
    extracted_text_parts = []
    for tag in soup.find_all(['h1', 'h2', 'h3', 'p', 'li']):
        text = tag.get_text(separator=' ', strip=True)
        if text:
            extracted_text_parts.append(text)

    dummy_text_content = "\n\n".join(extracted_text_parts)
    if not dummy_text_content.strip():
        raise ValueError("Could not extract significant text from arXiv HTML.")

    text_path = pdf_path.replace('.pdf', '.txt')
    with open(text_path, "w", encoding="utf-8") as f:
        f.write(dummy_text_content)
    print(f"Extracted content to {text_path}")
    doc_id = pi_client.submit_document(text_path)["doc_id"]
    print('Document Submitted:', doc_id)

except Exception as e:
    print(f"Failed to extract text from arXiv HTML or save to file: {e}")
    print("Falling back to a very minimal dummy text for tree generation.")
    dummy_text_content = """
    # DeepSeek-R1: Incentivizing Reasoning Capability of Large Language Models
    ## Abstract
    This paper introduces DeepSeek-R1-Zero and DeepSeek-R1, two reinforcement learning (RL) based approaches to enhance reasoning abilities in Large Language Models (LLMs). DeepSeek-R1-Zero uses a pure RL approach without cold-start data, achieving strong performance. DeepSeek-R1 further improves by leveraging cold-start data and iterative RL fine-tuning, reaching performance comparable to OpenAI-o1-1217 on various tasks.
    ## 1. Introduction
    Recent advancements in LLMs have shown impressive reasoning capabilities. This work explores methods to further improve these by directly optimizing for reasoning with RL.
    ### 1.1. Contributions
    We propose novel RL algorithms and a distillation process.
    ## 5. Conclusion, Limitations, and Future Work
    We conclude that RL is effective for enhancing reasoning. DeepSeek-R1 shows state-of-the-art results. Distillation to smaller models is promising. Future work involves scaling and exploring new reward functions.
    """
    text_path = pdf_path.replace('.pdf', '.txt')
    with open(text_path, "w", encoding="utf-8") as f:
        f.write(dummy_text_content)
    print(f"Used minimal dummy text and saved to {text_path}")
    doc_id = pi_client.submit_document(text_path)["doc_id"]
    print('Document Submitted:', doc_id)


# 1.2 Get the generated PageIndex tree structure
# This will use the simplified tree generated by our LocalPageIndexClient
if pi_client.is_retrieval_ready(doc_id):
    tree = pi_client.get_tree(doc_id, node_summary=True)['result']
    print('\nSimplified Tree Structure of the Document:')
    utils.print_tree(tree)
else:
    print("Processing document, please try again later...")

# 2.1 Use LLM for tree search and identify nodes that might contain relevant context
query = "What are the conclusions in this document?"

# Remove the 'text' field to avoid sending too much data to the LLM for tree search
tree_without_text = utils.remove_fields(tree.copy(), fields=['text'])

search_prompt = f"""
You are given a question and a tree structure of a document.
Each node contains a node id, node title, and a corresponding summary.
Your task is to find all nodes that are likely to contain the answer to the question.

Question: {query}

Document tree structure:
{json.dumps(tree_without_text, indent=2)}

Please reply in the following JSON format:
{{
    "thinking": "<Your thinking process on which nodes are relevant to the question>",
    "node_list": ["node_id_1", "node_id_2", ..., "node_id_n"]
}}
Directly return the final JSON structure. Do not output anything else.
"""

print("\nCalling local LLM for tree search (this might take a moment)...")
tree_search_result = await call_llm(search_prompt, model="llama3") # Use llama3 or your preferred local model

# 2.2 Print retrieved nodes and reasoning process
try:
    node_map = utils.create_node_mapping(tree)
    tree_search_result_json = json.loads(tree_search_result)

    print('\nReasoning Process:')
    utils.print_wrapped(tree_search_result_json.get('thinking', 'No thinking process provided by LLM.'))

    print('\nRetrieved Nodes:')
    retrieved_node_ids = tree_search_result_json.get("node_list", [])
    if not retrieved_node_ids:
        print("No nodes retrieved by LLM.")
    for node_id_key in retrieved_node_ids:
        # The node_id from the LLM might be just '0019', need to prepend doc_id if that's how we store
        # For this dummy client, the node_ids are like "doc_id_0019"
        # Let's adjust for the dummy structure if LLM outputs just the number
        actual_node_id = f"{doc_id}_{node_id_key}" if '_' not in node_id_key else node_id_key

        node = node_map.get(actual_node_id)
        if node:
            print(f"Node ID: {node['node_id']}\t Page: {node.get('page_index', 'N/A')}\t Title: {node['title']}")
        else:
            print(f"Node ID: {actual_node_id} (Not found in map - LLM might have hallucinated or ID format mismatch)")
except json.JSONDecodeError:
    print(f"\nError: LLM did not return valid JSON for tree search result:\n{tree_search_result}")
except Exception as e:
    print(f"\nAn error occurred during tree search result processing: {e}")


# 3.1 Extract relevant context from retrieved nodes
# Use the node_map to get the full text of the identified nodes
# Re-parse LLM result in case of error in previous block
try:
    retrieved_node_ids = json.loads(tree_search_result).get("node_list", [])
except json.JSONDecodeError:
    retrieved_node_ids = []

relevant_content = []
for node_id_key in retrieved_node_ids:
    actual_node_id = f"{doc_id}_{node_id_key}" if '_' not in node_id_key else node_id_key
    node = node_map.get(actual_node_id)
    if node and 'text' in node:
        relevant_content.append(node['text'])

relevant_content_str = "\n\n".join(relevant_content)

print('\nRetrieved Context:\n')
if relevant_content_str:
    utils.print_wrapped(relevant_content_str[:1000] + '...' if len(relevant_content_str) > 1000 else relevant_content_str)
else:
    print("No relevant context found based on LLM's node list.")


# 3.2 Generate answer based on retrieved context
if relevant_content_str:
    answer_prompt = f"""
    Answer the question based on the context:\n\nQuestion: {query}\nContext: {relevant_content_str}\n\nProvide a clear, concise answer based only on the context provided.
    """
    print('\nCalling local LLM for answer generation...')
    answer = await call_llm(answer_prompt, model="llama3:latest") # Use llama3 or your preferred local model
    print('\nGenerated Answer:\n')
    utils.print_wrapped(answer)
else:
    print("\nCannot generate an answer as no relevant context was retrieved.")

# --- END OF COLAB CELL ---

Using dummy LocalPageIndexClient for local execution.
Attempting to fetch text from https://arxiv.org/html/2501.12948v1.html for content simulation...
Failed to extract text from arXiv HTML or save to file: 404 Client Error: Not Found for url: https://arxiv.org/html/2501.12948v1.html
Falling back to a very minimal dummy text for tree generation.
Used minimal dummy text and saved to data/2501.12948
Document submitted locally: 2501_12948
Document Submitted: 2501_12948

Simplified Tree Structure of the Document:
DeepSeek-R1: Incentivizing Reasoning Cap... (ID: 2501_12948_0000)
  Abstract (ID: 2501_12948_0001)
  1. Introduction (ID: 2501_12948_0002)
    1.1. Contributions (ID: 2501_12948_0003)
  5. Conclusion, Limitations, and Future Work (ID: 2501_12948_0004)

Calling local LLM for tree search (this might take a moment)...

Reasoning Process:
The question is asking for conclusions in this document, so I will look for
nodes that have 'Conclusion' or 'Summary' in their title and also contai

In [ ]:
/content/data/2501.12948



    # DeepSeek-R1: Incentivizing Reasoning Capability of Large Language Models
    ## Abstract
    This paper introduces DeepSeek-R1-Zero and DeepSeek-R1, two reinforcement learning (RL) based approaches to enhance reasoning abilities in Large Language Models (LLMs). DeepSeek-R1-Zero uses a pure RL approach without cold-start data, achieving strong performance. DeepSeek-R1 further improves by leveraging cold-start data and iterative RL fine-tuning, reaching performance comparable to OpenAI-o1-1217 on various tasks.
    ## 1. Introduction
    Recent advancements in LLMs have shown impressive reasoning capabilities. This work explores methods to further improve these by directly optimizing for reasoning with RL.
    ### 1.1. Contributions
    We propose novel RL algorithms and a distillation process.
    ## 5. Conclusion, Limitations, and Future Work
    We conclude that RL is effective for enhancing reasoning. DeepSeek-R1 shows state-of-the-art results. Distillation to smaller models is promising. Future work involves scaling and exploring new reward functions.


In [ ]:
https://arxiv.org/html/2508.21069v1

In [ ]:
import os
import requests
import json
from bs4 import BeautifulSoup

# Dummy PageIndexClient for local execution (no actual API calls)
class LocalPageIndexClient:
    def __init__(self, api_key=None):
        print("Using dummy LocalPageIndexClient for local execution.")
        self.documents = {} # Store document content
        self.trees = {} # Store generated tree structures

    def submit_document(self, file_path):
        doc_id = os.path.basename(file_path).replace('.', '_').replace('-', '_') # Simple doc_id generation
        with open(file_path, 'r', encoding='utf-8') as f:
            self.documents[doc_id] = f.read()
        print(f"Document submitted locally: {doc_id}")
        # In a real scenario, you'd process the document here to generate a tree.
        # For this example, we'll manually create a simplified tree for demonstration.
        self.trees[doc_id] = self._generate_simple_tree_from_text(self.documents[doc_id], doc_id)
        return {"doc_id": doc_id}

    def is_retrieval_ready(self, doc_id):
        return doc_id in self.trees

    def get_tree(self, doc_id, node_summary=True):
        return {"result": self.trees.get(doc_id, [])}


    def _generate_simple_tree_from_text(self, text_content, doc_id):
        # This is a very basic way to simulate a tree from raw text for demonstration.
        # In a real PageIndex setup, this is a sophisticated process.

        tree_nodes = []
        root_node = {
            "title": "DeepSeek-R1: Incentivizing Reasoning Cap...",
            "node_id": f"{doc_id}_0000",
            "prefix_summary": "# DeepSeek-R1: Incentivizing Reasoning C...",
            "nodes": [],
            "page_index": 1 # Root starts at page 1
        }

        # Explicitly parse the minimal dummy text for distinct sections
        sections = {
            "Abstract": [],
            "1. Introduction": [],
            "1.1. Contributions": [],
            "5. Conclusion, Limitations, and Future Work": [],
        }

        current_section = None
        lines = text_content.split('\n')

        # Simple state machine to parse the dummy text
        for line in lines:
            stripped_line = line.strip()
            if stripped_line.startswith("## Abstract"):
                current_section = "Abstract"
                continue
            elif stripped_line.startswith("## 1. Introduction"):
                current_section = "1. Introduction"
                continue
            elif stripped_line.startswith("### 1.1. Contributions"):
                current_section = "1.1. Contributions"
                continue
            elif stripped_line.startswith("## 5. Conclusion, Limitations, and Future Work"):
                current_section = "5. Conclusion, Limitations, and Future Work"
                continue

            if current_section and stripped_line:
                sections[current_section].append(stripped_line)

        node_counter = 0
        node_map_for_nesting = {} # To hold references for nesting

        # Create nodes for the explicitly parsed sections
        for title_key, content_lines in sections.items():
            if not content_lines:
                continue

            node_counter += 1
            node_id = f"{doc_id}_{node_counter:04d}"
            text = "\n".join(content_lines)
            summary = text.split(' ', 20)
            summary = ' '.join(summary[:20]) + "..." if len(summary) > 20 else ' '.join(summary)

            node = {
                "title": title_key,
                "node_id": node_id,
                "summary": summary,
                "text": text,
                "page_index": 1 # Simplified page index for dummy text
            }

            # Simple nesting logic based on title prefixes
            if title_key == "Abstract":
                root_node["nodes"].append(node)
            elif title_key == "1. Introduction":
                root_node["nodes"].append(node)
                node_map_for_nesting["1. Introduction"] = node
            elif title_key == "1.1. Contributions":
                if "1. Introduction" in node_map_for_nesting:
                    if "nodes" not in node_map_for_nesting["1. Introduction"]:
                        node_map_for_nesting["1. Introduction"]["nodes"] = []
                    node_map_for_nesting["1. Introduction"]["nodes"].append(node)
                else: # Fallback to root if parent not found
                    root_node["nodes"].append(node)
            elif title_key == "5. Conclusion, Limitations, and Future Work":
                root_node["nodes"].append(node)
            else:
                root_node["nodes"].append(node)

        return [root_node] # Return as a list with the root node

# Dummy utils for local execution (mimicking pageindex.utils)
class LocalUtils:
    @staticmethod
    def print_tree(tree, indent=0):
        for node in tree:
            prefix = "  " * indent
            print(f"{prefix}{node.get('title', 'Untitled')} (ID: {node.get('node_id', 'N/A')})")
            if "nodes" in node:
                LocalUtils.print_tree(node["nodes"], indent + 1)

    @staticmethod
    def remove_fields(obj, fields):
        if isinstance(obj, dict):
            return {k: LocalUtils.remove_fields(v, fields) for k, v in obj.items() if k not in fields}
        elif isinstance(obj, list):
            return [LocalUtils.remove_fields(elem, fields) for elem in obj]
        else:
            return obj

    @staticmethod
    def create_node_mapping(tree, node_map=None):
        if node_map is None:
            node_map = {}
        for node in tree:
            node_map[node.get('node_id')] = node
            if "nodes" in node:
                LocalUtils.create_node_mapping(node["nodes"], node_map)
        return node_map

    @staticmethod
    def print_wrapped(text, width=80):
        import textwrap
        print(textwrap.fill(text, width=width))

pi_client = LocalPageIndexClient()
utils = LocalUtils()

# Ollama LLM client for local inference
import httpx

async def call_llm(prompt, model="llama3:latest", temperature=0):
    # Ensure Ollama server is running and model is pulled
    # Example: ollama run llama3
    client = httpx.AsyncClient(base_url="http://localhost:11434")
    try:
        response = await client.post(
            "/api/chat",
            json={
                "model": model,
                "messages": [{"role": "user", "content": prompt}],
                "stream": False,
                "options": {"temperature": temperature}
            },
            timeout=None # Disable timeout for potentially long responses
        )
        response.raise_for_status() # Raise an exception for HTTP errors
        return response.json()["message"]["content"].strip()
    except httpx.ConnectError as e:
        print(f"Error connecting to Ollama: {e}")
        print("Please ensure Ollama is running (e.g., 'ollama serve') and you have pulled the model (e.g., 'ollama pull llama3').")
        return "ERROR: Could not connect to Ollama. Please check your Ollama setup."
    except httpx.HTTPStatusError as e:
        print(f"Error from Ollama server: {e.response.status_code} - {e.response.text}")
        return f"ERROR: Ollama server responded with an error: {e.response.status_code}"
    except json.JSONDecodeError:
        print(f"Error decoding JSON from Ollama: {response.text}")
        return "ERROR: Invalid JSON response from Ollama."
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return f"ERROR: An unexpected error occurred: {e}"

# 1.1 Submit a document for generating PageIndex tree (locally)
pdf_url = "https://arxiv.org/pdf/2501.12948"
pdf_filename = pdf_url.split('/')[-1]
pdf_path = os.path.join("data", pdf_filename) # Use a 'data' directory

os.makedirs(os.path.dirname(pdf_path), exist_ok=True)

# Download the PDF content
print(f"Downloading {pdf_url}...")
response = requests.get(pdf_url)
response.raise_for_status() # Check for request errors

# Convert PDF content to text (simplified for local demo)
# In a real PageIndex client, this would involve robust PDF parsing.
# For this example, we'll try to get text from an existing HTML version if possible
# or just note that a real PDF parser is needed.
# Since the original notebook downloaded a PDF and then submitted it, we'll simulate
# having the text content. A full PDF-to-text conversion is outside this script's scope,
# but for demonstration, we'll assume we have the text content extracted.

# For a true local PDF text extraction, you'd use libraries like PyPDF2, pdfminer.six, or pypopper.
# Since we are focusing on *how* to integrate with Ollama once text is available,
# let's use a placeholder.
# For the purpose of making this runnable, let's create a dummy text file
# representing the content of the PDF.

# Fetch the abstract and intro from arXiv for a more realistic text
arxiv_id = pdf_filename.replace('.pdf', '')
arxiv_html_url = f"https://arxiv.org/html/{arxiv_id}v1.html" # Assuming v1 exists

print(f"Attempting to fetch text from {arxiv_html_url} for content simulation...")
try:
    html_response = requests.get(arxiv_html_url)
    html_response.raise_for_status()
    soup = BeautifulSoup(html_response.text, 'html.parser')

    # Extract main content - this will be highly dependent on arXiv's HTML structure
    # A more robust solution would be to use a PDF parser.
    # For now, let's grab all paragraphs and headings.
    extracted_text_parts = []
    for tag in soup.find_all(['h1', 'h2', 'h3', 'p', 'li']):
        text = tag.get_text(separator=' ', strip=True)
        if text:
            extracted_text_parts.append(text)

    dummy_text_content = "\n\n".join(extracted_text_parts)
    if not dummy_text_content.strip():
        raise ValueError("Could not extract significant text from arXiv HTML.")

    text_path = pdf_path.replace('.pdf', '.txt')
    with open(text_path, "w", encoding="utf-8") as f:
        f.write(dummy_text_content)
    print(f"Extracted content to {text_path}")
    doc_id = pi_client.submit_document(text_path)["doc_id"]
    print('Document Submitted:', doc_id)

except Exception as e:
    print(f"Failed to extract text from arXiv HTML or save to file: {e}")
    print("Falling back to a very minimal dummy text for tree generation.")
    dummy_text_content = """
    # DeepSeek-R1: Incentivizing Reasoning Capability of Large Language Models
    ## Abstract
    This paper introduces DeepSeek-R1-Zero and DeepSeek-R1, two reinforcement learning (RL) based approaches to enhance reasoning abilities in Large Language Models (LLMs). DeepSeek-R1-Zero uses a pure RL approach without cold-start data, achieving strong performance. DeepSeek-R1 further improves by leveraging cold-start data and iterative RL fine-tuning, reaching performance comparable to OpenAI-o1-1217 on various tasks.
    ## 1. Introduction
    Recent advancements in LLMs have shown impressive reasoning capabilities. This work explores methods to further improve these by directly optimizing for reasoning with RL.
    ### 1.1. Contributions
    We propose novel RL algorithms and a distillation process.
    ## 5. Conclusion, Limitations, and Future Work
    We conclude that RL is effective for enhancing reasoning. DeepSeek-R1 shows state-of-the-art results. Distillation to smaller models is promising. Future work involves scaling and exploring new reward functions.
    """
    text_path = pdf_path.replace('.pdf', '.txt')
    with open(text_path, "w", encoding="utf-8") as f:
        f.write(dummy_text_content)
    print(f"Used minimal dummy text and saved to {text_path}")
    doc_id = pi_client.submit_document(text_path)["doc_id"]
    print('Document Submitted:', doc_id)


# 1.2 Get the generated PageIndex tree structure
# This will use the simplified tree generated by our LocalPageIndexClient
if pi_client.is_retrieval_ready(doc_id):
    tree = pi_client.get_tree(doc_id, node_summary=True)['result']
    print('\nSimplified Tree Structure of the Document:')
    utils.print_tree(tree)
else:
    print("Processing document, please try again later...")

# 2.1 Use LLM for tree search and identify nodes that might contain relevant context
query = "What are the conclusions in this document?"

# Remove the 'text' field to avoid sending too much data to the LLM for tree search
tree_without_text = utils.remove_fields(tree.copy(), fields=['text'])

search_prompt = f"""
You are given a question and a tree structure of a document.
Each node contains a node id, node title, and a corresponding summary.
Your task is to find all nodes that are likely to contain the answer to the question.

Question: {query}

Document tree structure:
{json.dumps(tree_without_text, indent=2)}

Please reply in the following JSON format:
{{
    "thinking": "<Your thinking process on which nodes are relevant to the question>",
    "node_list": ["node_id_1", "node_id_2", ..., "node_id_n"]
}}
Directly return the final JSON structure. Do not output anything else.
"""

print("\nCalling local LLM for tree search (this might take a moment)...")
tree_search_result = await call_llm(search_prompt, model="llama3") # Use llama3 or your preferred local model

# 2.2 Print retrieved nodes and reasoning process
try:
    node_map = utils.create_node_mapping(tree)
    tree_search_result_json = json.loads(tree_search_result)

    print('\nReasoning Process:')
    utils.print_wrapped(tree_search_result_json.get('thinking', 'No thinking process provided by LLM.'))

    print('\nRetrieved Nodes:')
    retrieved_node_ids = tree_search_result_json.get("node_list", [])
    if not retrieved_node_ids:
        print("No nodes retrieved by LLM.")
    for node_id_key in retrieved_node_ids:
        # The node_id from the LLM might be just '0019', need to prepend doc_id if that's how we store
        # For this dummy client, the node_ids are like "doc_id_0019"
        # Let's adjust for the dummy structure if LLM outputs just the number
        actual_node_id = f"{doc_id}_{node_id_key}" if '_' not in node_id_key else node_id_key

        node = node_map.get(actual_node_id)
        if node:
            print(f"Node ID: {node['node_id']}\t Page: {node.get('page_index', 'N/A')}\t Title: {node['title']}")
        else:
            print(f"Node ID: {actual_node_id} (Not found in map - LLM might have hallucinated or ID format mismatch)")
except json.JSONDecodeError:
    print(f"\nError: LLM did not return valid JSON for tree search result:\n{tree_search_result}")
except Exception as e:
    print(f"\nAn error occurred during tree search result processing: {e}")


# 3.1 Extract relevant context from retrieved nodes
# Use the node_map to get the full text of the identified nodes
# Re-parse LLM result in case of error in previous block
try:
    retrieved_node_ids = json.loads(tree_search_result).get("node_list", [])
except json.JSONDecodeError:
    retrieved_node_ids = []

relevant_content = []
for node_id_key in retrieved_node_ids:
    actual_node_id = f"{doc_id}_{node_id_key}" if '_' not in node_id_key else node_id_key
    node = node_map.get(actual_node_id)
    if node and 'text' in node:
        relevant_content.append(node['text'])

relevant_content_str = "\n\n".join(relevant_content)

print('\nRetrieved Context:\n')
if relevant_content_str:
    utils.print_wrapped(relevant_content_str[:1000] + '...' if len(relevant_content_str) > 1000 else relevant_content_str)
else:
    print("No relevant context found based on LLM's node list.")


# 3.2 Generate answer based on retrieved context
if relevant_content_str:
    answer_prompt = f"""
    Answer the question based on the context:\n\nQuestion: {query}\nContext: {relevant_content_str}\n\nProvide a clear, concise answer based only on the context provided.
    """
    print('\nCalling local LLM for answer generation...')
    answer = await call_llm(answer_prompt, model="llama3:latest") # Use llama3 or your preferred local model
    print('\nGenerated Answer:\n')
    utils.print_wrapped(answer)
else:
    print("\nCannot generate an answer as no relevant context was retrieved.")

# --- END OF COLAB CELL ---

### فشل الرابط واستخدم كتاب وهمى

In [5]:
# --- START OF COLAB CELL ---

# 0.0 Setup Environment and Install Dependencies
# هذا الكود يقوم بتثبيت الحزم الضرورية وتهيئة البيئة.

# تثبيت PageIndex (إذا لم تكن مثبتة بالفعل)
%pip install -q --upgrade pageindex
%pip install -q --upgrade openai  # للاستخدام المستقبلي المحتمل أو خوادم OpenAI المتوافقة محليًا
%pip install -q requests beautifulsoup4  # لتنزيل المستندات وتحليلها

import os
import requests
import json
from bs4 import BeautifulSoup

# Dummy PageIndexClient للتنفيذ المحلي (لا توجد استدعاءات API فعلية)
class LocalPageIndexClient:
    def __init__(self, api_key=None):
        print("Using dummy LocalPageIndexClient for local execution.")
        self.documents = {} # تخزين محتوى المستند
        self.trees = {} # تخزين هياكل الشجرة التي تم إنشاؤها

    def submit_document(self, file_path):
        doc_id = os.path.basename(file_path).replace('.', '_').replace('-', '_') # توليد معرف مستند بسيط
        with open(file_path, 'r', encoding='utf-8') as f:
            self.documents[doc_id] = f.read()
        print(f"Document submitted locally: {doc_id}")
        # في سيناريو حقيقي، ستقوم بمعالجة المستند هنا لإنشاء شجرة.
        # لهذا المثال، سنقوم بإنشاء شجرة مبسطة يدويًا للتوضيح.
        self.trees[doc_id] = self._generate_simple_tree_from_text(self.documents[doc_id], doc_id)
        return {"doc_id": doc_id}

    def is_retrieval_ready(self, doc_id):
        return doc_id in self.trees

    def get_tree(self, doc_id, node_summary=True):
        return {"result": self.trees.get(doc_id, [])}

    def _generate_simple_tree_from_text(self, text_content, doc_id):
        # هذه طريقة أساسية جدًا لمحاكاة شجرة من نص خام للتوضيح.
        # في إعداد PageIndex حقيقي، هذه عملية معقدة.

        tree_nodes = []
        root_node = {
            "title": "DeepSeek-R1: Incentivizing Reasoning Cap...",
            "node_id": f"{doc_id}_0000",
            "prefix_summary": "# DeepSeek-R1: Incentivizing Reasoning C...",
            "nodes": [],
            "page_index": 1 # العقدة الجذرية تبدأ من الصفحة 1
        }

        # تحليل النص الوهمي البسيط بشكل صريح لأقسام مميزة
        sections = {
            "Abstract": [],
            "1. Introduction": [],
            "1.1. Contributions": [],
            "5. Conclusion, Limitations, and Future Work": [],
        }

        current_section = None
        lines = text_content.split('\n')

        # آلة حالة بسيطة لتحليل النص الوهمي
        for line in lines:
            stripped_line = line.strip()
            if stripped_line.startswith("## Abstract"):
                current_section = "Abstract"
                continue
            elif stripped_line.startswith("## 1. Introduction"):
                current_section = "1. Introduction"
                continue
            elif stripped_line.startswith("### 1.1. Contributions"):
                current_section = "1.1. Contributions"
                continue
            elif stripped_line.startswith("## 5. Conclusion, Limitations, and Future Work"):
                current_section = "5. Conclusion, Limitations, and Future Work"
                continue

            if current_section and stripped_line:
                sections[current_section].append(stripped_line)

        node_counter = 0
        node_map_for_nesting = {} # لتخزين المراجع من أجل التعشيش

        # إنشاء عقد للأقسام التي تم تحليلها بشكل صريح
        for title_key, content_lines in sections.items():
            if not content_lines:
                continue

            node_counter += 1
            # لجعل الـ node_id متوافقًا مع التوقع في اللوغ، استخدم تسلسل بسيط
            if title_key == "Abstract": node_id_suffix = "0001"
            elif title_key == "1. Introduction": node_id_suffix = "0003"
            elif title_key == "1.1. Contributions": node_id_suffix = "0004"
            elif title_key == "5. Conclusion, Limitations, and Future Work": node_id_suffix = "0019"
            else: node_id_suffix = f"{node_counter:04d}" # fallback

            node_id = f"{doc_id}_{node_id_suffix}"
            text = "\n".join(content_lines)
            summary = text.split(' ', 20)
            summary = ' '.join(summary[:20]) + "..." if len(summary) > 20 else ' '.join(summary)

            node = {
                "title": title_key,
                "node_id": node_id,
                "summary": summary,
                "text": text,
                "page_index": 1 # فهرس صفحة مبسط للنص الوهمي
            }

            # منطق تعشيش بسيط يعتمد على بادئات العناوين
            if title_key == "Abstract":
                root_node["nodes"].append(node)
            elif title_key == "1. Introduction":
                root_node["nodes"].append(node)
                node_map_for_nesting["1. Introduction"] = node
            elif title_key == "1.1. Contributions":
                if "1. Introduction" in node_map_for_nesting:
                    if "nodes" not in node_map_for_nesting["1. Introduction"]:
                        node_map_for_nesting["1. Introduction"]["nodes"] = []
                    node_map_for_nesting["1. Introduction"]["nodes"].append(node)
                else: # Fallback إلى الجذر إذا لم يتم العثور على الأصل
                    root_node["nodes"].append(node)
            elif title_key == "5. Conclusion, Limitations, and Future Work":
                root_node["nodes"].append(node)
            else:
                root_node["nodes"].append(node)

        return [root_node] # إرجاع كقائمة مع العقدة الجذرية

# أدوات وهمية للتنفيذ المحلي (تحاكي pageindex.utils)
class LocalUtils:
    @staticmethod
    def print_tree(tree, indent=0):
        for node in tree:
            prefix = "  " * indent
            print(f"{prefix}{node.get('title', 'Untitled')} (ID: {node.get('node_id', 'N/A')})")
            if "nodes" in node:
                LocalUtils.print_tree(node["nodes"], indent + 1)

    @staticmethod
    def remove_fields(obj, fields):
        if isinstance(obj, dict):
            return {k: LocalUtils.remove_fields(v, fields) for k, v in obj.items() if k not in fields}
        elif isinstance(obj, list):
            return [LocalUtils.remove_fields(elem, fields) for elem in obj]
        else:
            return obj

    @staticmethod
    def create_node_mapping(tree, node_map=None):
        if node_map is None:
            node_map = {}
        for node in tree:
            node_map[node.get('node_id')] = node
            if "nodes" in node:
                LocalUtils.create_node_mapping(node["nodes"], node_map)
        return node_map

    @staticmethod
    def print_wrapped(text, width=80):
        import textwrap
        print(textwrap.fill(text, width=width))

pi_client = LocalPageIndexClient()
utils = LocalUtils()

# عميل Ollama LLM للاستدلال المحلي
import httpx

async def call_llm(prompt, model="llama3", temperature=0):
    # تأكد من أن خادم Ollama يعمل وأن النموذج محمل
    # مثال: ollama run llama3
    client = httpx.AsyncClient(base_url="http://localhost:11434")
    try:
        response = await client.post(
            "/api/chat",
            json={
                "model": model,
                "messages": [{"role": "user", "content": prompt}],
                "stream": False,
                "options": {"temperature": temperature}
            },
            timeout=None # تعطيل المهلة للاستجابات التي قد تكون طويلة
        )
        response.raise_for_status() # رفع استثناء لأخطاء HTTP
        return response.json()["message"]["content"].strip()
    except httpx.ConnectError as e:
        print(f"Error connecting to Ollama: {e}")
        print("Please ensure Ollama is running (e.g., 'ollama serve') and you have pulled the model (e.g., 'ollama pull llama3').")
        return "ERROR: Could not connect to Ollama. Please check your Ollama setup."
    except httpx.HTTPStatusError as e:
        print(f"Error from Ollama server: {e.response.status_code} - {e.response.text}")
        return f"ERROR: Ollama server responded with an error: {e.response.status_code}"
    except json.JSONDecodeError:
        print(f"Error decoding JSON from Ollama: {response.text}")
        return "ERROR: Invalid JSON response from Ollama."
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return f"ERROR: An unexpected error occurred: {e}"

# 1.1 إرسال مستند لإنشاء شجرة PageIndex (محليًا)
pdf_url = "https://arxiv.org/pdf/2501.12948.pdf"
pdf_filename = pdf_url.split('/')[-1]
pdf_path = os.path.join("data", pdf_filename) # استخدام مجلد 'data'

os.makedirs(os.path.dirname(pdf_path), exist_ok=True)

# تنزيل محتوى PDF
print(f"Downloading {pdf_url}...")
response = requests.get(pdf_url)
response.raise_for_status() # التحقق من أخطاء الطلب

# تحويل محتوى PDF إلى نص (مبسط للعرض المحلي)
# في عميل PageIndex الحقيقي، سيتضمن ذلك تحليلًا قويًا لملف PDF.
# لهذا المثال، سنحاول الحصول على نص من إصدار HTML موجود إن أمكن
# أو فقط نلاحظ أن هناك حاجة إلى محلل PDF حقيقي.
# بما أن الدفتر الأصلي قام بتنزيل ملف PDF ثم إرساله، سنحاكي
# وجود محتوى النص. تحويل PDF إلى نص كامل خارج نطاق هذا السكريبت،
# ولكن للتوضيح، سنفترض أن لدينا محتوى النص المستخرج.

# لاستخراج نص PDF حقيقي محليًا، ستستخدم مكتبات مثل PyPDF2 أو pdfminer.six أو pypopper.
# نظرًا لأننا نركز على كيفية التكامل مع Ollama بمجرد توفر النص،
# دعونا نستخدم نصًا بديلًا.
# لجعله قابلاً للتشغيل، دعونا ننشئ ملفًا نصيًا وهميًا
# يمثل محتوى ملف PDF.

# جلب الملخص والمقدمة من arXiv للحصول على نص أكثر واقعية
arxiv_id = pdf_filename.replace('.pdf', '')
arxiv_html_url = f"https://arxiv.org/html/{arxiv_id}v1.html" # بافتراض أن v1 موجود

print(f"Attempting to fetch text from {arxiv_html_url} for content simulation...")
try:
    html_response = requests.get(arxiv_html_url)
    html_response.raise_for_status()
    soup = BeautifulSoup(html_response.text, 'html.parser')

    # استخراج المحتوى الرئيسي - سيعتمد هذا بشكل كبير على بنية HTML الخاصة بـ arXiv
    # سيكون الحل الأكثر قوة هو استخدام محلل PDF.
    # في الوقت الحالي، دعنا نلتقط جميع الفقرات والعناوين.
    extracted_text_parts = []
    for tag in soup.find_all(['h1', 'h2', 'h3', 'p', 'li']):
        text = tag.get_text(separator=' ', strip=True)
        if text:
            extracted_text_parts.append(text)

    dummy_text_content = "\n\n".join(extracted_text_parts)
    if not dummy_text_content.strip():
        raise ValueError("Could not extract significant text from arXiv HTML.")

    text_path = pdf_path.replace('.pdf', '.txt')
    with open(text_path, "w", encoding="utf-8") as f:
        f.write(dummy_text_content)
    print(f"Extracted content to {text_path}")
    doc_id = pi_client.submit_document(text_path)["doc_id"]
    print('Document Submitted:', doc_id)

except Exception as e:
    print(f"Failed to extract text from arXiv HTML or save to file: {e}")
    print("Falling back to a very minimal dummy text for tree generation.")
    dummy_text_content = """
    # DeepSeek-R1: Incentivizing Reasoning Capability of Large Language Models
    ## Abstract
    This paper introduces DeepSeek-R1-Zero and DeepSeek-R1, two reinforcement learning (RL) based approaches to enhance reasoning abilities in Large Language Models (LLMs). DeepSeek-R1-Zero uses a pure RL approach without cold-start data, achieving strong performance. DeepSeek-R1 further improves by leveraging cold-start data and iterative RL fine-tuning, reaching performance comparable to OpenAI-o1-1217 on various tasks.
    ## 1. Introduction
    Recent advancements in LLMs have shown impressive reasoning capabilities. This work explores methods to further improve these by directly optimizing for reasoning with RL.
    ### 1.1. Contributions
    We propose novel RL algorithms and a distillation process.
    ## 5. Conclusion, Limitations, and Future Work
    We conclude that RL is effective for enhancing reasoning. DeepSeek-R1 shows state-of-the-art results. Distillation to smaller models is promising. Future work involves scaling and exploring new reward functions.
    """
    text_path = pdf_path.replace('.pdf', '.txt')
    with open(text_path, "w", encoding="utf-8") as f:
        f.write(dummy_text_content)
    print(f"Used minimal dummy text and saved to {text_path}")
    doc_id = pi_client.submit_document(text_path)["doc_id"]
    print('Document Submitted:', doc_id)


# 1.2 الحصول على بنية شجرة PageIndex التي تم إنشاؤها
# سيستخدم هذا الشجرة المبسطة التي تم إنشاؤها بواسطة LocalPageIndexClient الخاص بنا
if pi_client.is_retrieval_ready(doc_id):
    tree = pi_client.get_tree(doc_id, node_summary=True)['result']
    print('\nSimplified Tree Structure of the Document:')
    utils.print_tree(tree)
else:
    print("Processing document, please try again later...")

# 2.1 استخدام LLM للبحث في الشجرة وتحديد العقد التي قد تحتوي على سياق ذي صلة
query = "What are the conclusions in this document?"

# إزالة حقل 'text' لتجنب إرسال الكثير من البيانات إلى LLM للبحث في الشجرة
tree_without_text = utils.remove_fields(tree.copy(), fields=['text'])

search_prompt = f"""
You are given a question and a tree structure of a document.
Each node contains a node id, node title, and a corresponding summary.
Your task is to find all nodes that are likely to contain the answer to the question.

Question: {query}

Document tree structure:
{json.dumps(tree_without_text, indent=2)}

Please reply in the following JSON format:
{{
    "thinking": "<Your thinking process on which nodes are relevant to the question>",
    "node_list": ["node_id_1", "node_id_2", ..., "node_id_n"]
}}
Directly return the final JSON structure. Do not output anything else.
"""

print("\nCalling local LLM for tree search (this might take a moment)...")
tree_search_result = await call_llm(search_prompt, model="llama3") # استخدم llama3 أو نموذجك المحلي المفضل

# 2.2 طباعة العقد المسترجعة وعملية التفكير
try:
    node_map = utils.create_node_mapping(tree)
    tree_search_result_json = json.loads(tree_search_result)

    print('\nReasoning Process:')
    utils.print_wrapped(tree_search_result_json.get('thinking', 'No thinking process provided by LLM.'))

    print('\nRetrieved Nodes:')
    retrieved_node_ids = tree_search_result_json.get("node_list", [])
    if not retrieved_node_ids:
        print("No nodes retrieved by LLM.")
    for node_id_key in retrieved_node_ids:
        # معرف العقدة من LLM قد يكون مجرد '0019'، نحتاج إلى إضافة doc_id إذا كانت هذه هي طريقة التخزين
        # لهذا العميل الوهمي، تكون معرفات العقد مثل "doc_id_0019"
        # دعنا نعدل لبنية الوهمية إذا كان LLM يُخرج الرقم فقط
        actual_node_id = f"{doc_id}_{node_id_key}" if '_' not in node_id_key and not node_id_key.startswith(doc_id) else node_id_key

        node = node_map.get(actual_node_id)
        if node:
            print(f"Node ID: {node['node_id']}\t Page: {node.get('page_index', 'N/A')}\t Title: {node['title']}")
        else:
            print(f"Node ID: {actual_node_id} (Not found in map - LLM might have hallucinated or ID format mismatch)")
except json.JSONDecodeError:
    print(f"\nError: LLM did not return valid JSON for tree search result:\n{tree_search_result}")
except Exception as e:
    print(f"\nAn error occurred during tree search result processing: {e}")


# 3.1 استخراج السياق ذي الصلة من العقد المسترجعة
# استخدم node_map للحصول على النص الكامل للعقد المحددة
# أعد تحليل نتيجة LLM في حالة وجود خطأ في الكتلة السابقة
try:
    retrieved_node_ids = json.loads(tree_search_result).get("node_list", [])
except json.JSONDecodeError:
    retrieved_node_ids = []

relevant_content = []
for node_id_key in retrieved_node_ids:
    actual_node_id = f"{doc_id}_{node_id_key}" if '_' not in node_id_key and not node_id_key.startswith(doc_id) else node_id_key
    node = node_map.get(actual_node_id)
    if node and 'text' in node:
        relevant_content.append(node['text'])

relevant_content_str = "\n\n".join(relevant_content)

print('\nRetrieved Context:\n')
if relevant_content_str:
    utils.print_wrapped(relevant_content_str[:1000] + '...' if len(relevant_content_str) > 1000 else relevant_content_str)
else:
    print("No relevant context found based on LLM's node list.")


# 3.2 توليد الإجابة بناءً على السياق المسترجع
if relevant_content_str:
    answer_prompt = f"""
    Answer the question based on the context:\n\nQuestion: {query}\nContext: {relevant_content_str}\n\nProvide a clear, concise answer based only on the context provided.
    """
    print('\nCalling local LLM for answer generation...')
    answer = await call_llm(answer_prompt, model="llama3") # استخدم llama3 أو نموذجك المحلي المفضل
    print('\nGenerated Answer:\n')
    utils.print_wrapped(answer)
else:
    print("\nCannot generate an answer as no relevant context was retrieved.")

# --- END OF COLAB CELL ---

Using dummy LocalPageIndexClient for local execution.
Attempting to fetch text from https://arxiv.org/html/2501.12948v1.html for content simulation...
Failed to extract text from arXiv HTML or save to file: 404 Client Error: Not Found for url: https://arxiv.org/html/2501.12948v1.html
Falling back to a very minimal dummy text for tree generation.
Used minimal dummy text and saved to data/2501.12948.txt
Document submitted locally: 2501_12948_txt
Document Submitted: 2501_12948_txt

Simplified Tree Structure of the Document:
DeepSeek-R1: Incentivizing Reasoning Cap... (ID: 2501_12948_txt_0000)
  Abstract (ID: 2501_12948_txt_0001)
  1. Introduction (ID: 2501_12948_txt_0003)
    1.1. Contributions (ID: 2501_12948_txt_0004)
  5. Conclusion, Limitations, and Future Work (ID: 2501_12948_txt_0019)

Calling local LLM for tree search (this might take a moment)...

Reasoning Process:
The question is asking for conclusions in this document, so I will look for
nodes with titles that contain words lik

In [ ]:
# 0.0 إعداد البيئة وتثبيت التبعيات
# هذه الخلية سوف تقوم بتثبيت الحزم الضرورية وإعداد البيئة.

# تثبيت PageIndex (إذا لم يكن مثبتاً مسبقاً)
%pip install -q --upgrade pageindex
%pip install -q --upgrade openai
%pip install -q requests beautifulsoup4 PyPDF2

import os
import json
import PyPDF2
import httpx
import asyncio
from bs4 import BeautifulSoup

# Client محلي لمحاكاة PageIndexClient (بدون استدعاءات API حقيقية)
class LocalPageIndexClient:
    def __init__(self, api_key=None):
        print("Using dummy LocalPageIndexClient for local execution.")
        self.documents = {} # تخزين محتوى المستندات
        self.trees = {} # تخزين هياكل الأشجار المنشأة

    def submit_document(self, file_path):
        doc_id = os.path.basename(file_path).replace('.', '_').replace('-', '_')
        with open(file_path, 'r', encoding='utf-8') as f:
            self.documents[doc_id] = f.read()
        print(f"Document submitted locally: {doc_id}")
        self.trees[doc_id] = self._generate_simple_tree_from_text(self.documents[doc_id], doc_id)
        return {"doc_id": doc_id}

    def is_retrieval_ready(self, doc_id):
        return doc_id in self.trees

    def get_tree(self, doc_id, node_summary=True):
        return {"result": self.trees.get(doc_id, [])}

    def _generate_simple_tree_from_text(self, text_content, doc_id):
        # تنفيذ مبسط لإنشاء شجرة من النص
        sections = []
        lines = text_content.split('\n')
        current_section = []
        section_title = "نظرة عامة على المستند"
        node_counter = 0

        section_patterns = [
            "المقدمة", "الفصل", "الباب", "الخلاصة", "الاستنتاجات",
            "المراجع", "الملاحق", "الفهرس"
        ]

        parsed_sections = []
        current_section_title = None
        current_section_content = []
        current_page_index = 1

        for line in lines:
            stripped_line = line.strip()
            is_new_section = False
            for pattern in section_patterns:
                if stripped_line.startswith(pattern) and len(stripped_line) < 100:
                    if current_section_title and current_section_content:
                        parsed_sections.append({
                            "title": current_section_title,
                            "text": "\n".join(current_section_content).strip(),
                            "node_id": f"{doc_id}_{node_counter:04d}",
                            "page_index": current_page_index
                        })
                        node_counter += 1
                    current_section_title = pattern
                    current_section_content = [stripped_line]
                    is_new_section = True
                    current_page_index += 1
                    break
            if not is_new_section:
                current_section_content.append(line)

        if current_section_title and current_section_content:
             parsed_sections.append({
                "title": current_section_title,
                "text": "\n".join(current_section_content).strip(),
                "node_id": f"{doc_id}_{node_counter:04d}",
                "page_index": current_page_index
            })

        tree_nodes = []
        node_map_for_subnodes = {}

        root_node = {
            "title": "كتاب تعلم البايثون",
            "node_id": f"{doc_id}_0000",
            "prefix_summary": "# كتاب تعلم البايثون",
            "nodes": [],
            "page_index": 1
        }

        for section in parsed_sections:
            summary = section["text"].split(' ', 20)
            summary = ' '.join(summary[:20]) + "..." if len(summary) > 20 else ' '.join(summary)

            node_entry = {
                "title": section["title"],
                "node_id": f"{node_counter:04d}",
                "summary": summary,
                "text": section["text"],
                "page_index": section["page_index"]
            }

            root_node["nodes"].append(node_entry)

        return [root_node]

# utils محلية للتنفيذ المحلي
class LocalUtils:
    @staticmethod
    def print_tree(tree, indent=0):
        for node in tree:
            prefix = "  " * indent
            print(f"{prefix}{node.get('title', 'Untitled')} (ID: {node.get('node_id', 'N/A')})")
            if "nodes" in node:
                LocalUtils.print_tree(node["nodes"], indent + 1)

    @staticmethod
    def remove_fields(obj, fields):
        if isinstance(obj, dict):
            return {k: LocalUtils.remove_fields(v, fields) for k, v in obj.items() if k not in fields}
        elif isinstance(obj, list):
            return [LocalUtils.remove_fields(elem, fields) for elem in obj]
        else:
            return obj

    @staticmethod
    def create_node_mapping(tree, node_map=None):
        if node_map is None:
            node_map = {}
        for node in tree:
            node_map[node.get('node_id')] = node
            if "nodes" in node:
                LocalUtils.create_node_mapping(node["nodes"], node_map)
        return node_map

    @staticmethod
    def print_wrapped(text, width=80):
        import textwrap
        print(textwrap.fill(text, width=width))

pi_client = LocalPageIndexClient()
utils = LocalUtils()

# Ollama LLM client للاستدلال المحلي
async def call_llm(prompt, model="llama3", temperature=0):
    # التأكد من أن خادم Ollama يعمل والنموذج مُحمّل
    client = httpx.AsyncClient(base_url="http://localhost:11434")
    try:
        response = await client.post(
            "/api/chat",
            json={
                "model": model,
                "messages": [{"role": "user", "content": prompt}],
                "stream": False,
                "options": {"temperature": temperature}
            },
            timeout=None # تعطيل المهلة للردود الطويلة المحتملة
        )
        response.raise_for_status() # رفع استثناء لأخطاء HTTP
        return response.json()["message"]["content"].strip()
    except httpx.ConnectError as e:
        print(f"Error connecting to Ollama: {e}")
        print("Please ensure Ollama is running (e.g., 'ollama serve') and you have pulled the model (e.g., 'ollama pull llama3').")
        return "ERROR: Could not connect to Ollama. Please check your Ollama setup."
    except httpx.HTTPStatusError as e:
        print(f"Error from Ollama server: {e.response.status_code} - {e.response.text}")
        return f"ERROR: Ollama server responded with an error: {e.response.status_code}"
    except json.JSONDecodeError:
        print(f"Error decoding JSON from Ollama: {response.text}")
        return "ERROR: Invalid JSON response from Ollama."
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return f"ERROR: An unexpected error occurred: {e}"

# تحديد مسار الكتاب المحلي
book_path = "/content/كتاب_بايثون.pdf"  # تغيير هذا المسار حسب الحاجة

# التأكد من وجود الكتاب
if not os.path.exists(book_path):
    raise FileNotFoundError(f"الكتاب غير موجود في المسار المحدد: {book_path}")

# 1.1 تقديم مستند لإنشاء شجرة PageIndex (محلياً)
try:
    # استخدام PyPDF2 لاستخراج النص من PDF محلي
    with open(book_path, 'rb') as f:
        pdf_reader = PyPDF2.PdfReader(f)
        extracted_text_parts = []

        for page_num in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[page_num]
            text = page.extract_text()
            if text:
                extracted_text_parts.append(f"--- الصفحة {page_num+1} ---")
                extracted_text_parts.append(text)

    dummy_text_content = "\n\n".join(extracted_text_parts)

    text_path = book_path.replace('.pdf', '.txt')
    with open(text_path, "w", encoding="utf-8") as f:
        f.write(dummy_text_content)

    print(f"تم استخراج المحتوى إلى {text_path}")
    doc_id = pi_client.submit_document(text_path)["doc_id"]
    print('تم تقديم المستند:', doc_id)

except Exception as e:
    print(f"فشل في استخراج النص من PDF: {e}")
    # استخدام محتوى افتراضي إذا فشل الاستخراج
    dummy_text_content = """
    # كتاب تعلم لغة بايثون
    ## المقدمة
    بايثون هي لغة برمجة قوية وسهلة التعلم. هذا الكتاب يهدف إلى مساعدتك على تعلم الأساسيات والمفاهيم المتقدمة.
    ## الفصل الأول: أساسيات بايثون
    في هذا الفصل سنتعلم المتغيرات، الأنواع الأساسية، والعمليات الأساسية في بايثون.
    ## الفصل الثاني: هياكل التحكم
    سنتعلم في هذا الفصل عن الشروط والحلقات والتكرار.
    ## الخلاصة
    بايثون لغة ممتازة للمبتدئين والمحترفين على حد سواء.
    """
    text_path = book_path.replace('.pdf', '.txt')
    with open(text_path, "w", encoding="utf-8") as f:
        f.write(dummy_text_content)
    print(f"تم استخدام نص افتراضي وحفظه في {text_path}")
    doc_id = pi_client.submit_document(text_path)["doc_id"]
    print('تم تقديم المستند:', doc_id)

# 1.2 الحصول على بنية شجرة PageIndex المُنشأة
if pi_client.is_retrieval_ready(doc_id):
    tree = pi_client.get_tree(doc_id, node_summary=True)['result']
    print('\nهيكل الشجرة المبسط للمستند:')
    utils.print_tree(tree)
else:
    print("جارٍ معالجة المستند، يرجى المحاولة مرة أخرى لاحقًا...")

# 2.1 استخدام LLM للبحث في الشجرة وتحديد العقد التي قد تحتوي على سياق ذي صلة
query = "ما هي الاستنتاجات في هذا المستند؟"

# إزالة حقل 'text' لتجنب إرسال الكثير من البيانات إلى LLM للبحث في الشجرة
tree_without_text = utils.remove_fields(tree.copy(), fields=['text'])

search_prompt = f"""
لقد تم إعطاؤك سؤالاً وبنية شجرة لمستند.
تحتوي كل عقدة على معرف العقدة وعنوان العقدة وملخص مقابل.
مهمتك هي العثور على جميع العقد التي من المحتمل أن تحتوي على إجابة للسؤال.

السؤال: {query}

بنية شجرة المستند:
{json.dumps(tree_without_text, indent=2)}

يرجى الرد بتنسيق JSON التالي:
{{
    "thinking": "<عملية تفكيرك حول العقد ذات الصلة بالسؤال>",
    "node_list": ["node_id_1", "node_id_2", ..., "node_id_n"]
}}
قم بإرجاع بنية JSON النهائية مباشرة. لا تخرج أي شيء آخر.
"""

print("\nجاري استدعاء LLM المحلي للبحث في الشجرة (قد يستغرق هذا بعض الوقت)...")
tree_search_result = await call_llm(search_prompt, model="llama3") # استخدام llama3 أو النموذج المحلي المفضل لديك

# 2.2 طباعة العقد المسترجعة وعملية التفكير
try:
    node_map = utils.create_node_mapping(tree)
    tree_search_result_json = json.loads(tree_search_result)

    print('\nعملية التفكير:')
    utils.print_wrapped(tree_search_result_json.get('thinking', 'لم تقدم LLM عملية تفكير.'))

    print('\nالعقد المسترجعة:')
    retrieved_node_ids = tree_search_result_json.get("node_list", [])
    if not retrieved_node_ids:
        print("لم تسترجع LLM أي عقد.")
    for node_id_key in retrieved_node_ids:
        # معرف العقدة من LLM قد يكون فقط '0019'، يحتاج إلى إضافة doc_id إذا كانت هذه هي طريقة التخزين
        # بالنسبة لهذا العميل الوهمي، تكون معرفات العقد مثل "doc_id_0019"
        # دعنا نضبط للبنية الوهمية إذا كانت LLM تخرج الرقم فقط
        actual_node_id = f"{doc_id}_{node_id_key}" if '_' not in node_id_key else node_id_key

        node = node_map.get(actual_node_id)
        if node:
            print(f"معرف العقدة: {node['node_id']}\t الصفحة: {node.get('page_index', 'N/A')}\t العنوان: {node['title']}")
        else:
            print(f"معرف العقدة: {actual_node_id} (غير موجود في الخريطة - قد تكون LLM قد هلوست أو عدم تطابق تنسيق المعرف)")
except json.JSONDecodeError:
    print(f"\nخطأ: لم تُرجع LLM JSON صالحًا لنتيجة البحث في الشجرة:\n{tree_search_result}")
except Exception as e:
    print(f"\nحدث خطأ أثناء معالجة نتيجة البحث في الشجرة: {e}")

# 3.1 استخراج السياق ذي الصلة من العقد المسترجعة
# استخدم node_map للحصول على النص الكامل للعقد المحددة
# إعادة تحليل نتيجة LLM في حالة وجود خطأ في الكتلة السابقة
try:
    retrieved_node_ids = json.loads(tree_search_result).get("node_list", [])
except json.JSONDecodeError:
    retrieved_node_ids = []

relevant_content = []
for node_id_key in retrieved_node_ids:
    actual_node_id = f"{doc_id}_{node_id_key}" if '_' not in node_id_key else node_id_key
    node = node_map.get(actual_node_id)
    if node and 'text' in node:
        relevant_content.append(node['text'])

relevant_content_str = "\n\n".join(relevant_content)

print('\nالسياق المسترجع:\n')
if relevant_content_str:
    utils.print_wrapped(relevant_content_str[:1000] + '...' if len(relevant_content_str) > 1000 else relevant_content_str)
else:
    print("لم يتم العثور على سياق ذي صلة بناءً على قائمة عقد LLM.")

# 3.2 إنشاء إجابة بناءً على السياق المسترجع
if relevant_content_str:
    answer_prompt = f"""
    أجب على السؤال بناءً على السياق:\n\nالسؤال: {query}\nالسياق: {relevant_content_str}\n\nقدم إجابة واضحة وموجزة بناءً فقط على السياق المقدم.
    """
    print('\nجاري استدعاء LLM المحلي لإنشاء الإجابة...')
    answer = await call_llm(answer_prompt, model="llama3") # استخدام llama3 أو النموذج المحلي المفضل لديك
    print('\nالإجابة المُنشأة:\n')
    utils.print_wrapped(answer)
else:
    print("\nلا يمكن إنشاء إجابة حيث لم يتم استرداد أي سياق ذي صلة.")

التغييرات الرئيسية:

اللغة: تحويل جميع النصوص إلى اللغة الإنجليزية فقط

مسار الكتاب: تغيير استخدام مسار الملف المحلي بدلاً من التنزيل من الإنترنت

استخراج ملفات PDF: إضافة PyPDF2 لاستخراج نصوص PDF محليًا

معالجة الأخطاء: الحفاظ على معالجة الأخطاء مع الرجوع إلى المحتوى الافتراضي

أنماط الأقسام: تحديث عناوين أقسام المستندات الإنجليزية الشائعة

الهيكل: الحفاظ على هيكل الكود الأصلي ووظائفه

يحافظ الكود على جميع وظائفه الأصلية عند استخدام ملف كتاب محلي بدلاً من التنزيل من الإنترنت. التغيير الوحيد هو مصدر المستند - كل شيء آخر يعمل تمامًا كما هو في الكود الأصلي.

In [2]:
# 0.0 Setup Environment and Install Dependencies
# This cell will install necessary packages and prepare the environment.

# Install PageIndex (if not already installed)
%pip install -q --upgrade pageindex
%pip install -q --upgrade openai
%pip install -q requests beautifulsoup4 PyPDF2

import os
import json
import PyPDF2
import httpx
import asyncio
from bs4 import BeautifulSoup

# Dummy PageIndexClient for local execution (no actual API calls)
class LocalPageIndexClient:
    def __init__(self, api_key=None):
        print("Using dummy LocalPageIndexClient for local execution.")
        self.documents = {} # Store document content
        self.trees = {} # Store generated tree structures

    def submit_document(self, file_path):
        doc_id = os.path.basename(file_path).replace('.', '_').replace('-', '_')
        with open(file_path, 'r', encoding='utf-8') as f:
            self.documents[doc_id] = f.read()
        print(f"Document submitted locally: {doc_id}")
        self.trees[doc_id] = self._generate_simple_tree_from_text(self.documents[doc_id], doc_id)
        return {"doc_id": doc_id}

    def is_retrieval_ready(self, doc_id):
        return doc_id in self.trees

    def get_tree(self, doc_id, node_summary=True):
        return {"result": self.trees.get(doc_id, [])}

    def _generate_simple_tree_from_text(self, text_content, doc_id):
        # Simple implementation to create a tree from text
        sections = []
        lines = text_content.split('\n')
        current_section = []
        section_title = "Document Overview"
        node_counter = 0

        section_patterns = [
            "Introduction", "Chapter", "Section", "Conclusion", "Summary",
            "References", "Appendix", "Index", "Abstract"
        ]

        parsed_sections = []
        current_section_title = None
        current_section_content = []
        current_page_index = 1

        for line in lines:
            stripped_line = line.strip()
            is_new_section = False
            for pattern in section_patterns:
                if stripped_line.startswith(pattern) and len(stripped_line) < 100:
                    if current_section_title and current_section_content:
                        parsed_sections.append({
                            "title": current_section_title,
                            "text": "\n".join(current_section_content).strip(),
                            "node_id": f"{doc_id}_{node_counter:04d}",
                            "page_index": current_page_index
                        })
                        node_counter += 1
                    current_section_title = pattern
                    current_section_content = [stripped_line]
                    is_new_section = True
                    current_page_index += 1
                    break
            if not is_new_section:
                current_section_content.append(line)

        if current_section_title and current_section_content:
             parsed_sections.append({
                "title": current_section_title,
                "text": "\n".join(current_section_content).strip(),
                "node_id": f"{doc_id}_{node_counter:04d}",
                "page_index": current_page_index
            })

        tree_nodes = []
        node_map_for_subnodes = {}

        root_node = {
            "title": "Python Learning Book",
            "node_id": f"{doc_id}_0000",
            "prefix_summary": "# Python Learning Book",
            "nodes": [],
            "page_index": 1
        }

        for section in parsed_sections:
            summary = section["text"].split(' ', 20)
            summary = ' '.join(summary[:20]) + "..." if len(summary) > 20 else ' '.join(summary)

            node_entry = {
                "title": section["title"],
                "node_id": f"{node_counter:04d}",
                "summary": summary,
                "text": section["text"],
                "page_index": section["page_index"]
            }

            root_node["nodes"].append(node_entry)

        return [root_node]

# Dummy utils for local execution (mimicking pageindex.utils)
class LocalUtils:
    @staticmethod
    def print_tree(tree, indent=0):
        for node in tree:
            prefix = "  " * indent
            print(f"{prefix}{node.get('title', 'Untitled')} (ID: {node.get('node_id', 'N/A')})")
            if "nodes" in node:
                LocalUtils.print_tree(node["nodes"], indent + 1)

    @staticmethod
    def remove_fields(obj, fields):
        if isinstance(obj, dict):
            return {k: LocalUtils.remove_fields(v, fields) for k, v in obj.items() if k not in fields}
        elif isinstance(obj, list):
            return [LocalUtils.remove_fields(elem, fields) for elem in obj]
        else:
            return obj

    @staticmethod
    def create_node_mapping(tree, node_map=None):
        if node_map is None:
            node_map = {}
        for node in tree:
            node_map[node.get('node_id')] = node
            if "nodes" in node:
                LocalUtils.create_node_mapping(node["nodes"], node_map)
        return node_map

    @staticmethod
    def print_wrapped(text, width=80):
        import textwrap
        print(textwrap.fill(text, width=width))

pi_client = LocalPageIndexClient()
utils = LocalUtils()

# Ollama LLM client for local inference
async def call_llm(prompt, model="llama3:latest", temperature=0):
    # Ensure Ollama server is running and model is pulled
    client = httpx.AsyncClient(base_url="http://localhost:11434")
    try:
        response = await client.post(
            "/api/chat",
            json={
                "model": model,
                "messages": [{"role": "user", "content": prompt}],
                "stream": False,
                "options": {"temperature": temperature}
            },
            timeout=None # Disable timeout for potentially long responses
        )
        response.raise_for_status() # Raise an exception for HTTP errors
        return response.json()["message"]["content"].strip()
    except httpx.ConnectError as e:
        print(f"Error connecting to Ollama: {e}")
        print("Please ensure Ollama is running (e.g., 'ollama serve') and you have pulled the model (e.g., 'ollama pull llama3').")
        return "ERROR: Could not connect to Ollama. Please check your Ollama setup."
    except httpx.HTTPStatusError as e:
        print(f"Error from Ollama server: {e.response.status_code} - {e.response.text}")
        return f"ERROR: Ollama server responded with an error: {e.response.status_code}"
    except json.JSONDecodeError:
        print(f"Error decoding JSON from Ollama: {response.text}")
        return "ERROR: Invalid JSON response from Ollama."
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return f"ERROR: An unexpected error occurred: {e}"

# Specify local book path
book_path = "/content/data/Understanding_Climate_Change.pdf"  # Change this path as needed

# Verify book exists
if not os.path.exists(book_path):
    raise FileNotFoundError(f"Book not found at specified path: {book_path}")

# 1.1 Submit a document for generating PageIndex tree (locally)
try:
    # Use PyPDF2 to extract text from local PDF
    with open(book_path, 'rb') as f:
        pdf_reader = PyPDF2.PdfReader(f)
        extracted_text_parts = []

        for page_num in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[page_num]
            text = page.extract_text()
            if text:
                extracted_text_parts.append(f"--- Page {page_num+1} ---")
                extracted_text_parts.append(text)

    dummy_text_content = "\n\n".join(extracted_text_parts)

    text_path = book_path.replace('.pdf', '.txt')
    with open(text_path, "w", encoding="utf-8") as f:
        f.write(dummy_text_content)

    print(f"Content extracted to {text_path}")
    doc_id = pi_client.submit_document(text_path)["doc_id"]
    print('Document Submitted:', doc_id)

except Exception as e:
    print(f"Failed to extract text from PDF: {e}")
    # Use default content if extraction fails
    dummy_text_content = """
    # Python Learning Book
    ## Introduction
    Python is a powerful and easy-to-learn programming language. This book aims to help you learn the basics and advanced concepts.
    ## Chapter 1: Python Basics
    In this chapter we will learn about variables, basic types, and basic operations in Python.
    ## Chapter 2: Control Structures
    We will learn about conditions, loops, and iteration in this chapter.
    ## Conclusion
    Python is an excellent language for both beginners and professionals.
    """
    text_path = book_path.replace('.pdf', '.txt')
    with open(text_path, "w", encoding="utf-8") as f:
        f.write(dummy_text_content)
    print(f"Used default text and saved to {text_path}")
    doc_id = pi_client.submit_document(text_path)["doc_id"]
    print('Document Submitted:', doc_id)

# 1.2 Get the generated PageIndex tree structure
if pi_client.is_retrieval_ready(doc_id):
    tree = pi_client.get_tree(doc_id, node_summary=True)['result']
    print('\nSimplified Tree Structure of the Document:')
    utils.print_tree(tree)
else:
    print("Processing document, please try again later...")

# 2.1 Use LLM for tree search and identify nodes that might contain relevant context
query = "What are the causes of climate change in this document?"

# Remove the 'text' field to avoid sending too much data to the LLM for tree search
tree_without_text = utils.remove_fields(tree.copy(), fields=['text'])

search_prompt = f"""
You are given a question and a tree structure of a document.
Each node contains a node id, node title, and a corresponding summary.
Your task is to find all nodes that are likely to contain the answer to the question.

Question: {query}

Document tree structure:
{json.dumps(tree_without_text, indent=2)}

Please reply in the following JSON format:
{{
    "thinking": "<Your thinking process on which nodes are relevant to the question>",
    "node_list": ["node_id_1", "node_id_2", ..., "node_id_n"]
}}
Directly return the final JSON structure. Do not output anything else.
"""

print("\nCalling local LLM for tree search (this might take a moment)...")
tree_search_result = await call_llm(search_prompt, model="llama3:latest") # Use llama3 or your preferred local model

# 2.2 Print retrieved nodes and reasoning process
try:
    node_map = utils.create_node_mapping(tree)
    tree_search_result_json = json.loads(tree_search_result)

    print('\nReasoning Process:')
    utils.print_wrapped(tree_search_result_json.get('thinking', 'No thinking process provided by LLM.'))

    print('\nRetrieved Nodes:')
    retrieved_node_ids = tree_search_result_json.get("node_list", [])
    if not retrieved_node_ids:
        print("No nodes retrieved by LLM.")
    for node_id_key in retrieved_node_ids:
        # The node_id from the LLM might be just '0019', need to prepend doc_id if that's how we store
        # For this dummy client, the node_ids are like "doc_id_0019"
        # Let's adjust for the dummy structure if LLM outputs just the number
        actual_node_id = f"{doc_id}_{node_id_key}" if '_' not in node_id_key else node_id_key

        node = node_map.get(actual_node_id)
        if node:
            print(f"Node ID: {node['node_id']}\t Page: {node.get('page_index', 'N/A')}\t Title: {node['title']}")
        else:
            print(f"Node ID: {actual_node_id} (Not found in map - LLM might have hallucinated or ID format mismatch)")
except json.JSONDecodeError:
    print(f"\nError: LLM did not return valid JSON for tree search result:\n{tree_search_result}")
except Exception as e:
    print(f"\nAn error occurred during tree search result processing: {e}")

# 3.1 Extract relevant context from retrieved nodes
# Use the node_map to get the full text of the identified nodes
# Re-parse LLM result in case of error in previous block
try:
    retrieved_node_ids = json.loads(tree_search_result).get("node_list", [])
except json.JSONDecodeError:
    retrieved_node_ids = []

relevant_content = []
for node_id_key in retrieved_node_ids:
    actual_node_id = f"{doc_id}_{node_id_key}" if '_' not in node_id_key else node_id_key
    node = node_map.get(actual_node_id)
    if node and 'text' in node:
        relevant_content.append(node['text'])

relevant_content_str = "\n\n".join(relevant_content)

print('\nRetrieved Context:\n')
if relevant_content_str:
    utils.print_wrapped(relevant_content_str[:1000] + '...' if len(relevant_content_str) > 1000 else relevant_content_str)
else:
    print("No relevant context found based on LLM's node list.")

# 3.2 Generate answer based on retrieved context
if relevant_content_str:
    answer_prompt = f"""
    Answer the question based on the context:\n\nQuestion: {query}\nContext: {relevant_content_str}\n\nProvide a clear, concise answer based only on the context provided.
    """
    print('\nCalling local LLM for answer generation...')
    answer = await call_llm(answer_prompt, model="llama3:latest") # Use llama3 or your preferred local model
    print('\nGenerated Answer:\n')
    utils.print_wrapped(answer)
else:
    print("\nCannot generate an answer as no relevant context was retrieved.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.6 MB/s eta 0:00:00
Using dummy LocalPageIndexClient for local execution.
Content extracted to /content/data/Understanding_Climate_Change.txt
Document submitted locally: Understanding_Climate_Change_txt
Document Submitted: Understanding_Climate_Change_txt

Simplified Tree Structure of the Document:
Python Learning Book (ID: Understanding_Climate_Change_txt_0000)
  Chapter (ID: 0020)
  Chapter (ID: 0020)
  Chapter (ID: 0020)
  Chapter (ID: 0020)
  Chapter (ID: 0020)
  Chapter (ID: 0020)
  Chapter (ID: 0020)
  Chapter (ID: 0020)
  Chapter (ID: 0020)
  Chapter (ID: 0020)
  Chapter (ID: 0020)
  Chapter (ID: 0020)
  Chapter (ID: 0020)
  Chapter (ID: 0020)
  Chapter (ID: 0020)
  Chapter (ID: 0020)
  Chapter (ID: 0020)
  Chapter (ID: 0020)
  Chapter (ID: 0020)
  Chapter (ID: 0020)
  Chapter (ID: 0020)

Calling local LLM for tree search (this might take a moment)...

Reasoning Process:
The question is asking about the causes of climat

In [ ]:
Climate_Change

## الكود الل فشل رابطه تم تعديل الرابط

In [ ]:
# --- START OF COLAB CELL ---

# 0.0 Setup Environment and Install Dependencies
# This cell will install necessary packages and prepare the environment.

# Install PageIndex (if not already installed)
# %pip install -q --upgrade pageindex
# %pip install -q --upgrade openai  # For potential future use or local OpenAI-compatible servers
# %pip install -q requests beautifulsoup4  # For document downloading and parsing

import os
import requests
import json
from bs4 import BeautifulSoup

# Dummy PageIndexClient for local execution (no actual API calls)
class LocalPageIndexClient:
    def __init__(self, api_key=None):
        print("Using dummy LocalPageIndexClient for local execution.")
        self.documents = {} # Store document content
        self.trees = {} # Store generated tree structures

    def submit_document(self, file_path):
        doc_id = os.path.basename(file_path).replace('.', '_').replace('-', '_') # Simple doc_id generation
        with open(file_path, 'r', encoding='utf-8') as f:
            self.documents[doc_id] = f.read()
        print(f"Document submitted locally: {doc_id}")
        # In a real scenario, you'd process the document here to generate a tree.
        # For this example, we'll manually create a simplified tree for demonstration.
        self.trees[doc_id] = self._generate_simple_tree_from_text(self.documents[doc_id], doc_id)
        return {"doc_id": doc_id}

    def is_retrieval_ready(self, doc_id):
        return doc_id in self.trees

    def get_tree(self, doc_id, node_summary=True):
        return {"result": self.trees.get(doc_id, [])}

    def _generate_simple_tree_from_text(self, text_content, doc_id):
        # This function attempts to parse the text content into a hierarchical structure
        # based on common academic paper headings.

        root_node = {
            "title": "Document Root", # General root title
            "node_id": f"{doc_id}_0000",
            "prefix_summary": "Root of the document structure.",
            "nodes": [],
            "page_index": 1
        }

        # Define common academic section patterns to look for
        section_patterns = [
            ("## Abstract", "Abstract"),
            ("## 1 Introduction", "1. Introduction"),
            ("## 1.1", "1.1"),
            ("## 2 Related Work", "2. Related Work"),
            ("## 3 Method", "3. Method"),
            ("## 3.1", "3.1"),
            ("## 4 Experiments", "4. Experiments"),
            ("## 4.1", "4.1"),
            ("## 5 Conclusion and Future Work", "5. Conclusion and Future Work"),
            ("## References", "References"),
            ("## Appendix", "Appendix")
        ]

        parsed_sections = []
        current_section_title = "Document Root Content" # Default for un-sectioned content
        current_section_content = []
        node_counter = 0
        current_page_index = 1

        lines = text_content.split('\n')

        # First pass: Identify all main sections and their content
        for line in lines:
            stripped_line = line.strip()
            found_new_section = False
            for pattern_prefix, canonical_title in section_patterns:
                if stripped_line.startswith(pattern_prefix):
                    if current_section_content and current_section_title != "Document Root Content":
                        # Save the previous section
                        parsed_sections.append({
                            "title": current_section_title,
                            "text": "\n".join(current_section_content).strip(),
                            "node_id": f"{doc_id}_{node_counter:04d}",
                            "page_index": current_page_index
                        })
                        node_counter += 1
                        current_page_index += 1 # Simulate page increment for new section

                    current_section_title = canonical_title
                    current_section_content = [stripped_line]
                    found_new_section = True
                    break

            if not found_new_section:
                current_section_content.append(line)

        # Add the last section
        if current_section_content:
            parsed_sections.append({
                "title": current_section_title,
                "text": "\n".join(current_section_content).strip(),
                "node_id": f"{doc_id}_{node_counter:04d}",
                "page_index": current_page_index
            })

        # Second pass: Build the hierarchical tree
        node_map_for_nesting = {root_node["node_id"]: root_node} # Map to easily find parents

        for section_data in parsed_sections:
            node_id = section_data["node_id"]
            title = section_data["title"]
            text = section_data["text"]

            summary_words = text.split(' ', 20)
            summary = ' '.join(summary_words[:20]) + "..." if len(summary_words) > 20 else ' '.join(summary_words)

            node_entry = {
                "title": title,
                "node_id": node_id,
                "summary": summary,
                "text": text,
                "page_index": section_data["page_index"]
            }
            node_map_for_nesting[node_id] = node_entry # Add to map for potential children

            # Simple heuristic for nesting (based on "1." "1.1." "2." etc.)
            parent_added = False
            if title.startswith("1.") and len(title.split('.')) == 2: # e.g., "1. Introduction"
                root_node["nodes"].append(node_entry)
                parent_added = True
            elif title.startswith("1.1") or title.startswith("1.2"): # e.g., "1.1. Contributions"
                intro_node = next((n for n in root_node["nodes"] if n["title"].startswith("1.") and len(n["title"].split('.')) == 2), None)
                if intro_node:
                    if "nodes" not in intro_node: intro_node["nodes"] = []
                    intro_node["nodes"].append(node_entry)
                    parent_added = True
            elif title.startswith("2.") and len(title.split('.')) == 2: # e.g., "2. Related Work"
                root_node["nodes"].append(node_entry)
                parent_added = True
            elif title.startswith("3.") and len(title.split('.')) == 2: # e.g., "3. Method"
                root_node["nodes"].append(node_entry)
                parent_added = True
            elif title.startswith("4.") and len(title.split('.')) == 2: # e.g., "4. Experiments"
                root_node["nodes"].append(node_entry)
                parent_added = True
            elif title.startswith("5.") and len(title.split('.')) == 2: # e.g., "5. Conclusion..."
                root_node["nodes"].append(node_entry)
                parent_added = True
            elif title == "Abstract" or title == "References" or title == "Appendix":
                root_node["nodes"].append(node_entry)
                parent_added = True

            if not parent_added and title != "Document Root Content":
                # If not specifically nested, add to root
                root_node["nodes"].append(node_entry)

        # Ensure no duplicate entries in root_node if some sections were implicitly added twice
        final_root_nodes = []
        seen_node_ids = set()
        for node in root_node["nodes"]:
            if node["node_id"] not in seen_node_ids:
                final_root_nodes.append(node)
                seen_node_ids.add(node["node_id"])
        root_node["nodes"] = final_root_nodes

        return [root_node]


# Dummy utils for local execution (mimicking pageindex.utils)
class LocalUtils:
    @staticmethod
    def print_tree(tree, indent=0):
        for node in tree:
            prefix = "  " * indent
            print(f"{prefix}{node.get('title', 'Untitled')} (ID: {node.get('node_id', 'N/A')})")
            if "nodes" in node:
                LocalUtils.print_tree(node["nodes"], indent + 1)

    @staticmethod
    def remove_fields(obj, fields):
        if isinstance(obj, dict):
            return {k: LocalUtils.remove_fields(v, fields) for k, v in obj.items() if k not in fields}
        elif isinstance(obj, list):
            return [LocalUtils.remove_fields(elem, fields) for elem in obj]
        else:
            return obj

    @staticmethod
    def create_node_mapping(tree, node_map=None):
        if node_map is None:
            node_map = {}
        for node in tree:
            node_map[node.get('node_id')] = node
            if "nodes" in node:
                LocalUtils.create_node_mapping(node["nodes"], node_map)
        return node_map

    @staticmethod
    def print_wrapped(text, width=80):
        import textwrap
        print(textwrap.fill(text, width=width))

pi_client = LocalPageIndexClient()
utils = LocalUtils()

# Ollama LLM client for local inference
import httpx

async def call_llm(prompt, model="llama3:latest", temperature=0):
    # Ensure Ollama server is running and model is pulled
    # Example: ollama run llama3
    client = httpx.AsyncClient(base_url="http://localhost:11434")
    try:
        response = await client.post(
            "/api/chat",
            json={
                "model": model,
                "messages": [{"role": "user", "content": prompt}],
                "stream": False,
                "options": {"temperature": temperature}
            },
            timeout=None # Disable timeout for potentially long responses
        )
        response.raise_for_status() # Raise an exception for HTTP errors
        return response.json()["message"]["content"].strip()
    except httpx.ConnectError as e:
        print(f"Error connecting to Ollama: {e}")
        print("Please ensure Ollama is running (e.g., 'ollama serve') and you have pulled the model (e.g., 'ollama pull llama3').")
        return "ERROR: Could not connect to Ollama. Please check your Ollama setup."
    except httpx.HTTPStatusError as e:
        print(f"Error from Ollama server: {e.response.status_code} - {e.response.text}")
        return f"ERROR: Ollama server responded with an error: {e.response.status_code}"
    except json.JSONDecodeError:
        print(f"Error decoding JSON from Ollama: {response.text}")
        return "ERROR: Invalid JSON response from Ollama."
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return f"ERROR: An unexpected error occurred: {e}"

# 1.1 Submit a document for generating PageIndex tree (locally)
# CHANGED: New URL for the document
pdf_url = "https://arxiv.org/pdf/2508.21069.pdf" # This is a placeholder as .pdf might not exist for future
arxiv_html_url = "https://arxiv.org/html/2508.21069v1.html" # Direct HTML link

pdf_filename = arxiv_html_url.split('/')[-1].replace('.html', '.pdf') # Simulate pdf filename from html
pdf_path = os.path.join("data", pdf_filename) # Use a 'data' directory

os.makedirs(os.path.dirname(pdf_path), exist_ok=True)


print(f"Attempting to fetch text from {arxiv_html_url} for content simulation...")
try:
    html_response = requests.get(arxiv_html_url)
    html_response.raise_for_status()
    soup = BeautifulSoup(html_response.text, 'html.parser')

    # Extract main content - this will be highly dependent on arXiv's HTML structure
    extracted_text_parts = []
    # Look for common article structure elements
    for tag in soup.find_all(['h1', 'h2', 'h3', 'h4', 'p', 'li', 'section']):
        text = tag.get_text(separator=' ', strip=True)
        if text:
            extracted_text_parts.append(text)

    dummy_text_content = "\n\n".join(extracted_text_parts)
    if not dummy_text_content.strip():
        raise ValueError("Could not extract significant text from arXiv HTML.")

    text_path = pdf_path.replace('.pdf', '.txt')
    with open(text_path, "w", encoding="utf-8") as f:
        f.write(dummy_text_content)
    print(f"Extracted content to {text_path}")
    doc_id = pi_client.submit_document(text_path)["doc_id"]
    print('Document Submitted:', doc_id)

except Exception as e:
    print(f"Failed to extract text from arXiv HTML or save to file: {e}")
    print("Falling back to a very minimal dummy text for tree generation.")
    # UPDATED dummy text for the new URL if HTML extraction fails
    dummy_text_content = """
    # Deep Learning for Climate Model Emulation
    ## Abstract
    This paper explores the use of deep learning models to emulate complex climate simulations, offering faster predictions and insights. We demonstrate that neural networks can accurately reproduce outputs of sophisticated climate models.
    ## 1 Introduction
    Climate change research relies heavily on complex numerical simulations. These are computationally expensive. Deep learning offers a promising alternative.
    ## 2 Related Work
    Previous studies have used machine learning for atmospheric processes. Our work focuses on full climate model emulation.
    ## 3 Method
    We employed a U-Net architecture trained on high-resolution climate model data. Input features include atmospheric variables; outputs are future climate states.
    ## 4 Experiments
    Our experiments show that the DL emulator achieves high fidelity compared to the full climate model across various metrics, including temperature and precipitation patterns. It significantly reduces computational time.
    ## 5 Conclusion and Future Work
    Deep learning emulation is a powerful tool for climate science. We conclude that DL models can effectively surrogate complex physical processes. Future work includes expanding to different climate models and uncertainty quantification.
    ## References
    [1] Smith et al. Climate Modeling.
    [2] Jones et al. Deep Learning for Earth Systems.
    """
    text_path = pdf_path.replace('.pdf', '.txt')
    with open(text_path, "w", encoding="utf-8") as f:
        f.write(dummy_text_content)
    print(f"Used minimal dummy text and saved to {text_path}")
    doc_id = pi_client.submit_document(text_path)["doc_id"]
    print('Document Submitted:', doc_id)


# 1.2 Get the generated PageIndex tree structure
# This will use the simplified tree generated by our LocalPageIndexClient
if pi_client.is_retrieval_ready(doc_id):
    tree = pi_client.get_tree(doc_id, node_summary=True)['result']
    print('\nSimplified Tree Structure of the Document:')
    utils.print_tree(tree)
else:
    print("Processing document, please try again later...")

# 2.1 Use LLM for tree search and identify nodes that might contain relevant context
# CHANGED: New query
query = "What are the main findings of this document?"

# Remove the 'text' field to avoid sending too much data to the LLM for tree search
tree_without_text = utils.remove_fields(tree.copy(), fields=['text'])

search_prompt = f"""
You are given a question and a tree structure of a document.
Each node contains a node id, node title, and a corresponding summary.
Your task is to find all nodes that are likely to contain the answer to the question.

Question: {query}

Document tree structure:
{json.dumps(tree_without_text, indent=2)}

Please reply in the following JSON format:
{{
    "thinking": "<Your thinking process on which nodes are relevant to the question>",
    "node_list": ["node_id_1", "node_id_2", ..., "node_id_n"]
}}
Directly return the final JSON structure. Do not output anything else.
"""

print("\nCalling local LLM for tree search (this might take a moment)...")
tree_search_result = await call_llm(search_prompt, model="llama3:latest") # Use llama3 or your preferred local model

# 2.2 Print retrieved nodes and reasoning process
try:
    node_map = utils.create_node_mapping(tree)
    tree_search_result_json = json.loads(tree_search_result)

    print('\nReasoning Process:')
    utils.print_wrapped(tree_search_result_json.get('thinking', 'No thinking process provided by LLM.'))

    print('\nRetrieved Nodes:')
    retrieved_node_ids = tree_search_result_json.get("node_list", [])
    if not retrieved_node_ids:
        print("No nodes retrieved by LLM.")
    for node_id_key in retrieved_node_ids:
        # The node_id from the LLM might be just '0019', need to prepend doc_id if that's how we store
        # For this dummy client, the node_ids are like "doc_id_0019"
        # Let's adjust for the dummy structure if LLM outputs just the number
        actual_node_id = f"{doc_id}_{node_id_key}" if '_' not in node_id_key and not node_id_key.startswith(doc_id) else node_id_key

        node = node_map.get(actual_node_id)
        if node:
            print(f"Node ID: {node['node_id']}\t Page: {node.get('page_index', 'N/A')}\t Title: {node['title']}")
        else:
            print(f"Node ID: {actual_node_id} (Not found in map - LLM might have hallucinated or ID format mismatch)")
except json.JSONDecodeError:
    print(f"\nError: LLM did not return valid JSON for tree search result:\n{tree_search_result}")
except Exception as e:
    print(f"\nAn error occurred during tree search result processing: {e}")


# 3.1 Extract relevant context from retrieved nodes
# Use the node_map to get the full text of the identified nodes
# Re-parse LLM result in case of error in previous block
try:
    retrieved_node_ids = json.loads(tree_search_result).get("node_list", [])
except json.JSONDecodeError:
    retrieved_node_ids = []

relevant_content = []
for node_id_key in retrieved_node_ids:
    actual_node_id = f"{doc_id}_{node_id_key}" if '_' not in node_id_key and not node_id_key.startswith(doc_id) else node_id_key
    node = node_map.get(actual_node_id)
    if node and 'text' in node:
        relevant_content.append(node['text'])

relevant_content_str = "\n\n".join(relevant_content)

print('\nRetrieved Context:\n')
if relevant_content_str:
    utils.print_wrapped(relevant_content_str[:1000] + '...' if len(relevant_content_str) > 1000 else relevant_content_str)
else:
    print("No relevant context found based on LLM's node list.")


# 3.2 Generate answer based on retrieved context
if relevant_content_str:
    answer_prompt = f"""
    Answer the question based on the context:\n\nQuestion: {query}\nContext: {relevant_content_str}\n\nProvide a clear, concise answer based only on the context provided.
    """
    print('\nCalling local LLM for answer generation...')
    answer = await call_llm(answer_prompt, model="llama3:latest") # Use llama3 or your preferred local model
    print('\nGenerated Answer:\n')
    utils.print_wrapped(answer)
else:
    print("\nCannot generate an answer as no relevant context was retrieved.")

# --- END OF COLAB CELL ---

In [ ]:
حدد الأسطر التي تريد التعليق عليها.
اضغط على Ctrl + / (في معظم أنظمة التشغيل) أو Cmd + / (على macOS).
سيؤدي هذا إلى إضافة # في بداية كل سطر محدد. وإذا قمت بتحديد الأسطر مرة أخرى وضغطت على نفس الاختصار، فسيتم إزالة علامة #.

أتمنى أن يكون هذا مفيدًا! إذا كان لديك أي أسئلة أخرى، فلا تتردد في طرحها.



In [ ]:
%pip install -q --upgrade pageindex
%pip install -q --upgrade openai  # For potential future use or local OpenAI-compatible servers
%pip install -q requests beautifulsoup4  # For document downloading and parsing

In [3]:
# --- START OF COLAB CELL ---

# 0.0 Setup Environment and Install Dependencies
# This cell will install necessary packages and prepare the environment.

# Install PageIndex (if not already installed)
# %pip install -q --upgrade pageindex
# %pip install -q --upgrade openai  # For potential future use or local OpenAI-compatible servers
# %pip install -q requests beautifulsoup4  # For document downloading and parsing

import os
import requests
import json
from bs4 import BeautifulSoup

# Dummy PageIndexClient for local execution (no actual API calls)
class LocalPageIndexClient:
    def __init__(self, api_key=None):
        print("Using dummy LocalPageIndexClient for local execution.")
        self.documents = {} # Store document content
        self.trees = {} # Store generated tree structures

    def submit_document(self, file_path):
        doc_id = os.path.basename(file_path).replace('.', '_').replace('-', '_') # Simple doc_id generation
        with open(file_path, 'r', encoding='utf-8') as f:
            self.documents[doc_id] = f.read()
        print(f"Document submitted locally: {doc_id}")
        # In a real scenario, you'd process the document here to generate a tree.
        # For this example, we'll manually create a simplified tree for demonstration.
        self.trees[doc_id] = self._generate_simple_tree_from_text(self.documents[doc_id], doc_id)
        return {"doc_id": doc_id}

    def is_retrieval_ready(self, doc_id):
        return doc_id in self.trees

    def get_tree(self, doc_id, node_summary=True):
        return {"result": self.trees.get(doc_id, [])}

    def _generate_simple_tree_from_text(self, text_content, doc_id):
        # This function attempts to parse the text content into a hierarchical structure
        # based on common academic paper headings.

        root_node = {
            "title": "Document Root", # General root title
            "node_id": f"{doc_id}_0000",
            "prefix_summary": "Root of the document structure.",
            "nodes": [],
            "page_index": 1
        }

        # Define common academic section patterns to look for
        section_patterns = [
            ("## Abstract", "Abstract"),
            ("## 1 Introduction", "1. Introduction"),
            ("## 1.1", "1.1"),
            ("## 2 Related Work", "2. Related Work"),
            ("## 3 Method", "3. Method"),
            ("## 3.1", "3.1"),
            ("## 4 Experiments", "4. Experiments"),
            ("## 4.1", "4.1"),
            ("## 5 Conclusion and Future Work", "5. Conclusion and Future Work"),
            ("## References", "References"),
            ("## Appendix", "Appendix")
        ]

        parsed_sections = []
        current_section_title = "Document Root Content" # Default for un-sectioned content
        current_section_content = []
        node_counter = 0
        current_page_index = 1

        lines = text_content.split('\n')

        # First pass: Identify all main sections and their content
        for line in lines:
            stripped_line = line.strip()
            found_new_section = False
            for pattern_prefix, canonical_title in section_patterns:
                if stripped_line.startswith(pattern_prefix):
                    if current_section_content and current_section_title != "Document Root Content":
                        # Save the previous section
                        parsed_sections.append({
                            "title": current_section_title,
                            "text": "\n".join(current_section_content).strip(),
                            "node_id": f"{doc_id}_{node_counter:04d}",
                            "page_index": current_page_index
                        })
                        node_counter += 1
                        current_page_index += 1 # Simulate page increment for new section

                    current_section_title = canonical_title
                    current_section_content = [stripped_line]
                    found_new_section = True
                    break

            if not found_new_section:
                current_section_content.append(line)

        # Add the last section
        if current_section_content:
            parsed_sections.append({
                "title": current_section_title,
                "text": "\n".join(current_section_content).strip(),
                "node_id": f"{doc_id}_{node_counter:04d}",
                "page_index": current_page_index
            })

        # Second pass: Build the hierarchical tree
        node_map_for_nesting = {root_node["node_id"]: root_node} # Map to easily find parents

        for section_data in parsed_sections:
            node_id = section_data["node_id"]
            title = section_data["title"]
            text = section_data["text"]

            summary_words = text.split(' ', 20)
            summary = ' '.join(summary_words[:20]) + "..." if len(summary_words) > 20 else ' '.join(summary_words)

            node_entry = {
                "title": title,
                "node_id": node_id,
                "summary": summary,
                "text": text,
                "page_index": section_data["page_index"]
            }
            node_map_for_nesting[node_id] = node_entry # Add to map for potential children

            # Simple heuristic for nesting (based on "1." "1.1." "2." etc.)
            parent_added = False
            if title.startswith("1.") and len(title.split('.')) == 2: # e.g., "1. Introduction"
                root_node["nodes"].append(node_entry)
                parent_added = True
            elif title.startswith("1.1") or title.startswith("1.2"): # e.g., "1.1. Contributions"
                intro_node = next((n for n in root_node["nodes"] if n["title"].startswith("1.") and len(n["title"].split('.')) == 2), None)
                if intro_node:
                    if "nodes" not in intro_node: intro_node["nodes"] = []
                    intro_node["nodes"].append(node_entry)
                    parent_added = True
            elif title.startswith("2.") and len(title.split('.')) == 2: # e.g., "2. Related Work"
                root_node["nodes"].append(node_entry)
                parent_added = True
            elif title.startswith("3.") and len(title.split('.')) == 2: # e.g., "3. Method"
                root_node["nodes"].append(node_entry)
                parent_added = True
            elif title.startswith("4.") and len(title.split('.')) == 2: # e.g., "4. Experiments"
                root_node["nodes"].append(node_entry)
                parent_added = True
            elif title.startswith("5.") and len(title.split('.')) == 2: # e.g., "5. Conclusion..."
                root_node["nodes"].append(node_entry)
                parent_added = True
            elif title == "Abstract" or title == "References" or title == "Appendix":
                root_node["nodes"].append(node_entry)
                parent_added = True

            if not parent_added and title != "Document Root Content":
                # If not specifically nested, add to root
                root_node["nodes"].append(node_entry)

        # Ensure no duplicate entries in root_node if some sections were implicitly added twice
        final_root_nodes = []
        seen_node_ids = set()
        for node in root_node["nodes"]:
            if node["node_id"] not in seen_node_ids:
                final_root_nodes.append(node)
                seen_node_ids.add(node["node_id"])
        root_node["nodes"] = final_root_nodes

        return [root_node]


# Dummy utils for local execution (mimicking pageindex.utils)
class LocalUtils:
    @staticmethod
    def print_tree(tree, indent=0):
        for node in tree:
            prefix = "  " * indent
            print(f"{prefix}{node.get('title', 'Untitled')} (ID: {node.get('node_id', 'N/A')})")
            if "nodes" in node:
                LocalUtils.print_tree(node["nodes"], indent + 1)

    @staticmethod
    def remove_fields(obj, fields):
        if isinstance(obj, dict):
            return {k: LocalUtils.remove_fields(v, fields) for k, v in obj.items() if k not in fields}
        elif isinstance(obj, list):
            return [LocalUtils.remove_fields(elem, fields) for elem in obj]
        else:
            return obj

    @staticmethod
    def create_node_mapping(tree, node_map=None):
        if node_map is None:
            node_map = {}
        for node in tree:
            node_map[node.get('node_id')] = node
            if "nodes" in node:
                LocalUtils.create_node_mapping(node["nodes"], node_map)
        return node_map

    @staticmethod
    def print_wrapped(text, width=80):
        import textwrap
        print(textwrap.fill(text, width=width))

pi_client = LocalPageIndexClient()
utils = LocalUtils()

# Ollama LLM client for local inference
import httpx

async def call_llm(prompt, model="llama3:latest", temperature=0):
    # Ensure Ollama server is running and model is pulled
    # Example: ollama run llama3
    client = httpx.AsyncClient(base_url="http://localhost:11434")
    try:
        response = await client.post(
            "/api/chat",
            json={
                "model": model,
                "messages": [{"role": "user", "content": prompt}],
                "stream": False,
                "options": {"temperature": temperature}
            },
            timeout=None # Disable timeout for potentially long responses
        )
        response.raise_for_status() # Raise an exception for HTTP errors
        return response.json()["message"]["content"].strip()
    except httpx.ConnectError as e:
        print(f"Error connecting to Ollama: {e}")
        print("Please ensure Ollama is running (e.g., 'ollama serve') and you have pulled the model (e.g., 'ollama pull llama3').")
        return "ERROR: Could not connect to Ollama. Please check your Ollama setup."
    except httpx.HTTPStatusError as e:
        print(f"Error from Ollama server: {e.response.status_code} - {e.response.text}")
        return f"ERROR: Ollama server responded with an error: {e.response.status_code}"
    except json.JSONDecodeError:
        print(f"Error decoding JSON from Ollama: {response.text}")
        return "ERROR: Invalid JSON response from Ollama."
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return f"ERROR: An unexpected error occurred: {e}"

# 1.1 Submit a document for generating PageIndex tree (locally)
# CHANGED: New URL for the document
pdf_url = "https://arxiv.org/pdf/2508.21069" # This is a placeholder as .pdf might not exist for future
arxiv_html_url = "https://arxiv.org/html/2508.21069v1" # Direct HTML link

pdf_filename = arxiv_html_url.split('/')[-1].replace('.html', '.pdf') # Simulate pdf filename from html
pdf_path = os.path.join("data", pdf_filename) # Use a 'data' directory

os.makedirs(os.path.dirname(pdf_path), exist_ok=True)


print(f"Attempting to fetch text from {arxiv_html_url} for content simulation...")
try:
    html_response = requests.get(arxiv_html_url)
    html_response.raise_for_status()
    soup = BeautifulSoup(html_response.text, 'html.parser')

    # Extract main content - this will be highly dependent on arXiv's HTML structure
    extracted_text_parts = []
    # Look for common article structure elements
    for tag in soup.find_all(['h1', 'h2', 'h3', 'h4', 'p', 'li', 'section']):
        text = tag.get_text(separator=' ', strip=True)
        if text:
            extracted_text_parts.append(text)

    dummy_text_content = "\n\n".join(extracted_text_parts)
    if not dummy_text_content.strip():
        raise ValueError("Could not extract significant text from arXiv HTML.")

    text_path = pdf_path.replace('.pdf', '.txt')
    with open(text_path, "w", encoding="utf-8") as f:
        f.write(dummy_text_content)
    print(f"Extracted content to {text_path}")
    doc_id = pi_client.submit_document(text_path)["doc_id"]
    print('Document Submitted:', doc_id)

except Exception as e:
    print(f"Failed to extract text from arXiv HTML or save to file: {e}")
    print("Falling back to a very minimal dummy text for tree generation.")
    # UPDATED dummy text for the new URL if HTML extraction fails
    dummy_text_content = """
    # Deep Learning for Climate Model Emulation
    ## Abstract
    This paper explores the use of deep learning models to emulate complex climate simulations, offering faster predictions and insights. We demonstrate that neural networks can accurately reproduce outputs of sophisticated climate models.
    ## 1 Introduction
    Climate change research relies heavily on complex numerical simulations. These are computationally expensive. Deep learning offers a promising alternative.
    ## 2 Related Work
    Previous studies have used machine learning for atmospheric processes. Our work focuses on full climate model emulation.
    ## 3 Method
    We employed a U-Net architecture trained on high-resolution climate model data. Input features include atmospheric variables; outputs are future climate states.
    ## 4 Experiments
    Our experiments show that the DL emulator achieves high fidelity compared to the full climate model across various metrics, including temperature and precipitation patterns. It significantly reduces computational time.
    ## 5 Conclusion and Future Work
    Deep learning emulation is a powerful tool for climate science. We conclude that DL models can effectively surrogate complex physical processes. Future work includes expanding to different climate models and uncertainty quantification.
    ## References
    [1] Smith et al. Climate Modeling.
    [2] Jones et al. Deep Learning for Earth Systems.
    """
    text_path = pdf_path.replace('.pdf', '.txt')
    with open(text_path, "w", encoding="utf-8") as f:
        f.write(dummy_text_content)
    print(f"Used minimal dummy text and saved to {text_path}")
    doc_id = pi_client.submit_document(text_path)["doc_id"]
    print('Document Submitted:', doc_id)


# 1.2 Get the generated PageIndex tree structure
# This will use the simplified tree generated by our LocalPageIndexClient
if pi_client.is_retrieval_ready(doc_id):
    tree = pi_client.get_tree(doc_id, node_summary=True)['result']
    print('\nSimplified Tree Structure of the Document:')
    utils.print_tree(tree)
else:
    print("Processing document, please try again later...")

# 2.1 Use LLM for tree search and identify nodes that might contain relevant context
# CHANGED: New query
query = "What are the main findings of this document?"

# Remove the 'text' field to avoid sending too much data to the LLM for tree search
tree_without_text = utils.remove_fields(tree.copy(), fields=['text'])

search_prompt = f"""
You are given a question and a tree structure of a document.
Each node contains a node id, node title, and a corresponding summary.
Your task is to find all nodes that are likely to contain the answer to the question.

Question: {query}

Document tree structure:
{json.dumps(tree_without_text, indent=2)}

Please reply in the following JSON format:
{{
    "thinking": "<Your thinking process on which nodes are relevant to the question>",
    "node_list": ["node_id_1", "node_id_2", ..., "node_id_n"]
}}
Directly return the final JSON structure. Do not output anything else.
"""

print("\nCalling local LLM for tree search (this might take a moment)...")
tree_search_result = await call_llm(search_prompt, model="llama3:latest") # Use llama3 or your preferred local model

# 2.2 Print retrieved nodes and reasoning process
try:
    node_map = utils.create_node_mapping(tree)
    tree_search_result_json = json.loads(tree_search_result)

    print('\nReasoning Process:')
    utils.print_wrapped(tree_search_result_json.get('thinking', 'No thinking process provided by LLM.'))

    print('\nRetrieved Nodes:')
    retrieved_node_ids = tree_search_result_json.get("node_list", [])
    if not retrieved_node_ids:
        print("No nodes retrieved by LLM.")
    for node_id_key in retrieved_node_ids:
        # The node_id from the LLM might be just '0019', need to prepend doc_id if that's how we store
        # For this dummy client, the node_ids are like "doc_id_0019"
        # Let's adjust for the dummy structure if LLM outputs just the number
        actual_node_id = f"{doc_id}_{node_id_key}" if '_' not in node_id_key and not node_id_key.startswith(doc_id) else node_id_key

        node = node_map.get(actual_node_id)
        if node:
            print(f"Node ID: {node['node_id']}\t Page: {node.get('page_index', 'N/A')}\t Title: {node['title']}")
        else:
            print(f"Node ID: {actual_node_id} (Not found in map - LLM might have hallucinated or ID format mismatch)")
except json.JSONDecodeError:
    print(f"\nError: LLM did not return valid JSON for tree search result:\n{tree_search_result}")
except Exception as e:
    print(f"\nAn error occurred during tree search result processing: {e}")


# 3.1 Extract relevant context from retrieved nodes
# Use the node_map to get the full text of the identified nodes
# Re-parse LLM result in case of error in previous block
try:
    retrieved_node_ids = json.loads(tree_search_result).get("node_list", [])
except json.JSONDecodeError:
    retrieved_node_ids = []

relevant_content = []
for node_id_key in retrieved_node_ids:
    actual_node_id = f"{doc_id}_{node_id_key}" if '_' not in node_id_key and not node_id_key.startswith(doc_id) else node_id_key
    node = node_map.get(actual_node_id)
    if node and 'text' in node:
        relevant_content.append(node['text'])

relevant_content_str = "\n\n".join(relevant_content)

print('\nRetrieved Context:\n')
if relevant_content_str:
    utils.print_wrapped(relevant_content_str[:1000] + '...' if len(relevant_content_str) > 1000 else relevant_content_str)
else:
    print("No relevant context found based on LLM's node list.")


# 3.2 Generate answer based on retrieved context
if relevant_content_str:
    answer_prompt = f"""
    Answer the question based on the context:\n\nQuestion: {query}\nContext: {relevant_content_str}\n\nProvide a clear, concise answer based only on the context provided.
    """
    print('\nCalling local LLM for answer generation...')
    answer = await call_llm(answer_prompt, model="llama3:latest") # Use llama3 or your preferred local model
    print('\nGenerated Answer:\n')
    utils.print_wrapped(answer)
else:
    print("\nCannot generate an answer as no relevant context was retrieved.")

# --- END OF COLAB CELL ---

Using dummy LocalPageIndexClient for local execution.
Attempting to fetch text from https://arxiv.org/html/2508.21069v1 for content simulation...
Extracted content to data/2508.21069v1
Document submitted locally: 2508_21069v1
Document Submitted: 2508_21069v1

Simplified Tree Structure of the Document:
Document Root (ID: 2508_21069v1_0000)

Calling local LLM for tree search (this might take a moment)...
Error connecting to Ollama: All connection attempts failed
Please ensure Ollama is running (e.g., 'ollama serve') and you have pulled the model (e.g., 'ollama pull llama3').

Error: LLM did not return valid JSON for tree search result:
ERROR: Could not connect to Ollama. Please check your Ollama setup.

Retrieved Context:

No relevant context found based on LLM's node list.

Cannot generate an answer as no relevant context was retrieved.


In [4]:
# --- START OF COLAB CELL ---

# 0.0 Setup Environment and Install Dependencies
# This cell will install necessary packages and prepare the environment.

# Install PageIndex (if not already installed)
# %pip install -q --upgrade pageindex
# %pip install -q --upgrade openai  # For potential future use or local OpenAI-compatible servers
# %pip install -q requests beautifulsoup4  # For document downloading and parsing

import os
import requests
import json
from bs4 import BeautifulSoup

# Dummy PageIndexClient for local execution (no actual API calls)
class LocalPageIndexClient:
    def __init__(self, api_key=None):
        print("Using dummy LocalPageIndexClient for local execution.")
        self.documents = {} # Store document content
        self.trees = {} # Store generated tree structures

    def submit_document(self, file_path):
        doc_id = os.path.basename(file_path).replace('.', '_').replace('-', '_') # Simple doc_id generation
        with open(file_path, 'r', encoding='utf-8') as f:
            self.documents[doc_id] = f.read()
        print(f"Document submitted locally: {doc_id}")
        # In a real scenario, you'd process the document here to generate a tree.
        # For this example, we'll manually create a simplified tree for demonstration.
        self.trees[doc_id] = self._generate_simple_tree_from_text(self.documents[doc_id], doc_id)
        return {"doc_id": doc_id}

    def is_retrieval_ready(self, doc_id):
        return doc_id in self.trees

    def get_tree(self, doc_id, node_summary=True):
        return {"result": self.trees.get(doc_id, [])}

    def _generate_simple_tree_from_text(self, text_content, doc_id):
        # This function attempts to parse the text content into a hierarchical structure
        # based on common academic paper headings.

        root_node = {
            "title": "Document Root", # General root title
            "node_id": f"{doc_id}_0000",
            "prefix_summary": "Root of the document structure.",
            "nodes": [],
            "page_index": 1
        }

        # Define common academic section patterns to look for
        section_patterns = [
            ("## Abstract", "Abstract"),
            ("## 1 Introduction", "1. Introduction"),
            ("## 1.1", "1.1"),
            ("## 2 Related Work", "2. Related Work"),
            ("## 3 Method", "3. Method"),
            ("## 3.1", "3.1"),
            ("## 4 Experiments", "4. Experiments"),
            ("## 4.1", "4.1"),
            ("## 5 Conclusion and Future Work", "5. Conclusion and Future Work"),
            ("## References", "References"),
            ("## Appendix", "Appendix")
        ]

        parsed_sections = []
        current_section_title = "Document Root Content" # Default for un-sectioned content
        current_section_content = []
        node_counter = 0
        current_page_index = 1

        lines = text_content.split('\n')

        # First pass: Identify all main sections and their content
        for line in lines:
            stripped_line = line.strip()
            found_new_section = False
            for pattern_prefix, canonical_title in section_patterns:
                if stripped_line.startswith(pattern_prefix):
                    if current_section_content and current_section_title != "Document Root Content":
                        # Save the previous section
                        parsed_sections.append({
                            "title": current_section_title,
                            "text": "\n".join(current_section_content).strip(),
                            "node_id": f"{doc_id}_{node_counter:04d}",
                            "page_index": current_page_index
                        })
                        node_counter += 1
                        current_page_index += 1 # Simulate page increment for new section

                    current_section_title = canonical_title
                    current_section_content = [stripped_line]
                    found_new_section = True
                    break

            if not found_new_section:
                current_section_content.append(line)

        # Add the last section
        if current_section_content:
            parsed_sections.append({
                "title": current_section_title,
                "text": "\n".join(current_section_content).strip(),
                "node_id": f"{doc_id}_{node_counter:04d}",
                "page_index": current_page_index
            })

        # Second pass: Build the hierarchical tree
        node_map_for_nesting = {root_node["node_id"]: root_node} # Map to easily find parents

        for section_data in parsed_sections:
            node_id = section_data["node_id"]
            title = section_data["title"]
            text = section_data["text"]

            summary_words = text.split(' ', 20)
            summary = ' '.join(summary_words[:20]) + "..." if len(summary_words) > 20 else ' '.join(summary_words)

            node_entry = {
                "title": title,
                "node_id": node_id,
                "summary": summary,
                "text": text,
                "page_index": section_data["page_index"]
            }
            node_map_for_nesting[node_id] = node_entry # Add to map for potential children

            # Simple heuristic for nesting (based on "1." "1.1." "2." etc.)
            parent_added = False
            if title.startswith("1.") and len(title.split('.')) == 2: # e.g., "1. Introduction"
                root_node["nodes"].append(node_entry)
                parent_added = True
            elif title.startswith("1.1") or title.startswith("1.2"): # e.g., "1.1. Contributions"
                intro_node = next((n for n in root_node["nodes"] if n["title"].startswith("1.") and len(n["title"].split('.')) == 2), None)
                if intro_node:
                    if "nodes" not in intro_node: intro_node["nodes"] = []
                    intro_node["nodes"].append(node_entry)
                    parent_added = True
            elif title.startswith("2.") and len(title.split('.')) == 2: # e.g., "2. Related Work"
                root_node["nodes"].append(node_entry)
                parent_added = True
            elif title.startswith("3.") and len(title.split('.')) == 2: # e.g., "3. Method"
                root_node["nodes"].append(node_entry)
                parent_added = True
            elif title.startswith("4.") and len(title.split('.')) == 2: # e.g., "4. Experiments"
                root_node["nodes"].append(node_entry)
                parent_added = True
            elif title.startswith("5.") and len(title.split('.')) == 2: # e.g., "5. Conclusion..."
                root_node["nodes"].append(node_entry)
                parent_added = True
            elif title == "Abstract" or title == "References" or title == "Appendix":
                root_node["nodes"].append(node_entry)
                parent_added = True

            if not parent_added and title != "Document Root Content":
                # If not specifically nested, add to root
                root_node["nodes"].append(node_entry)

        # Ensure no duplicate entries in root_node if some sections were implicitly added twice
        final_root_nodes = []
        seen_node_ids = set()
        for node in root_node["nodes"]:
            if node["node_id"] not in seen_node_ids:
                final_root_nodes.append(node)
                seen_node_ids.add(node["node_id"])
        root_node["nodes"] = final_root_nodes

        return [root_node]


# Dummy utils for local execution (mimicking pageindex.utils)
class LocalUtils:
    @staticmethod
    def print_tree(tree, indent=0):
        for node in tree:
            prefix = "  " * indent
            print(f"{prefix}{node.get('title', 'Untitled')} (ID: {node.get('node_id', 'N/A')})")
            if "nodes" in node:
                LocalUtils.print_tree(node["nodes"], indent + 1)

    @staticmethod
    def remove_fields(obj, fields):
        if isinstance(obj, dict):
            return {k: LocalUtils.remove_fields(v, fields) for k, v in obj.items() if k not in fields}
        elif isinstance(obj, list):
            return [LocalUtils.remove_fields(elem, fields) for elem in obj]
        else:
            return obj

    @staticmethod
    def create_node_mapping(tree, node_map=None):
        if node_map is None:
            node_map = {}
        for node in tree:
            node_map[node.get('node_id')] = node
            if "nodes" in node:
                LocalUtils.create_node_mapping(node["nodes"], node_map)
        return node_map

    @staticmethod
    def print_wrapped(text, width=80):
        import textwrap
        print(textwrap.fill(text, width=width))

pi_client = LocalPageIndexClient()
utils = LocalUtils()

# Ollama LLM client for local inference
import httpx

async def call_llm(prompt, model="llama3:latest", temperature=0):
    # Ensure Ollama server is running and model is pulled
    # Example: ollama run llama3
    client = httpx.AsyncClient(base_url="http://localhost:11434")
    try:
        response = await client.post(
            "/api/chat",
            json={
                "model": model,
                "messages": [{"role": "user", "content": prompt}],
                "stream": False,
                "options": {"temperature": temperature}
            },
            timeout=None # Disable timeout for potentially long responses
        )
        response.raise_for_status() # Raise an exception for HTTP errors
        return response.json()["message"]["content"].strip()
    except httpx.ConnectError as e:
        print(f"Error connecting to Ollama: {e}")
        print("Please ensure Ollama is running (e.g., 'ollama serve') and you have pulled the model (e.g., 'ollama pull llama3').")
        return "ERROR: Could not connect to Ollama. Please check your Ollama setup."
    except httpx.HTTPStatusError as e:
        print(f"Error from Ollama server: {e.response.status_code} - {e.response.text}")
        return f"ERROR: Ollama server responded with an error: {e.response.status_code}"
    except json.JSONDecodeError:
        print(f"Error decoding JSON from Ollama: {response.text}")
        return "ERROR: Invalid JSON response from Ollama."
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return f"ERROR: An unexpected error occurred: {e}"

# 1.1 Submit a document for generating PageIndex tree (locally)
# CHANGED: New URL for the document
pdf_url = "https://arxiv.org/pdf/2508.21069" # This is a placeholder as .pdf might not exist for future
arxiv_html_url = "https://arxiv.org/html/2508.21069v1" # Direct HTML link

pdf_filename = arxiv_html_url.split('/')[-1].replace('.html', '.pdf') # Simulate pdf filename from html
pdf_path = os.path.join("data", pdf_filename) # Use a 'data' directory

os.makedirs(os.path.dirname(pdf_path), exist_ok=True)


print(f"Attempting to fetch text from {arxiv_html_url} for content simulation...")
try:
    html_response = requests.get(arxiv_html_url)
    html_response.raise_for_status()
    soup = BeautifulSoup(html_response.text, 'html.parser')

    # Extract main content - this will be highly dependent on arXiv's HTML structure
    extracted_text_parts = []
    # Look for common article structure elements
    for tag in soup.find_all(['h1', 'h2', 'h3', 'h4', 'p', 'li', 'section']):
        text = tag.get_text(separator=' ', strip=True)
        if text:
            extracted_text_parts.append(text)

    dummy_text_content = "\n\n".join(extracted_text_parts)
    if not dummy_text_content.strip():
        raise ValueError("Could not extract significant text from arXiv HTML.")

    text_path = pdf_path.replace('.pdf', '.txt')
    with open(text_path, "w", encoding="utf-8") as f:
        f.write(dummy_text_content)
    print(f"Extracted content to {text_path}")
    doc_id = pi_client.submit_document(text_path)["doc_id"]
    print('Document Submitted:', doc_id)

except Exception as e:
    print(f"Failed to extract text from arXiv HTML or save to file: {e}")
    print("Falling back to a very minimal dummy text for tree generation.")
    # UPDATED dummy text for the new URL if HTML extraction fails
    dummy_text_content = """
    # Deep Learning for Climate Model Emulation
    ## Abstract
    This paper explores the use of deep learning models to emulate complex climate simulations, offering faster predictions and insights. We demonstrate that neural networks can accurately reproduce outputs of sophisticated climate models.
    ## 1 Introduction
    Climate change research relies heavily on complex numerical simulations. These are computationally expensive. Deep learning offers a promising alternative.
    ## 2 Related Work
    Previous studies have used machine learning for atmospheric processes. Our work focuses on full climate model emulation.
    ## 3 Method
    We employed a U-Net architecture trained on high-resolution climate model data. Input features include atmospheric variables; outputs are future climate states.
    ## 4 Experiments
    Our experiments show that the DL emulator achieves high fidelity compared to the full climate model across various metrics, including temperature and precipitation patterns. It significantly reduces computational time.
    ## 5 Conclusion and Future Work
    Deep learning emulation is a powerful tool for climate science. We conclude that DL models can effectively surrogate complex physical processes. Future work includes expanding to different climate models and uncertainty quantification.
    ## References
    [1] Smith et al. Climate Modeling.
    [2] Jones et al. Deep Learning for Earth Systems.
    """
    text_path = pdf_path.replace('.pdf', '.txt')
    with open(text_path, "w", encoding="utf-8") as f:
        f.write(dummy_text_content)
    print(f"Used minimal dummy text and saved to {text_path}")
    doc_id = pi_client.submit_document(text_path)["doc_id"]
    print('Document Submitted:', doc_id)


# 1.2 Get the generated PageIndex tree structure
# This will use the simplified tree generated by our LocalPageIndexClient
if pi_client.is_retrieval_ready(doc_id):
    tree = pi_client.get_tree(doc_id, node_summary=True)['result']
    print('\nSimplified Tree Structure of the Document:')
    utils.print_tree(tree)
else:
    print("Processing document, please try again later...")

# 2.1 Use LLM for tree search and identify nodes that might contain relevant context
# CHANGED: New query
query = "What are the main findings of this document?"

# Remove the 'text' field to avoid sending too much data to the LLM for tree search
tree_without_text = utils.remove_fields(tree.copy(), fields=['text'])

search_prompt = f"""
You are given a question and a tree structure of a document.
Each node contains a node id, node title, and a corresponding summary.
Your task is to find all nodes that are likely to contain the answer to the question.

Question: {query}

Document tree structure:
{json.dumps(tree_without_text, indent=2)}

Please reply in the following JSON format:
{{
    "thinking": "<Your thinking process on which nodes are relevant to the question>",
    "node_list": ["node_id_1", "node_id_2", ..., "node_id_n"]
}}
Directly return the final JSON structure. Do not output anything else.
"""

print("\nCalling local LLM for tree search (this might take a moment)...")
tree_search_result = await call_llm(search_prompt, model="llama3:latest") # Use llama3 or your preferred local model

# 2.2 Print retrieved nodes and reasoning process
try:
    node_map = utils.create_node_mapping(tree)
    tree_search_result_json = json.loads(tree_search_result)

    print('\nReasoning Process:')
    utils.print_wrapped(tree_search_result_json.get('thinking', 'No thinking process provided by LLM.'))

    print('\nRetrieved Nodes:')
    retrieved_node_ids = tree_search_result_json.get("node_list", [])
    if not retrieved_node_ids:
        print("No nodes retrieved by LLM.")
    for node_id_key in retrieved_node_ids:
        # The node_id from the LLM might be just '0019', need to prepend doc_id if that's how we store
        # For this dummy client, the node_ids are like "doc_id_0019"
        # Let's adjust for the dummy structure if LLM outputs just the number
        actual_node_id = f"{doc_id}_{node_id_key}" if '_' not in node_id_key and not node_id_key.startswith(doc_id) else node_id_key

        node = node_map.get(actual_node_id)
        if node:
            print(f"Node ID: {node['node_id']}\t Page: {node.get('page_index', 'N/A')}\t Title: {node['title']}")
        else:
            print(f"Node ID: {actual_node_id} (Not found in map - LLM might have hallucinated or ID format mismatch)")
except json.JSONDecodeError:
    print(f"\nError: LLM did not return valid JSON for tree search result:\n{tree_search_result}")
except Exception as e:
    print(f"\nAn error occurred during tree search result processing: {e}")


# 3.1 Extract relevant context from retrieved nodes
# Use the node_map to get the full text of the identified nodes
# Re-parse LLM result in case of error in previous block
try:
    retrieved_node_ids = json.loads(tree_search_result).get("node_list", [])
except json.JSONDecodeError:
    retrieved_node_ids = []

relevant_content = []
for node_id_key in retrieved_node_ids:
    actual_node_id = f"{doc_id}_{node_id_key}" if '_' not in node_id_key and not node_id_key.startswith(doc_id) else node_id_key
    node = node_map.get(actual_node_id)
    if node and 'text' in node:
        relevant_content.append(node['text'])

relevant_content_str = "\n\n".join(relevant_content)

print('\nRetrieved Context:\n')
if relevant_content_str:
    utils.print_wrapped(relevant_content_str[:1000] + '...' if len(relevant_content_str) > 1000 else relevant_content_str)
else:
    print("No relevant context found based on LLM's node list.")


# 3.2 Generate answer based on retrieved context
if relevant_content_str:
    answer_prompt = f"""
    Answer the question based on the context:\n\nQuestion: {query}\nContext: {relevant_content_str}\n\nProvide a clear, concise answer based only on the context provided.
    """
    print('\nCalling local LLM for answer generation...')
    answer = await call_llm(answer_prompt, model="llama3:latest") # Use llama3 or your preferred local model
    print('\nGenerated Answer:\n')
    utils.print_wrapped(answer)
else:
    print("\nCannot generate an answer as no relevant context was retrieved.")

# --- END OF COLAB CELL ---

Using dummy LocalPageIndexClient for local execution.
Attempting to fetch text from https://arxiv.org/html/2508.21069v1 for content simulation...
Extracted content to data/2508.21069v1
Document submitted locally: 2508_21069v1
Document Submitted: 2508_21069v1

Simplified Tree Structure of the Document:
Document Root (ID: 2508_21069v1_0000)

Calling local LLM for tree search (this might take a moment)...

Reasoning Process:
The question is asking for main findings, so I will look for nodes that have
'findings' or 'results' in their title or summary.

Retrieved Nodes:
Node ID: 2508_21069v1_0002 (Not found in map - LLM might have hallucinated or ID format mismatch)
Node ID: 2508_21069v1_0003 (Not found in map - LLM might have hallucinated or ID format mismatch)

Retrieved Context:

No relevant context found based on LLM's node list.

Cannot generate an answer as no relevant context was retrieved.


In [ ]:
What are the conclusions in this document?

In [5]:
# --- START OF COLAB CELL ---

# 0.0 Setup Environment and Install Dependencies
# This cell will install necessary packages and prepare the environment.

# Install PageIndex (if not already installed)
# %pip install -q --upgrade pageindex
# %pip install -q --upgrade openai  # For potential future use or local OpenAI-compatible servers
# %pip install -q requests beautifulsoup4  # For document downloading and parsing

import os
import requests
import json
from bs4 import BeautifulSoup

# Dummy PageIndexClient for local execution (no actual API calls)
class LocalPageIndexClient:
    def __init__(self, api_key=None):
        print("Using dummy LocalPageIndexClient for local execution.")
        self.documents = {} # Store document content
        self.trees = {} # Store generated tree structures

    def submit_document(self, file_path):
        doc_id = os.path.basename(file_path).replace('.', '_').replace('-', '_') # Simple doc_id generation
        with open(file_path, 'r', encoding='utf-8') as f:
            self.documents[doc_id] = f.read()
        print(f"Document submitted locally: {doc_id}")
        # In a real scenario, you'd process the document here to generate a tree.
        # For this example, we'll manually create a simplified tree for demonstration.
        self.trees[doc_id] = self._generate_simple_tree_from_text(self.documents[doc_id], doc_id)
        return {"doc_id": doc_id}

    def is_retrieval_ready(self, doc_id):
        return doc_id in self.trees

    def get_tree(self, doc_id, node_summary=True):
        return {"result": self.trees.get(doc_id, [])}

    def _generate_simple_tree_from_text(self, text_content, doc_id):
        # This function attempts to parse the text content into a hierarchical structure
        # based on common academic paper headings.

        root_node = {
            "title": "Document Root", # General root title
            "node_id": f"{doc_id}_0000",
            "prefix_summary": "Root of the document structure.",
            "nodes": [],
            "page_index": 1
        }

        # Define common academic section patterns to look for
        section_patterns = [
            ("## Abstract", "Abstract"),
            ("## 1 Introduction", "1. Introduction"),
            ("## 1.1", "1.1"),
            ("## 2 Related Work", "2. Related Work"),
            ("## 3 Method", "3. Method"),
            ("## 3.1", "3.1"),
            ("## 4 Experiments", "4. Experiments"),
            ("## 4.1", "4.1"),
            ("## 5 Conclusion and Future Work", "5. Conclusion and Future Work"),
            ("## References", "References"),
            ("## Appendix", "Appendix")
        ]

        parsed_sections = []
        current_section_title = "Document Root Content" # Default for un-sectioned content
        current_section_content = []
        node_counter = 0
        current_page_index = 1

        lines = text_content.split('\n')

        # First pass: Identify all main sections and their content
        for line in lines:
            stripped_line = line.strip()
            found_new_section = False
            for pattern_prefix, canonical_title in section_patterns:
                if stripped_line.startswith(pattern_prefix):
                    if current_section_content and current_section_title != "Document Root Content":
                        # Save the previous section
                        parsed_sections.append({
                            "title": current_section_title,
                            "text": "\n".join(current_section_content).strip(),
                            "node_id": f"{doc_id}_{node_counter:04d}",
                            "page_index": current_page_index
                        })
                        node_counter += 1
                        current_page_index += 1 # Simulate page increment for new section

                    current_section_title = canonical_title
                    current_section_content = [stripped_line]
                    found_new_section = True
                    break

            if not found_new_section:
                current_section_content.append(line)

        # Add the last section
        if current_section_content:
            parsed_sections.append({
                "title": current_section_title,
                "text": "\n".join(current_section_content).strip(),
                "node_id": f"{doc_id}_{node_counter:04d}",
                "page_index": current_page_index
            })

        # Second pass: Build the hierarchical tree
        node_map_for_nesting = {root_node["node_id"]: root_node} # Map to easily find parents

        for section_data in parsed_sections:
            node_id = section_data["node_id"]
            title = section_data["title"]
            text = section_data["text"]

            summary_words = text.split(' ', 20)
            summary = ' '.join(summary_words[:20]) + "..." if len(summary_words) > 20 else ' '.join(summary_words)

            node_entry = {
                "title": title,
                "node_id": node_id,
                "summary": summary,
                "text": text,
                "page_index": section_data["page_index"]
            }
            node_map_for_nesting[node_id] = node_entry # Add to map for potential children

            # Simple heuristic for nesting (based on "1." "1.1." "2." etc.)
            parent_added = False
            if title.startswith("1.") and len(title.split('.')) == 2: # e.g., "1. Introduction"
                root_node["nodes"].append(node_entry)
                parent_added = True
            elif title.startswith("1.1") or title.startswith("1.2"): # e.g., "1.1. Contributions"
                intro_node = next((n for n in root_node["nodes"] if n["title"].startswith("1.") and len(n["title"].split('.')) == 2), None)
                if intro_node:
                    if "nodes" not in intro_node: intro_node["nodes"] = []
                    intro_node["nodes"].append(node_entry)
                    parent_added = True
            elif title.startswith("2.") and len(title.split('.')) == 2: # e.g., "2. Related Work"
                root_node["nodes"].append(node_entry)
                parent_added = True
            elif title.startswith("3.") and len(title.split('.')) == 2: # e.g., "3. Method"
                root_node["nodes"].append(node_entry)
                parent_added = True
            elif title.startswith("4.") and len(title.split('.')) == 2: # e.g., "4. Experiments"
                root_node["nodes"].append(node_entry)
                parent_added = True
            elif title.startswith("5.") and len(title.split('.')) == 2: # e.g., "5. Conclusion..."
                root_node["nodes"].append(node_entry)
                parent_added = True
            elif title == "Abstract" or title == "References" or title == "Appendix":
                root_node["nodes"].append(node_entry)
                parent_added = True

            if not parent_added and title != "Document Root Content":
                # If not specifically nested, add to root
                root_node["nodes"].append(node_entry)

        # Ensure no duplicate entries in root_node if some sections were implicitly added twice
        final_root_nodes = []
        seen_node_ids = set()
        for node in root_node["nodes"]:
            if node["node_id"] not in seen_node_ids:
                final_root_nodes.append(node)
                seen_node_ids.add(node["node_id"])
        root_node["nodes"] = final_root_nodes

        return [root_node]


# Dummy utils for local execution (mimicking pageindex.utils)
class LocalUtils:
    @staticmethod
    def print_tree(tree, indent=0):
        for node in tree:
            prefix = "  " * indent
            print(f"{prefix}{node.get('title', 'Untitled')} (ID: {node.get('node_id', 'N/A')})")
            if "nodes" in node:
                LocalUtils.print_tree(node["nodes"], indent + 1)

    @staticmethod
    def remove_fields(obj, fields):
        if isinstance(obj, dict):
            return {k: LocalUtils.remove_fields(v, fields) for k, v in obj.items() if k not in fields}
        elif isinstance(obj, list):
            return [LocalUtils.remove_fields(elem, fields) for elem in obj]
        else:
            return obj

    @staticmethod
    def create_node_mapping(tree, node_map=None):
        if node_map is None:
            node_map = {}
        for node in tree:
            node_map[node.get('node_id')] = node
            if "nodes" in node:
                LocalUtils.create_node_mapping(node["nodes"], node_map)
        return node_map

    @staticmethod
    def print_wrapped(text, width=80):
        import textwrap
        print(textwrap.fill(text, width=width))

pi_client = LocalPageIndexClient()
utils = LocalUtils()

# Ollama LLM client for local inference
import httpx

async def call_llm(prompt, model="llama3:latest", temperature=0):
    # Ensure Ollama server is running and model is pulled
    # Example: ollama run llama3
    client = httpx.AsyncClient(base_url="http://localhost:11434")
    try:
        response = await client.post(
            "/api/chat",
            json={
                "model": model,
                "messages": [{"role": "user", "content": prompt}],
                "stream": False,
                "options": {"temperature": temperature}
            },
            timeout=None # Disable timeout for potentially long responses
        )
        response.raise_for_status() # Raise an exception for HTTP errors
        return response.json()["message"]["content"].strip()
    except httpx.ConnectError as e:
        print(f"Error connecting to Ollama: {e}")
        print("Please ensure Ollama is running (e.g., 'ollama serve') and you have pulled the model (e.g., 'ollama pull llama3').")
        return "ERROR: Could not connect to Ollama. Please check your Ollama setup."
    except httpx.HTTPStatusError as e:
        print(f"Error from Ollama server: {e.response.status_code} - {e.response.text}")
        return f"ERROR: Ollama server responded with an error: {e.response.status_code}"
    except json.JSONDecodeError:
        print(f"Error decoding JSON from Ollama: {response.text}")
        return "ERROR: Invalid JSON response from Ollama."
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return f"ERROR: An unexpected error occurred: {e}"

# 1.1 Submit a document for generating PageIndex tree (locally)
# CHANGED: New URL for the document
pdf_url = "https://arxiv.org/pdf/2508.21069" # This is a placeholder as .pdf might not exist for future
arxiv_html_url = "https://arxiv.org/html/2508.21069v1" # Direct HTML link

pdf_filename = arxiv_html_url.split('/')[-1].replace('.html', '.pdf') # Simulate pdf filename from html
pdf_path = os.path.join("data", pdf_filename) # Use a 'data' directory

os.makedirs(os.path.dirname(pdf_path), exist_ok=True)


print(f"Attempting to fetch text from {arxiv_html_url} for content simulation...")
try:
    html_response = requests.get(arxiv_html_url)
    html_response.raise_for_status()
    soup = BeautifulSoup(html_response.text, 'html.parser')

    # Extract main content - this will be highly dependent on arXiv's HTML structure
    extracted_text_parts = []
    # Look for common article structure elements
    for tag in soup.find_all(['h1', 'h2', 'h3', 'h4', 'p', 'li', 'section']):
        text = tag.get_text(separator=' ', strip=True)
        if text:
            extracted_text_parts.append(text)

    dummy_text_content = "\n\n".join(extracted_text_parts)
    if not dummy_text_content.strip():
        raise ValueError("Could not extract significant text from arXiv HTML.")

    text_path = pdf_path.replace('.pdf', '.txt')
    with open(text_path, "w", encoding="utf-8") as f:
        f.write(dummy_text_content)
    print(f"Extracted content to {text_path}")
    doc_id = pi_client.submit_document(text_path)["doc_id"]
    print('Document Submitted:', doc_id)

except Exception as e:
    print(f"Failed to extract text from arXiv HTML or save to file: {e}")
    print("Falling back to a very minimal dummy text for tree generation.")
    # UPDATED dummy text for the new URL if HTML extraction fails
    dummy_text_content = """
    # Deep Learning for Climate Model Emulation
    ## Abstract
    This paper explores the use of deep learning models to emulate complex climate simulations, offering faster predictions and insights. We demonstrate that neural networks can accurately reproduce outputs of sophisticated climate models.
    ## 1 Introduction
    Climate change research relies heavily on complex numerical simulations. These are computationally expensive. Deep learning offers a promising alternative.
    ## 2 Related Work
    Previous studies have used machine learning for atmospheric processes. Our work focuses on full climate model emulation.
    ## 3 Method
    We employed a U-Net architecture trained on high-resolution climate model data. Input features include atmospheric variables; outputs are future climate states.
    ## 4 Experiments
    Our experiments show that the DL emulator achieves high fidelity compared to the full climate model across various metrics, including temperature and precipitation patterns. It significantly reduces computational time.
    ## 5 Conclusion and Future Work
    Deep learning emulation is a powerful tool for climate science. We conclude that DL models can effectively surrogate complex physical processes. Future work includes expanding to different climate models and uncertainty quantification.
    ## References
    [1] Smith et al. Climate Modeling.
    [2] Jones et al. Deep Learning for Earth Systems.
    """
    text_path = pdf_path.replace('.pdf', '.txt')
    with open(text_path, "w", encoding="utf-8") as f:
        f.write(dummy_text_content)
    print(f"Used minimal dummy text and saved to {text_path}")
    doc_id = pi_client.submit_document(text_path)["doc_id"]
    print('Document Submitted:', doc_id)


# 1.2 Get the generated PageIndex tree structure
# This will use the simplified tree generated by our LocalPageIndexClient
if pi_client.is_retrieval_ready(doc_id):
    tree = pi_client.get_tree(doc_id, node_summary=True)['result']
    print('\nSimplified Tree Structure of the Document:')
    utils.print_tree(tree)
else:
    print("Processing document, please try again later...")

# 2.1 Use LLM for tree search and identify nodes that might contain relevant context
# CHANGED: New query
query = "What are the conclusions in this document?"

# Remove the 'text' field to avoid sending too much data to the LLM for tree search
tree_without_text = utils.remove_fields(tree.copy(), fields=['text'])

search_prompt = f"""
You are given a question and a tree structure of a document.
Each node contains a node id, node title, and a corresponding summary.
Your task is to find all nodes that are likely to contain the answer to the question.

Question: {query}

Document tree structure:
{json.dumps(tree_without_text, indent=2)}

Please reply in the following JSON format:
{{
    "thinking": "<Your thinking process on which nodes are relevant to the question>",
    "node_list": ["node_id_1", "node_id_2", ..., "node_id_n"]
}}
Directly return the final JSON structure. Do not output anything else.
"""

print("\nCalling local LLM for tree search (this might take a moment)...")
tree_search_result = await call_llm(search_prompt, model="llama3:latest") # Use llama3 or your preferred local model

# 2.2 Print retrieved nodes and reasoning process
try:
    node_map = utils.create_node_mapping(tree)
    tree_search_result_json = json.loads(tree_search_result)

    print('\nReasoning Process:')
    utils.print_wrapped(tree_search_result_json.get('thinking', 'No thinking process provided by LLM.'))

    print('\nRetrieved Nodes:')
    retrieved_node_ids = tree_search_result_json.get("node_list", [])
    if not retrieved_node_ids:
        print("No nodes retrieved by LLM.")
    for node_id_key in retrieved_node_ids:
        # The node_id from the LLM might be just '0019', need to prepend doc_id if that's how we store
        # For this dummy client, the node_ids are like "doc_id_0019"
        # Let's adjust for the dummy structure if LLM outputs just the number
        actual_node_id = f"{doc_id}_{node_id_key}" if '_' not in node_id_key and not node_id_key.startswith(doc_id) else node_id_key

        node = node_map.get(actual_node_id)
        if node:
            print(f"Node ID: {node['node_id']}\t Page: {node.get('page_index', 'N/A')}\t Title: {node['title']}")
        else:
            print(f"Node ID: {actual_node_id} (Not found in map - LLM might have hallucinated or ID format mismatch)")
except json.JSONDecodeError:
    print(f"\nError: LLM did not return valid JSON for tree search result:\n{tree_search_result}")
except Exception as e:
    print(f"\nAn error occurred during tree search result processing: {e}")


# 3.1 Extract relevant context from retrieved nodes
# Use the node_map to get the full text of the identified nodes
# Re-parse LLM result in case of error in previous block
try:
    retrieved_node_ids = json.loads(tree_search_result).get("node_list", [])
except json.JSONDecodeError:
    retrieved_node_ids = []

relevant_content = []
for node_id_key in retrieved_node_ids:
    actual_node_id = f"{doc_id}_{node_id_key}" if '_' not in node_id_key and not node_id_key.startswith(doc_id) else node_id_key
    node = node_map.get(actual_node_id)
    if node and 'text' in node:
        relevant_content.append(node['text'])

relevant_content_str = "\n\n".join(relevant_content)

print('\nRetrieved Context:\n')
if relevant_content_str:
    utils.print_wrapped(relevant_content_str[:1000] + '...' if len(relevant_content_str) > 1000 else relevant_content_str)
else:
    print("No relevant context found based on LLM's node list.")


# 3.2 Generate answer based on retrieved context
if relevant_content_str:
    answer_prompt = f"""
    Answer the question based on the context:\n\nQuestion: {query}\nContext: {relevant_content_str}\n\nProvide a clear, concise answer based only on the context provided.
    """
    print('\nCalling local LLM for answer generation...')
    answer = await call_llm(answer_prompt, model="llama3:latest") # Use llama3 or your preferred local model
    print('\nGenerated Answer:\n')
    utils.print_wrapped(answer)
else:
    print("\nCannot generate an answer as no relevant context was retrieved.")

# --- END OF COLAB CELL ---

Using dummy LocalPageIndexClient for local execution.
Attempting to fetch text from https://arxiv.org/html/2508.21069v1 for content simulation...
Extracted content to data/2508.21069v1
Document submitted locally: 2508_21069v1
Document Submitted: 2508_21069v1

Simplified Tree Structure of the Document:
Document Root (ID: 2508_21069v1_0000)

Calling local LLM for tree search (this might take a moment)...

Reasoning Process:
The question is asking about conclusions in this document, so I will look for
nodes that have 'conclusion' or 'summary' in their title or summary.

Retrieved Nodes:
Node ID: 2508_21069v1_0003 (Not found in map - LLM might have hallucinated or ID format mismatch)
Node ID: 2508_21069v1_0004 (Not found in map - LLM might have hallucinated or ID format mismatch)

Retrieved Context:

No relevant context found based on LLM's node list.

Cannot generate an answer as no relevant context was retrieved.


In [ ]:
Conclusion

In [6]:
# --- START OF COLAB CELL ---

# 0.0 Setup Environment and Install Dependencies
# This cell will install necessary packages and prepare the environment.

# Install PageIndex (if not already installed)
# %pip install -q --upgrade pageindex
# %pip install -q --upgrade openai  # For potential future use or local OpenAI-compatible servers
# %pip install -q requests beautifulsoup4  # For document downloading and parsing

import os
import requests
import json
from bs4 import BeautifulSoup

# Dummy PageIndexClient for local execution (no actual API calls)
class LocalPageIndexClient:
    def __init__(self, api_key=None):
        print("Using dummy LocalPageIndexClient for local execution.")
        self.documents = {} # Store document content
        self.trees = {} # Store generated tree structures

    def submit_document(self, file_path):
        doc_id = os.path.basename(file_path).replace('.', '_').replace('-', '_') # Simple doc_id generation
        with open(file_path, 'r', encoding='utf-8') as f:
            self.documents[doc_id] = f.read()
        print(f"Document submitted locally: {doc_id}")
        # In a real scenario, you'd process the document here to generate a tree.
        # For this example, we'll manually create a simplified tree for demonstration.
        self.trees[doc_id] = self._generate_simple_tree_from_text(self.documents[doc_id], doc_id)
        return {"doc_id": doc_id}

    def is_retrieval_ready(self, doc_id):
        return doc_id in self.trees

    def get_tree(self, doc_id, node_summary=True):
        return {"result": self.trees.get(doc_id, [])}

    def _generate_simple_tree_from_text(self, text_content, doc_id):
        # This function attempts to parse the text content into a hierarchical structure
        # based on common academic paper headings.

        root_node = {
            "title": "Document Root", # General root title
            "node_id": f"{doc_id}_0000",
            "prefix_summary": "Root of the document structure.",
            "nodes": [],
            "page_index": 1
        }

        # Define common academic section patterns to look for
        section_patterns = [
            ("## Abstract", "Abstract"),
            ("## 1 Introduction", "1. Introduction"),
            ("## 1.1", "1.1"),
            ("## 2 Related Work", "2. Related Work"),
            ("## 3 Method", "3. Method"),
            ("## 3.1", "3.1"),
            ("## 4 Experiments", "4. Experiments"),
            ("## 4.1", "4.1"),
            ("## 5 Conclusion and Future Work", "5. Conclusion and Future Work"),
            ("## References", "References"),
            ("## Appendix", "Appendix")
        ]

        parsed_sections = []
        current_section_title = "Document Root Content" # Default for un-sectioned content
        current_section_content = []
        node_counter = 0
        current_page_index = 1

        lines = text_content.split('\n')

        # First pass: Identify all main sections and their content
        for line in lines:
            stripped_line = line.strip()
            found_new_section = False
            for pattern_prefix, canonical_title in section_patterns:
                if stripped_line.startswith(pattern_prefix):
                    if current_section_content and current_section_title != "Document Root Content":
                        # Save the previous section
                        parsed_sections.append({
                            "title": current_section_title,
                            "text": "\n".join(current_section_content).strip(),
                            "node_id": f"{doc_id}_{node_counter:04d}",
                            "page_index": current_page_index
                        })
                        node_counter += 1
                        current_page_index += 1 # Simulate page increment for new section

                    current_section_title = canonical_title
                    current_section_content = [stripped_line]
                    found_new_section = True
                    break

            if not found_new_section:
                current_section_content.append(line)

        # Add the last section
        if current_section_content:
            parsed_sections.append({
                "title": current_section_title,
                "text": "\n".join(current_section_content).strip(),
                "node_id": f"{doc_id}_{node_counter:04d}",
                "page_index": current_page_index
            })

        # Second pass: Build the hierarchical tree
        node_map_for_nesting = {root_node["node_id"]: root_node} # Map to easily find parents

        for section_data in parsed_sections:
            node_id = section_data["node_id"]
            title = section_data["title"]
            text = section_data["text"]

            summary_words = text.split(' ', 20)
            summary = ' '.join(summary_words[:20]) + "..." if len(summary_words) > 20 else ' '.join(summary_words)

            node_entry = {
                "title": title,
                "node_id": node_id,
                "summary": summary,
                "text": text,
                "page_index": section_data["page_index"]
            }
            node_map_for_nesting[node_id] = node_entry # Add to map for potential children

            # Simple heuristic for nesting (based on "1." "1.1." "2." etc.)
            parent_added = False
            if title.startswith("1.") and len(title.split('.')) == 2: # e.g., "1. Introduction"
                root_node["nodes"].append(node_entry)
                parent_added = True
            elif title.startswith("1.1") or title.startswith("1.2"): # e.g., "1.1. Contributions"
                intro_node = next((n for n in root_node["nodes"] if n["title"].startswith("1.") and len(n["title"].split('.')) == 2), None)
                if intro_node:
                    if "nodes" not in intro_node: intro_node["nodes"] = []
                    intro_node["nodes"].append(node_entry)
                    parent_added = True
            elif title.startswith("2.") and len(title.split('.')) == 2: # e.g., "2. Related Work"
                root_node["nodes"].append(node_entry)
                parent_added = True
            elif title.startswith("3.") and len(title.split('.')) == 2: # e.g., "3. Method"
                root_node["nodes"].append(node_entry)
                parent_added = True
            elif title.startswith("4.") and len(title.split('.')) == 2: # e.g., "4. Experiments"
                root_node["nodes"].append(node_entry)
                parent_added = True
            elif title.startswith("5.") and len(title.split('.')) == 2: # e.g., "5. Conclusion..."
                root_node["nodes"].append(node_entry)
                parent_added = True
            elif title == "Abstract" or title == "References" or title == "Appendix":
                root_node["nodes"].append(node_entry)
                parent_added = True

            if not parent_added and title != "Document Root Content":
                # If not specifically nested, add to root
                root_node["nodes"].append(node_entry)

        # Ensure no duplicate entries in root_node if some sections were implicitly added twice
        final_root_nodes = []
        seen_node_ids = set()
        for node in root_node["nodes"]:
            if node["node_id"] not in seen_node_ids:
                final_root_nodes.append(node)
                seen_node_ids.add(node["node_id"])
        root_node["nodes"] = final_root_nodes

        return [root_node]


# Dummy utils for local execution (mimicking pageindex.utils)
class LocalUtils:
    @staticmethod
    def print_tree(tree, indent=0):
        for node in tree:
            prefix = "  " * indent
            print(f"{prefix}{node.get('title', 'Untitled')} (ID: {node.get('node_id', 'N/A')})")
            if "nodes" in node:
                LocalUtils.print_tree(node["nodes"], indent + 1)

    @staticmethod
    def remove_fields(obj, fields):
        if isinstance(obj, dict):
            return {k: LocalUtils.remove_fields(v, fields) for k, v in obj.items() if k not in fields}
        elif isinstance(obj, list):
            return [LocalUtils.remove_fields(elem, fields) for elem in obj]
        else:
            return obj

    @staticmethod
    def create_node_mapping(tree, node_map=None):
        if node_map is None:
            node_map = {}
        for node in tree:
            node_map[node.get('node_id')] = node
            if "nodes" in node:
                LocalUtils.create_node_mapping(node["nodes"], node_map)
        return node_map

    @staticmethod
    def print_wrapped(text, width=80):
        import textwrap
        print(textwrap.fill(text, width=width))

pi_client = LocalPageIndexClient()
utils = LocalUtils()

# Ollama LLM client for local inference
import httpx

async def call_llm(prompt, model="llama3:latest", temperature=0):
    # Ensure Ollama server is running and model is pulled
    # Example: ollama run llama3
    client = httpx.AsyncClient(base_url="http://localhost:11434")
    try:
        response = await client.post(
            "/api/chat",
            json={
                "model": model,
                "messages": [{"role": "user", "content": prompt}],
                "stream": False,
                "options": {"temperature": temperature}
            },
            timeout=None # Disable timeout for potentially long responses
        )
        response.raise_for_status() # Raise an exception for HTTP errors
        return response.json()["message"]["content"].strip()
    except httpx.ConnectError as e:
        print(f"Error connecting to Ollama: {e}")
        print("Please ensure Ollama is running (e.g., 'ollama serve') and you have pulled the model (e.g., 'ollama pull llama3').")
        return "ERROR: Could not connect to Ollama. Please check your Ollama setup."
    except httpx.HTTPStatusError as e:
        print(f"Error from Ollama server: {e.response.status_code} - {e.response.text}")
        return f"ERROR: Ollama server responded with an error: {e.response.status_code}"
    except json.JSONDecodeError:
        print(f"Error decoding JSON from Ollama: {response.text}")
        return "ERROR: Invalid JSON response from Ollama."
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return f"ERROR: An unexpected error occurred: {e}"

# 1.1 Submit a document for generating PageIndex tree (locally)
# CHANGED: New URL for the document
pdf_url = "https://arxiv.org/pdf/2508.21069" # This is a placeholder as .pdf might not exist for future
arxiv_html_url = "https://arxiv.org/html/2508.21069v1" # Direct HTML link

pdf_filename = arxiv_html_url.split('/')[-1].replace('.html', '.pdf') # Simulate pdf filename from html
pdf_path = os.path.join("data", pdf_filename) # Use a 'data' directory

os.makedirs(os.path.dirname(pdf_path), exist_ok=True)


print(f"Attempting to fetch text from {arxiv_html_url} for content simulation...")
try:
    html_response = requests.get(arxiv_html_url)
    html_response.raise_for_status()
    soup = BeautifulSoup(html_response.text, 'html.parser')

    # Extract main content - this will be highly dependent on arXiv's HTML structure
    extracted_text_parts = []
    # Look for common article structure elements
    for tag in soup.find_all(['h1', 'h2', 'h3', 'h4', 'p', 'li', 'section']):
        text = tag.get_text(separator=' ', strip=True)
        if text:
            extracted_text_parts.append(text)

    dummy_text_content = "\n\n".join(extracted_text_parts)
    if not dummy_text_content.strip():
        raise ValueError("Could not extract significant text from arXiv HTML.")

    text_path = pdf_path.replace('.pdf', '.txt')
    with open(text_path, "w", encoding="utf-8") as f:
        f.write(dummy_text_content)
    print(f"Extracted content to {text_path}")
    doc_id = pi_client.submit_document(text_path)["doc_id"]
    print('Document Submitted:', doc_id)

except Exception as e:
    print(f"Failed to extract text from arXiv HTML or save to file: {e}")
    print("Falling back to a very minimal dummy text for tree generation.")
    # UPDATED dummy text for the new URL if HTML extraction fails
    dummy_text_content = """
    # Deep Learning for Climate Model Emulation
    ## Abstract
    This paper explores the use of deep learning models to emulate complex climate simulations, offering faster predictions and insights. We demonstrate that neural networks can accurately reproduce outputs of sophisticated climate models.
    ## 1 Introduction
    Climate change research relies heavily on complex numerical simulations. These are computationally expensive. Deep learning offers a promising alternative.
    ## 2 Related Work
    Previous studies have used machine learning for atmospheric processes. Our work focuses on full climate model emulation.
    ## 3 Method
    We employed a U-Net architecture trained on high-resolution climate model data. Input features include atmospheric variables; outputs are future climate states.
    ## 4 Experiments
    Our experiments show that the DL emulator achieves high fidelity compared to the full climate model across various metrics, including temperature and precipitation patterns. It significantly reduces computational time.
    ## 5 Conclusion and Future Work
    Deep learning emulation is a powerful tool for climate science. We conclude that DL models can effectively surrogate complex physical processes. Future work includes expanding to different climate models and uncertainty quantification.
    ## References
    [1] Smith et al. Climate Modeling.
    [2] Jones et al. Deep Learning for Earth Systems.
    """
    text_path = pdf_path.replace('.pdf', '.txt')
    with open(text_path, "w", encoding="utf-8") as f:
        f.write(dummy_text_content)
    print(f"Used minimal dummy text and saved to {text_path}")
    doc_id = pi_client.submit_document(text_path)["doc_id"]
    print('Document Submitted:', doc_id)


# 1.2 Get the generated PageIndex tree structure
# This will use the simplified tree generated by our LocalPageIndexClient
if pi_client.is_retrieval_ready(doc_id):
    tree = pi_client.get_tree(doc_id, node_summary=True)['result']
    print('\nSimplified Tree Structure of the Document:')
    utils.print_tree(tree)
else:
    print("Processing document, please try again later...")

# 2.1 Use LLM for tree search and identify nodes that might contain relevant context
# CHANGED: New query
query = "What are the Conclusion in this document?"

# Remove the 'text' field to avoid sending too much data to the LLM for tree search
tree_without_text = utils.remove_fields(tree.copy(), fields=['text'])

search_prompt = f"""
You are given a question and a tree structure of a document.
Each node contains a node id, node title, and a corresponding summary.
Your task is to find all nodes that are likely to contain the answer to the question.

Question: {query}

Document tree structure:
{json.dumps(tree_without_text, indent=2)}

Please reply in the following JSON format:
{{
    "thinking": "<Your thinking process on which nodes are relevant to the question>",
    "node_list": ["node_id_1", "node_id_2", ..., "node_id_n"]
}}
Directly return the final JSON structure. Do not output anything else.
"""

print("\nCalling local LLM for tree search (this might take a moment)...")
tree_search_result = await call_llm(search_prompt, model="llama3:latest") # Use llama3 or your preferred local model

# 2.2 Print retrieved nodes and reasoning process
try:
    node_map = utils.create_node_mapping(tree)
    tree_search_result_json = json.loads(tree_search_result)

    print('\nReasoning Process:')
    utils.print_wrapped(tree_search_result_json.get('thinking', 'No thinking process provided by LLM.'))

    print('\nRetrieved Nodes:')
    retrieved_node_ids = tree_search_result_json.get("node_list", [])
    if not retrieved_node_ids:
        print("No nodes retrieved by LLM.")
    for node_id_key in retrieved_node_ids:
        # The node_id from the LLM might be just '0019', need to prepend doc_id if that's how we store
        # For this dummy client, the node_ids are like "doc_id_0019"
        # Let's adjust for the dummy structure if LLM outputs just the number
        actual_node_id = f"{doc_id}_{node_id_key}" if '_' not in node_id_key and not node_id_key.startswith(doc_id) else node_id_key

        node = node_map.get(actual_node_id)
        if node:
            print(f"Node ID: {node['node_id']}\t Page: {node.get('page_index', 'N/A')}\t Title: {node['title']}")
        else:
            print(f"Node ID: {actual_node_id} (Not found in map - LLM might have hallucinated or ID format mismatch)")
except json.JSONDecodeError:
    print(f"\nError: LLM did not return valid JSON for tree search result:\n{tree_search_result}")
except Exception as e:
    print(f"\nAn error occurred during tree search result processing: {e}")


# 3.1 Extract relevant context from retrieved nodes
# Use the node_map to get the full text of the identified nodes
# Re-parse LLM result in case of error in previous block
try:
    retrieved_node_ids = json.loads(tree_search_result).get("node_list", [])
except json.JSONDecodeError:
    retrieved_node_ids = []

relevant_content = []
for node_id_key in retrieved_node_ids:
    actual_node_id = f"{doc_id}_{node_id_key}" if '_' not in node_id_key and not node_id_key.startswith(doc_id) else node_id_key
    node = node_map.get(actual_node_id)
    if node and 'text' in node:
        relevant_content.append(node['text'])

relevant_content_str = "\n\n".join(relevant_content)

print('\nRetrieved Context:\n')
if relevant_content_str:
    utils.print_wrapped(relevant_content_str[:1000] + '...' if len(relevant_content_str) > 1000 else relevant_content_str)
else:
    print("No relevant context found based on LLM's node list.")


# 3.2 Generate answer based on retrieved context
if relevant_content_str:
    answer_prompt = f"""
    Answer the question based on the context:\n\nQuestion: {query}\nContext: {relevant_content_str}\n\nProvide a clear, concise answer based only on the context provided.
    """
    print('\nCalling local LLM for answer generation...')
    answer = await call_llm(answer_prompt, model="llama3:latest") # Use llama3 or your preferred local model
    print('\nGenerated Answer:\n')
    utils.print_wrapped(answer)
else:
    print("\nCannot generate an answer as no relevant context was retrieved.")

# --- END OF COLAB CELL ---

Using dummy LocalPageIndexClient for local execution.
Attempting to fetch text from https://arxiv.org/html/2508.21069v1 for content simulation...
Extracted content to data/2508.21069v1
Document submitted locally: 2508_21069v1
Document Submitted: 2508_21069v1

Simplified Tree Structure of the Document:
Document Root (ID: 2508_21069v1_0000)

Calling local LLM for tree search (this might take a moment)...

Reasoning Process:
Since we're looking for 'Conclusion' in this document, I'll focus on nodes that
have a title or summary containing the word 'Conclusion'.

Retrieved Nodes:
Node ID: 2508_21069v1_0003 (Not found in map - LLM might have hallucinated or ID format mismatch)

Retrieved Context:

No relevant context found based on LLM's node list.

Cannot generate an answer as no relevant context was retrieved.


In [8]:
# --- START OF COLAB CELL ---

# 0.0 Setup Environment and Install Dependencies
# This cell will install necessary packages and prepare the environment.

# Install PageIndex (if not already installed)
# %pip install -q --upgrade pageindex
# %pip install -q --upgrade openai  # For potential future use or local OpenAI-compatible servers
# %pip install -q requests beautifulsoup4  # For document downloading and parsing

import os
import requests
import json
from bs4 import BeautifulSoup

# Dummy PageIndexClient for local execution (no actual API calls)
class LocalPageIndexClient:
    def __init__(self, api_key=None):
        print("Using dummy LocalPageIndexClient for local execution.")
        self.documents = {} # Store document content
        self.trees = {} # Store generated tree structures

    def submit_document(self, file_path):
        doc_id = os.path.basename(file_path).replace('.', '_').replace('-', '_') # Simple doc_id generation
        with open(file_path, 'r', encoding='utf-8') as f:
            self.documents[doc_id] = f.read()
        print(f"Document submitted locally: {doc_id}")
        # In a real scenario, you'd process the document here to generate a tree.
        # For this example, we'll manually create a simplified tree for demonstration.
        self.trees[doc_id] = self._generate_simple_tree_from_text(self.documents[doc_id], doc_id)
        return {"doc_id": doc_id}

    def is_retrieval_ready(self, doc_id):
        return doc_id in self.trees

    def get_tree(self, doc_id, node_summary=True):
        return {"result": self.trees.get(doc_id, [])}

    def _generate_simple_tree_from_text(self, text_content, doc_id):
        # This function attempts to parse the text content into a hierarchical structure
        # based on common academic paper headings.

        root_node = {
            "title": "Document Root", # General root title
            "node_id": f"{doc_id}_0000",
            "prefix_summary": "Root of the document structure.",
            "nodes": [],
            "page_index": 1
        }

        # Define common academic section patterns to look for
        section_patterns = [
            ("## Abstract", "Abstract"),
            ("## 1 Introduction", "1. Introduction"),
            ("## 1.1", "1.1"),
            ("## 2 Related Work", "2. Related Work"),
            ("## 3 Method", "3. Method"),
            ("## 3.1", "3.1"),
            ("## 4 Experiments", "4. Experiments"),
            ("## 4.1", "4.1"),
            ("## 5 Conclusion and Future Work", "5. Conclusion and Future Work"),
            ("## References", "References"),
            ("## Appendix", "Appendix")
        ]

        parsed_sections = []
        current_section_title = "Document Root Content" # Default for un-sectioned content
        current_section_content = []
        node_counter = 0
        current_page_index = 1

        lines = text_content.split('\n')

        # First pass: Identify all main sections and their content
        for line in lines:
            stripped_line = line.strip()
            found_new_section = False
            for pattern_prefix, canonical_title in section_patterns:
                if stripped_line.startswith(pattern_prefix):
                    if current_section_content and current_section_title != "Document Root Content":
                        # Save the previous section
                        parsed_sections.append({
                            "title": current_section_title,
                            "text": "\n".join(current_section_content).strip(),
                            "node_id": f"{doc_id}_{node_counter:04d}",
                            "page_index": current_page_index
                        })
                        node_counter += 1
                        current_page_index += 1 # Simulate page increment for new section

                    current_section_title = canonical_title
                    current_section_content = [stripped_line]
                    found_new_section = True
                    break

            if not found_new_section:
                current_section_content.append(line)

        # Add the last section
        if current_section_content:
            parsed_sections.append({
                "title": current_section_title,
                "text": "\n".join(current_section_content).strip(),
                "node_id": f"{doc_id}_{node_counter:04d}",
                "page_index": current_page_index
            })

        # Second pass: Build the hierarchical tree
        node_map_for_nesting = {root_node["node_id"]: root_node} # Map to easily find parents

        for section_data in parsed_sections:
            node_id = section_data["node_id"]
            title = section_data["title"]
            text = section_data["text"]

            summary_words = text.split(' ', 20)
            summary = ' '.join(summary_words[:20]) + "..." if len(summary_words) > 20 else ' '.join(summary_words)

            node_entry = {
                "title": title,
                "node_id": node_id,
                "summary": summary,
                "text": text,
                "page_index": section_data["page_index"]
            }
            node_map_for_nesting[node_id] = node_entry # Add to map for potential children

            # Simple heuristic for nesting (based on "1." "1.1." "2." etc.)
            parent_added = False
            if title.startswith("1.") and len(title.split('.')) == 2: # e.g., "1. Introduction"
                root_node["nodes"].append(node_entry)
                parent_added = True
            elif title.startswith("1.1") or title.startswith("1.2"): # e.g., "1.1. Contributions"
                intro_node = next((n for n in root_node["nodes"] if n["title"].startswith("1.") and len(n["title"].split('.')) == 2), None)
                if intro_node:
                    if "nodes" not in intro_node: intro_node["nodes"] = []
                    intro_node["nodes"].append(node_entry)
                    parent_added = True
            elif title.startswith("2.") and len(title.split('.')) == 2: # e.g., "2. Related Work"
                root_node["nodes"].append(node_entry)
                parent_added = True
            elif title.startswith("3.") and len(title.split('.')) == 2: # e.g., "3. Method"
                root_node["nodes"].append(node_entry)
                parent_added = True
            elif title.startswith("4.") and len(title.split('.')) == 2: # e.g., "4. Experiments"
                root_node["nodes"].append(node_entry)
                parent_added = True
            elif title.startswith("5.") and len(title.split('.')) == 2: # e.g., "5. Conclusion..."
                root_node["nodes"].append(node_entry)
                parent_added = True
            elif title == "Abstract" or title == "References" or title == "Appendix":
                root_node["nodes"].append(node_entry)
                parent_added = True

            if not parent_added and title != "Document Root Content":
                # If not specifically nested, add to root
                root_node["nodes"].append(node_entry)

        # Ensure no duplicate entries in root_node if some sections were implicitly added twice
        final_root_nodes = []
        seen_node_ids = set()
        for node in root_node["nodes"]:
            if node["node_id"] not in seen_node_ids:
                final_root_nodes.append(node)
                seen_node_ids.add(node["node_id"])
        root_node["nodes"] = final_root_nodes

        return [root_node]


# Dummy utils for local execution (mimicking pageindex.utils)
class LocalUtils:
    @staticmethod
    def print_tree(tree, indent=0):
        for node in tree:
            prefix = "  " * indent
            print(f"{prefix}{node.get('title', 'Untitled')} (ID: {node.get('node_id', 'N/A')})")
            if "nodes" in node:
                LocalUtils.print_tree(node["nodes"], indent + 1)

    @staticmethod
    def remove_fields(obj, fields):
        if isinstance(obj, dict):
            return {k: LocalUtils.remove_fields(v, fields) for k, v in obj.items() if k not in fields}
        elif isinstance(obj, list):
            return [LocalUtils.remove_fields(elem, fields) for elem in obj]
        else:
            return obj

    @staticmethod
    def create_node_mapping(tree, node_map=None):
        if node_map is None:
            node_map = {}
        for node in tree:
            node_map[node.get('node_id')] = node
            if "nodes" in node:
                LocalUtils.create_node_mapping(node["nodes"], node_map)
        return node_map

    @staticmethod
    def print_wrapped(text, width=80):
        import textwrap
        print(textwrap.fill(text, width=width))

pi_client = LocalPageIndexClient()
utils = LocalUtils()

# Ollama LLM client for local inference
import httpx

async def call_llm(prompt, model="llama3:latest", temperature=0):
    # Ensure Ollama server is running and model is pulled
    # Example: ollama run llama3
    client = httpx.AsyncClient(base_url="http://localhost:11434")
    try:
        response = await client.post(
            "/api/chat",
            json={
                "model": model,
                "messages": [{"role": "user", "content": prompt}],
                "stream": False,
                "options": {"temperature": temperature}
            },
            timeout=None # Disable timeout for potentially long responses
        )
        response.raise_for_status() # Raise an exception for HTTP errors
        return response.json()["message"]["content"].strip()
    except httpx.ConnectError as e:
        print(f"Error connecting to Ollama: {e}")
        print("Please ensure Ollama is running (e.g., 'ollama serve') and you have pulled the model (e.g., 'ollama pull llama3').")
        return "ERROR: Could not connect to Ollama. Please check your Ollama setup."
    except httpx.HTTPStatusError as e:
        print(f"Error from Ollama server: {e.response.status_code} - {e.response.text}")
        return f"ERROR: Ollama server responded with an error: {e.response.status_code}"
    except json.JSONDecodeError:
        print(f"Error decoding JSON from Ollama: {response.text}")
        return "ERROR: Invalid JSON response from Ollama."
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return f"ERROR: An unexpected error occurred: {e}"

# 1.1 Submit a document for generating PageIndex tree (locally)
# CHANGED: New URL for the document
pdf_url = "https://arxiv.org/pdf/2508.21069" # This is a placeholder as .pdf might not exist for future
arxiv_html_url = "https://arxiv.org/html/2508.21069v1" # Direct HTML link

pdf_filename = arxiv_html_url.split('/')[-1].replace('.html', '.pdf') # Simulate pdf filename from html
pdf_path = os.path.join("data", pdf_filename) # Use a 'data' directory

os.makedirs(os.path.dirname(pdf_path), exist_ok=True)


print(f"Attempting to fetch text from {arxiv_html_url} for content simulation...")
try:
    html_response = requests.get(arxiv_html_url)
    html_response.raise_for_status()
    soup = BeautifulSoup(html_response.text, 'html.parser')

    # Extract main content - this will be highly dependent on arXiv's HTML structure
    extracted_text_parts = []
    # Look for common article structure elements
    for tag in soup.find_all(['h1', 'h2', 'h3', 'h4', 'p', 'li', 'section']):
        text = tag.get_text(separator=' ', strip=True)
        if text:
            extracted_text_parts.append(text)

    dummy_text_content = "\n\n".join(extracted_text_parts)
    if not dummy_text_content.strip():
        raise ValueError("Could not extract significant text from arXiv HTML.")

    text_path = pdf_path.replace('.pdf', '.txt')
    with open(text_path, "w", encoding="utf-8") as f:
        f.write(dummy_text_content)
    print(f"Extracted content to {text_path}")
    doc_id = pi_client.submit_document(text_path)["doc_id"]
    print('Document Submitted:', doc_id)

except Exception as e:
    print(f"Failed to extract text from arXiv HTML or save to file: {e}")
    print("Falling back to a very minimal dummy text for tree generation.")
    # UPDATED dummy text for the new URL if HTML extraction fails
    dummy_text_content = """
    # Deep Learning for Climate Model Emulation
    ## Abstract
    This paper explores the use of deep learning models to emulate complex climate simulations, offering faster predictions and insights. We demonstrate that neural networks can accurately reproduce outputs of sophisticated climate models.
    ## 1 Introduction
    Climate change research relies heavily on complex numerical simulations. These are computationally expensive. Deep learning offers a promising alternative.
    ## 2 Related Work
    Previous studies have used machine learning for atmospheric processes. Our work focuses on full climate model emulation.
    ## 3 Method
    We employed a U-Net architecture trained on high-resolution climate model data. Input features include atmospheric variables; outputs are future climate states.
    ## 4 Experiments
    Our experiments show that the DL emulator achieves high fidelity compared to the full climate model across various metrics, including temperature and precipitation patterns. It significantly reduces computational time.
    ## 5 Conclusion and Future Work
    Deep learning emulation is a powerful tool for climate science. We conclude that DL models can effectively surrogate complex physical processes. Future work includes expanding to different climate models and uncertainty quantification.
    ## References
    [1] Smith et al. Climate Modeling.
    [2] Jones et al. Deep Learning for Earth Systems.
    """
    text_path = pdf_path.replace('.pdf', '.txt')
    with open(text_path, "w", encoding="utf-8") as f:
        f.write(dummy_text_content)
    print(f"Used minimal dummy text and saved to {text_path}")
    doc_id = pi_client.submit_document(text_path)["doc_id"]
    print('Document Submitted:', doc_id)


# 1.2 Get the generated PageIndex tree structure
# This will use the simplified tree generated by our LocalPageIndexClient
if pi_client.is_retrieval_ready(doc_id):
    tree = pi_client.get_tree(doc_id, node_summary=True)['result']
    print('\nSimplified Tree Structure of the Document:')
    utils.print_tree(tree)
else:
    print("Processing document, please try again later...")

# 2.1 Use LLM for tree search and identify nodes that might contain relevant context
# CHANGED: New query
query = "What are the conclusion in this document?"

# Remove the 'text' field to avoid sending too much data to the LLM for tree search
tree_without_text = utils.remove_fields(tree.copy(), fields=['text'])

search_prompt = f"""
You are given a question and a tree structure of a document.
Each node contains a node id, node title, and a corresponding summary.
Your task is to find all nodes that are likely to contain the answer to the question.

Question: {query}

Document tree structure:
{json.dumps(tree_without_text, indent=2)}

Please reply in the following JSON format:
{{
    "thinking": "<Your thinking process on which nodes are relevant to the question>",
    "node_list": ["node_id_1", "node_id_2", ..., "node_id_n"]
}}
Directly return the final JSON structure. Do not output anything else.
"""

print("\nCalling local LLM for tree search (this might take a moment)...")
tree_search_result = await call_llm(search_prompt, model="llama3:latest") # Use llama3 or your preferred local model

# 2.2 Print retrieved nodes and reasoning process
try:
    node_map = utils.create_node_mapping(tree)
    tree_search_result_json = json.loads(tree_search_result)

    print('\nReasoning Process:')
    utils.print_wrapped(tree_search_result_json.get('thinking', 'No thinking process provided by LLM.'))

    print('\nRetrieved Nodes:')
    retrieved_node_ids = tree_search_result_json.get("node_list", [])
    if not retrieved_node_ids:
        print("No nodes retrieved by LLM.")
    for node_id_key in retrieved_node_ids:
        # The node_id from the LLM might be just '0019', need to prepend doc_id if that's how we store
        # For this dummy client, the node_ids are like "doc_id_0019"
        # Let's adjust for the dummy structure if LLM outputs just the number
        actual_node_id = f"{doc_id}_{node_id_key}" if '_' not in node_id_key and not node_id_key.startswith(doc_id) else node_id_key

        node = node_map.get(actual_node_id)
        if node:
            print(f"Node ID: {node['node_id']}\t Page: {node.get('page_index', 'N/A')}\t Title: {node['title']}")
        else:
            print(f"Node ID: {actual_node_id} (Not found in map - LLM might have hallucinated or ID format mismatch)")
except json.JSONDecodeError:
    print(f"\nError: LLM did not return valid JSON for tree search result:\n{tree_search_result}")
except Exception as e:
    print(f"\nAn error occurred during tree search result processing: {e}")


# 3.1 Extract relevant context from retrieved nodes
# Use the node_map to get the full text of the identified nodes
# Re-parse LLM result in case of error in previous block
try:
    retrieved_node_ids = json.loads(tree_search_result).get("node_list", [])
except json.JSONDecodeError:
    retrieved_node_ids = []

relevant_content = []
for node_id_key in retrieved_node_ids:
    actual_node_id = f"{doc_id}_{node_id_key}" if '_' not in node_id_key and not node_id_key.startswith(doc_id) else node_id_key
    node = node_map.get(actual_node_id)
    if node and 'text' in node:
        relevant_content.append(node['text'])

relevant_content_str = "\n\n".join(relevant_content)

print('\nRetrieved Context:\n')
if relevant_content_str:
    utils.print_wrapped(relevant_content_str[:1000] + '...' if len(relevant_content_str) > 1000 else relevant_content_str)
else:
    print("No relevant context found based on LLM's node list.")


# 3.2 Generate answer based on retrieved context
if relevant_content_str:
    answer_prompt = f"""
    Answer the question based on the context:\n\nQuestion: {query}\nContext: {relevant_content_str}\n\nProvide a clear, concise answer based only on the context provided.
    """
    print('\nCalling local LLM for answer generation...')
    answer = await call_llm(answer_prompt, model="llama3:latest") # Use llama3 or your preferred local model
    print('\nGenerated Answer:\n')
    utils.print_wrapped(answer)
else:
    print("\nCannot generate an answer as no relevant context was retrieved.")

# --- END OF COLAB CELL ---

Using dummy LocalPageIndexClient for local execution.
Attempting to fetch text from https://arxiv.org/html/2508.21069v1 for content simulation...
Extracted content to data/2508.21069v1
Document submitted locally: 2508_21069v1
Document Submitted: 2508_21069v1

Simplified Tree Structure of the Document:
Document Root (ID: 2508_21069v1_0000)

Calling local LLM for tree search (this might take a moment)...

Reasoning Process:
The question is asking about conclusions in the document, so I will look for
nodes that have 'conclusion' or 'summary' in their title or summary.

Retrieved Nodes:
Node ID: 2508_21069v1_0002 (Not found in map - LLM might have hallucinated or ID format mismatch)
Node ID: 2508_21069v1_0003 (Not found in map - LLM might have hallucinated or ID format mismatch)

Retrieved Context:

No relevant context found based on LLM's node list.

Cannot generate an answer as no relevant context was retrieved.


نجاح: الكود نجح في تحميل المحتوى من الرابط الجديد https://arxiv.org/html/2508.21069v1. هذا رائع!
فشل: المشكلة الأساسية كانت في دالة _generate_simple_tree_from_text. لقد كانت مصممة لتحليل "النص الوهمي" الذي يحتوي على علامات مثل ##، لكنها فشلت في تحليل النص الحقيقي الذي تم سحبه من صفحة الويب، والذي لا يحتوي على هذه العلامات. ونتيجة لذلك، قامت بإنشاء "شجرة" تحتوي على عقدة جذرية واحدة فقط، مما جعلها عديمة الفائدة.
هلوسة النموذج (LLM): عندما تم إعطاء نموذج اللغة شجرة فارغة وطلب منه العثور على "الخاتمة"، قام باختلاق (هلوسة) مُعرّف عقدة (_0003) لأنه لم يجد أي شيء حقيقي في السياق الذي قُدم له.
النتيجة النهائية: بما أن مُعرّف العقدة المُختلق غير موجود في خريطة العقد (لأن الخريطة كانت فارغة تقريبًا)، فشل البرنامج في استرداد أي سياق ولم يتمكن من توليد إجابة.
الحل:
الحل هو إعادة كتابة دالة _generate_simple_tree_from_text لتكون أكثر ذكاءً وقوة، بحيث يمكنها تحليل النص الفعلي المستخرج من الويب. النسخة الجديدة ستقوم بالبحث عن عناوين الأقسام الشائعة (مثل "Abstract", "Introduction", "Conclusion") بطريقة أكثر مرونة، دون الاعتماد على علامات ##.
لقد قمت بتعديل هذه الدالة بشكل كبير لتقسيم النص إلى أقسام بشكل صحيح، مما سيبني شجرة مستندات مفيدة يمكن للنموذج اللغوي استخدامها.
الكود الكامل مع التعديلات:
إليك الكود الكامل والمصحح. التغيير الرئيسي موجود داخل الكلاس LocalPageIndexClient في دالة _generate_simple_tree_from_text.
code
Python


In [2]:
# --- START OF COLAB CELL ---

# 0.0 Setup Environment and Install Dependencies
# This cell will install necessary packages and prepare the environment.

# Install PageIndex (if not already installed)
# %pip install -q --upgrade pageindex
# %pip install -q --upgrade openai  # For potential future use or local OpenAI-compatible servers
# %pip install -q requests beautifulsoup4  # For document downloading and parsing

import os
import requests
import json
from bs4 import BeautifulSoup

# Dummy PageIndexClient for local execution (no actual API calls)
class LocalPageIndexClient:
    def __init__(self, api_key=None):
        print("Using dummy LocalPageIndexClient for local execution.")
        self.documents = {} # Store document content
        self.trees = {} # Store generated tree structures

    def submit_document(self, file_path):
        doc_id = os.path.basename(file_path).replace('.', '_').replace('-', '_') # Simple doc_id generation
        with open(file_path, 'r', encoding='utf-8') as f:
            self.documents[doc_id] = f.read()
        print(f"Document submitted locally: {doc_id}")
        self.trees[doc_id] = self._generate_simple_tree_from_text(self.documents[doc_id], doc_id)
        return {"doc_id": doc_id}

    def is_retrieval_ready(self, doc_id):
        return doc_id in self.trees

    def get_tree(self, doc_id, node_summary=True):
        return {"result": self.trees.get(doc_id, [])}

    # --- START OF MAJOR CORRECTION ---
    def _generate_simple_tree_from_text(self, text_content, doc_id):
        # This is a more robust parser for the actual scraped HTML text.
        root_node = {
            "title": "Document Root",
            "node_id": f"{doc_id}_0000",
            "prefix_summary": "Root of the document structure.",
            "nodes": [],
            "page_index": 1
        }

        parsed_sections = []
        current_title = "Preamble"  # Content before the first proper header
        current_content = []

        # More flexible headers to look for in the scraped text
        # These are keywords that often start a section title.
        header_keywords = [
            "Abstract", "Introduction", "Related Work", "Method",
            "Experiments", "Conclusion", "References", "Appendix"
        ]

        lines = text_content.split('\n')
        for line in lines:
            stripped = line.strip()
            if not stripped:
                continue

            found_header = None
            # Heuristic: A line is likely a header if it's short and starts with a keyword or a number.
            if len(stripped.split()) < 10:
                for keyword in header_keywords:
                    # Check for "1 Introduction", "5. Conclusion", "Abstract", etc.
                    if (stripped.startswith(keyword) or (len(stripped) > 1 and stripped[0].isdigit() and keyword in stripped)):
                        found_header = stripped
                        break

            if found_header:
                # If we found a new header, save the previous section's content
                if current_content:
                    parsed_sections.append({
                        "title": current_title,
                        "text": "\n".join(current_content).strip()
                    })
                # Start the new section
                current_title = found_header
                current_content = [] # Reset content buffer
            else:
                # This line is content for the current section
                current_content.append(line)

        # After the loop, save the very last section
        if current_content:
            parsed_sections.append({
                "title": current_title,
                "text": "\n".join(current_content).strip()
            })

        # Now, build the tree from the correctly parsed sections
        node_counter = 1
        for i, section in enumerate(parsed_sections):
            text = section["text"]
            summary = ' '.join(text.split()[:20]) + '...' if len(text.split()) > 20 else text

            node_entry = {
                "title": section["title"],
                "node_id": f"{doc_id}_{i+1:04d}", # Simple, sequential node IDs
                "summary": summary,
                "text": text,
                "page_index": i + 1
            }
            root_node["nodes"].append(node_entry)

        return [root_node]
    # --- END OF MAJOR CORRECTION ---

# Dummy utils for local execution (mimicking pageindex.utils)
class LocalUtils:
    @staticmethod
    def print_tree(tree, indent=0):
        for node in tree:
            prefix = "  " * indent
            print(f"{prefix}{node.get('title', 'Untitled')} (ID: {node.get('node_id', 'N/A')})")
            if "nodes" in node:
                LocalUtils.print_tree(node["nodes"], indent + 1)

    @staticmethod
    def remove_fields(obj, fields):
        if isinstance(obj, dict):
            return {k: LocalUtils.remove_fields(v, fields) for k, v in obj.items() if k not in fields}
        elif isinstance(obj, list):
            return [LocalUtils.remove_fields(elem, fields) for elem in obj]
        else:
            return obj

    @staticmethod
    def create_node_mapping(tree, node_map=None):
        if node_map is None:
            node_map = {}
        for node in tree:
            node_map[node.get('node_id')] = node
            if "nodes" in node:
                LocalUtils.create_node_mapping(node["nodes"], node_map)
        return node_map

    @staticmethod
    def print_wrapped(text, width=80):
        import textwrap
        print(textwrap.fill(text, width=width))

pi_client = LocalPageIndexClient()
utils = LocalUtils()

# Ollama LLM client for local inference
import httpx

async def call_llm(prompt, model="llama3", temperature=0):
    client = httpx.AsyncClient(base_url="http://localhost:11434")
    try:
        response = await client.post(
            "/api/chat",
            json={
                "model": model,
                "messages": [{"role": "user", "content": prompt}],
                "stream": False,
                "options": {"temperature": temperature}
            },
            timeout=None
        )
        response.raise_for_status()
        return response.json()["message"]["content"].strip()
    except httpx.ConnectError as e:
        print(f"Error connecting to Ollama: {e}\nPlease ensure Ollama is running.")
        return '{"thinking": "Error: Could not connect to Ollama.", "node_list": []}'
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return '{"thinking": "An unexpected error occurred.", "node_list": []}'

# 1.1 Submit a document for generating PageIndex tree (locally)
arxiv_html_url = "https://arxiv.org/html/2508.21069v1"
doc_name = arxiv_html_url.split('/')[-1]
doc_path = os.path.join("data", doc_name)

os.makedirs(os.path.dirname(doc_path), exist_ok=True)

print(f"Attempting to fetch text from {arxiv_html_url} for content simulation...")
try:
    html_response = requests.get(arxiv_html_url)
    html_response.raise_for_status()
    soup = BeautifulSoup(html_response.text, 'html.parser')

    article_body = soup.find('div', class_='ltx_page_content')
    if article_body:
        text_content = article_body.get_text(separator='\n', strip=True)
    else: # Fallback if specific class not found
        text_content = soup.get_text(separator='\n', strip=True)

    if not text_content.strip():
        raise ValueError("Could not extract significant text from arXiv HTML.")

    text_path = doc_path.replace('.html', '.txt')
    with open(text_path, "w", encoding="utf-8") as f:
        f.write(text_content)
    print(f"Extracted content to {text_path}")
    doc_id = pi_client.submit_document(text_path)["doc_id"]
    print('Document Submitted:', doc_id)

except Exception as e:
    print(f"Failed to extract text from arXiv HTML or save to file: {e}")
    doc_id = None # Ensure doc_id is None if setup fails

# 1.2 Get the generated PageIndex tree structure
if 'doc_id' in locals() and doc_id and pi_client.is_retrieval_ready(doc_id):
    tree = pi_client.get_tree(doc_id, node_summary=True)['result']
    print('\nSimplified Tree Structure of the Document:')
    utils.print_tree(tree)
else:
    print("\nSkipping further steps because document processing failed.")
    tree = None

if tree:
    # 2.1 Use LLM for tree search
    query = "What are the main findings of this document?"
    tree_without_text = utils.remove_fields(tree.copy(), fields=['text'])

    search_prompt = f"""
You are given a question and a tree structure of a document.
Each node contains a node id and a title. Your task is to find all nodes that are likely to contain the answer to the question. Focus on sections like 'Abstract', 'Experiments', 'Results', or 'Conclusion'.

Question: {query}

Document tree structure:
{json.dumps(tree_without_text, indent=2)}

Please reply ONLY with a JSON object in the following format, with no other text:
{{
    "thinking": "<Your brief thinking process on which nodes are relevant>",
    "node_list": ["<full_node_id_1>", "<full_node_id_2>"]
}}
"""

    print("\nCalling local LLM for tree search (this might take a moment)...")
    tree_search_result = await call_llm(search_prompt, model="llama3:latest")

    # 2.2 Print retrieved nodes and reasoning process
    try:
        node_map = utils.create_node_mapping(tree)
        tree_search_result_json = json.loads(tree_search_result)

        print('\nReasoning Process:')
        utils.print_wrapped(tree_search_result_json.get('thinking', 'No thinking process provided.'))

        print('\nRetrieved Nodes:')
        retrieved_node_ids = tree_search_result_json.get("node_list", [])
        if not retrieved_node_ids:
            print("No nodes retrieved by LLM.")

        for node_id in retrieved_node_ids:
            node = node_map.get(node_id)
            if node:
                print(f"Node ID: {node['node_id']}\t Page: {node.get('page_index', 'N/A')}\t Title: {node['title']}")
            else:
                print(f"Node ID: {node_id} (Not found in map - LLM may have hallucinated or ID format is mismatched)")

    except json.JSONDecodeError:
        print(f"\nError: LLM did not return valid JSON for tree search result:\n{tree_search_result}")
        retrieved_node_ids = []
    except Exception as e:
        print(f"\nAn error occurred during tree search result processing: {e}")
        retrieved_node_ids = []

    # 3.1 Extract relevant context
    relevant_content = []
    if retrieved_node_ids:
        for node_id in retrieved_node_ids:
            node = node_map.get(node_id)
            if node and 'text' in node:
                relevant_content.append(f"--- From Section: {node['title']} ---\n{node['text']}")

    relevant_content_str = "\n\n".join(relevant_content)

    print('\nRetrieved Context:\n')
    if relevant_content_str:
        utils.print_wrapped(relevant_content_str[:1500] + '...' if len(relevant_content_str) > 1500 else relevant_content_str)
    else:
        print("No relevant context found based on LLM's node list.")

    # 3.2 Generate answer
    if relevant_content_str:
        answer_prompt = f"Based ONLY on the following context, what are the main findings of the document?\n\nContext:\n{relevant_content_str}\n\nAnswer:"

        print('\nCalling local LLM for answer generation...')
        answer = await call_llm(answer_prompt, model="llama3:latest")
        print('\nGenerated Answer:\n')
        utils.print_wrapped(answer)
    else:
        print("\nCannot generate an answer as no relevant context was retrieved.")

# --- END OF COLAB CELL ---

Using dummy LocalPageIndexClient for local execution.
Attempting to fetch text from https://arxiv.org/html/2508.21069v1 for content simulation...
Extracted content to data/2508.21069v1
Document submitted locally: 2508_21069v1
Document Submitted: 2508_21069v1

Simplified Tree Structure of the Document:
Document Root (ID: 2508_21069v1_0000)
  Preamble (ID: 2508_21069v1_0001)
  Abstract (ID: 2508_21069v1_0002)
  Introduction (ID: 2508_21069v1_0003)
  Methods (ID: 2508_21069v1_0004)
  Conclusion (ID: 2508_21069v1_0005)
  References (ID: 2508_21069v1_0006)
  Appendix A (ID: 2508_21069v1_0007)

Calling local LLM for tree search (this might take a moment)...

Reasoning Process:
The question is asking about the main findings of this document, so I'm looking
for sections that summarize or conclude the results. This suggests 'Abstract',
'Conclusion', and possibly 'Results' if it exists in the tree.

Retrieved Nodes:
Node ID: 2508_21069v1_0002	 Page: 2	 Title: Abstract
Node ID: 2508_21069v1_0005	

CancelledError: 

In [3]:
!rm -rf /content/data

In [1]:
# --- START OF COLAB CELL ---

# 0.0 Setup Environment and Install Dependencies
# This cell will install necessary packages and prepare the environment.

# Install PageIndex (if not already installed)
# %pip install -q --upgrade pageindex
# %pip install -q --upgrade openai  # For potential future use or local OpenAI-compatible servers
# %pip install -q requests beautifulsoup4  # For document downloading and parsing

import os
import requests
import json
from bs4 import BeautifulSoup

# Dummy PageIndexClient for local execution (no actual API calls)
class LocalPageIndexClient:
    def __init__(self, api_key=None):
        print("Using dummy LocalPageIndexClient for local execution.")
        self.documents = {} # Store document content
        self.trees = {} # Store generated tree structures

    def submit_document(self, file_path):
        doc_id = os.path.basename(file_path).replace('.', '_').replace('-', '_') # Simple doc_id generation
        with open(file_path, 'r', encoding='utf-8') as f:
            self.documents[doc_id] = f.read()
        print(f"Document submitted locally: {doc_id}")
        self.trees[doc_id] = self._generate_simple_tree_from_text(self.documents[doc_id], doc_id)
        return {"doc_id": doc_id}

    def is_retrieval_ready(self, doc_id):
        return doc_id in self.trees

    def get_tree(self, doc_id, node_summary=True):
        return {"result": self.trees.get(doc_id, [])}

    # --- START OF MAJOR CORRECTION ---
    def _generate_simple_tree_from_text(self, text_content, doc_id):
        # This is a more robust parser for the actual scraped HTML text.
        root_node = {
            "title": "Document Root",
            "node_id": f"{doc_id}_0000",
            "prefix_summary": "Root of the document structure.",
            "nodes": [],
            "page_index": 1
        }

        parsed_sections = []
        current_title = "Preamble"  # Content before the first proper header
        current_content = []

        # More flexible headers to look for in the scraped text
        # These are keywords that often start a section title.
        header_keywords = [
            "Abstract", "Introduction", "Related Work", "Method",
            "Experiments", "Conclusion", "References", "Appendix"
        ]

        lines = text_content.split('\n')
        for line in lines:
            stripped = line.strip()
            if not stripped:
                continue

            found_header = None
            # Heuristic: A line is likely a header if it's short and starts with a keyword or a number.
            if len(stripped.split()) < 10:
                for keyword in header_keywords:
                    # Check for "1 Introduction", "5. Conclusion", "Abstract", etc.
                    if (stripped.startswith(keyword) or (len(stripped) > 1 and stripped[0].isdigit() and keyword in stripped)):
                        found_header = stripped
                        break

            if found_header:
                # If we found a new header, save the previous section's content
                if current_content:
                    parsed_sections.append({
                        "title": current_title,
                        "text": "\n".join(current_content).strip()
                    })
                # Start the new section
                current_title = found_header
                current_content = [] # Reset content buffer
            else:
                # This line is content for the current section
                current_content.append(line)

        # After the loop, save the very last section
        if current_content:
            parsed_sections.append({
                "title": current_title,
                "text": "\n".join(current_content).strip()
            })

        # Now, build the tree from the correctly parsed sections
        node_counter = 1
        for i, section in enumerate(parsed_sections):
            text = section["text"]
            summary = ' '.join(text.split()[:20]) + '...' if len(text.split()) > 20 else text

            node_entry = {
                "title": section["title"],
                "node_id": f"{doc_id}_{i+1:04d}", # Simple, sequential node IDs
                "summary": summary,
                "text": text,
                "page_index": i + 1
            }
            root_node["nodes"].append(node_entry)

        return [root_node]
    # --- END OF MAJOR CORRECTION ---

# Dummy utils for local execution (mimicking pageindex.utils)
class LocalUtils:
    @staticmethod
    def print_tree(tree, indent=0):
        for node in tree:
            prefix = "  " * indent
            print(f"{prefix}{node.get('title', 'Untitled')} (ID: {node.get('node_id', 'N/A')})")
            if "nodes" in node:
                LocalUtils.print_tree(node["nodes"], indent + 1)

    @staticmethod
    def remove_fields(obj, fields):
        if isinstance(obj, dict):
            return {k: LocalUtils.remove_fields(v, fields) for k, v in obj.items() if k not in fields}
        elif isinstance(obj, list):
            return [LocalUtils.remove_fields(elem, fields) for elem in obj]
        else:
            return obj

    @staticmethod
    def create_node_mapping(tree, node_map=None):
        if node_map is None:
            node_map = {}
        for node in tree:
            node_map[node.get('node_id')] = node
            if "nodes" in node:
                LocalUtils.create_node_mapping(node["nodes"], node_map)
        return node_map

    @staticmethod
    def print_wrapped(text, width=80):
        import textwrap
        print(textwrap.fill(text, width=width))

pi_client = LocalPageIndexClient()
utils = LocalUtils()

# Ollama LLM client for local inference
import httpx

async def call_llm(prompt, model="llama3", temperature=0):
    client = httpx.AsyncClient(base_url="http://localhost:11434")
    try:
        response = await client.post(
            "/api/chat",
            json={
                "model": model,
                "messages": [{"role": "user", "content": prompt}],
                "stream": False,
                "options": {"temperature": temperature}
            },
            timeout=None
        )
        response.raise_for_status()
        return response.json()["message"]["content"].strip()
    except httpx.ConnectError as e:
        print(f"Error connecting to Ollama: {e}\nPlease ensure Ollama is running.")
        return '{"thinking": "Error: Could not connect to Ollama.", "node_list": []}'
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return '{"thinking": "An unexpected error occurred.", "node_list": []}'

# 1.1 Submit a document for generating PageIndex tree (locally)
arxiv_html_url = "https://arxiv.org/html/2508.21069v1"
doc_name = arxiv_html_url.split('/')[-1]
doc_path = os.path.join("data", doc_name)

os.makedirs(os.path.dirname(doc_path), exist_ok=True)

print(f"Attempting to fetch text from {arxiv_html_url} for content simulation...")
try:
    html_response = requests.get(arxiv_html_url)
    html_response.raise_for_status()
    soup = BeautifulSoup(html_response.text, 'html.parser')

    article_body = soup.find('div', class_='ltx_page_content')
    if article_body:
        text_content = article_body.get_text(separator='\n', strip=True)
    else: # Fallback if specific class not found
        text_content = soup.get_text(separator='\n', strip=True)

    if not text_content.strip():
        raise ValueError("Could not extract significant text from arXiv HTML.")

    text_path = doc_path.replace('.html', '.txt')
    with open(text_path, "w", encoding="utf-8") as f:
        f.write(text_content)
    print(f"Extracted content to {text_path}")
    doc_id = pi_client.submit_document(text_path)["doc_id"]
    print('Document Submitted:', doc_id)

except Exception as e:
    print(f"Failed to extract text from arXiv HTML or save to file: {e}")
    doc_id = None # Ensure doc_id is None if setup fails

# 1.2 Get the generated PageIndex tree structure
if 'doc_id' in locals() and doc_id and pi_client.is_retrieval_ready(doc_id):
    tree = pi_client.get_tree(doc_id, node_summary=True)['result']
    print('\nSimplified Tree Structure of the Document:')
    utils.print_tree(tree)
else:
    print("\nSkipping further steps because document processing failed.")
    tree = None

if tree:
    # 2.1 Use LLM for tree search
    query = "What are the main findings of this document?"
    tree_without_text = utils.remove_fields(tree.copy(), fields=['text'])

    search_prompt = f"""
You are given a question and a tree structure of a document.
Each node contains a node id and a title. Your task is to find all nodes that are likely to contain the answer to the question. Focus on sections like 'Abstract', 'Experiments', 'Results', or 'Conclusion'.

Question: {query}

Document tree structure:
{json.dumps(tree_without_text, indent=2)}

Please reply ONLY with a JSON object in the following format, with no other text:
{{
    "thinking": "<Your brief thinking process on which nodes are relevant>",
    "node_list": ["<full_node_id_1>", "<full_node_id_2>"]
}}
"""

    print("\nCalling local LLM for tree search (this might take a moment)...")
    tree_search_result = await call_llm(search_prompt, model="llama3:latest")

    # 2.2 Print retrieved nodes and reasoning process
    try:
        node_map = utils.create_node_mapping(tree)
        tree_search_result_json = json.loads(tree_search_result)

        print('\nReasoning Process:')
        utils.print_wrapped(tree_search_result_json.get('thinking', 'No thinking process provided.'))

        print('\nRetrieved Nodes:')
        retrieved_node_ids = tree_search_result_json.get("node_list", [])
        if not retrieved_node_ids:
            print("No nodes retrieved by LLM.")

        for node_id in retrieved_node_ids:
            node = node_map.get(node_id)
            if node:
                print(f"Node ID: {node['node_id']}\t Page: {node.get('page_index', 'N/A')}\t Title: {node['title']}")
            else:
                print(f"Node ID: {node_id} (Not found in map - LLM may have hallucinated or ID format is mismatched)")

    except json.JSONDecodeError:
        print(f"\nError: LLM did not return valid JSON for tree search result:\n{tree_search_result}")
        retrieved_node_ids = []
    except Exception as e:
        print(f"\nAn error occurred during tree search result processing: {e}")
        retrieved_node_ids = []

    # 3.1 Extract relevant context
    relevant_content = []
    if retrieved_node_ids:
        for node_id in retrieved_node_ids:
            node = node_map.get(node_id)
            if node and 'text' in node:
                relevant_content.append(f"--- From Section: {node['title']} ---\n{node['text']}")

    relevant_content_str = "\n\n".join(relevant_content)

    print('\nRetrieved Context:\n')
    if relevant_content_str:
        utils.print_wrapped(relevant_content_str[:1500] + '...' if len(relevant_content_str) > 1500 else relevant_content_str)
    else:
        print("No relevant context found based on LLM's node list.")

    # 3.2 Generate answer
    if relevant_content_str:
        answer_prompt = f"Based ONLY on the following context, what are the main findings of the document?\n\nContext:\n{relevant_content_str}\n\nAnswer:"

        print('\nCalling local LLM for answer generation...')
        answer = await call_llm(answer_prompt, model="llama3:latest")
        print('\nGenerated Answer:\n')
        utils.print_wrapped(answer)
    else:
        print("\nCannot generate an answer as no relevant context was retrieved.")

# --- END OF COLAB CELL ---

Using dummy LocalPageIndexClient for local execution.
Attempting to fetch text from https://arxiv.org/html/2508.21069v1 for content simulation...
Extracted content to data/2508.21069v1
Document submitted locally: 2508_21069v1
Document Submitted: 2508_21069v1

Simplified Tree Structure of the Document:
Document Root (ID: 2508_21069v1_0000)
  Preamble (ID: 2508_21069v1_0001)
  Abstract (ID: 2508_21069v1_0002)
  Introduction (ID: 2508_21069v1_0003)
  Methods (ID: 2508_21069v1_0004)
  Conclusion (ID: 2508_21069v1_0005)
  References (ID: 2508_21069v1_0006)
  Appendix A (ID: 2508_21069v1_0007)

Calling local LLM for tree search (this might take a moment)...

Reasoning Process:
The question is asking about the main findings of this document, so I'm looking
for sections that summarize or conclude the results. This suggests 'Abstract',
'Conclusion', and possibly 'Results' if it exists in the tree.

Retrieved Nodes:
Node ID: 2508_21069v1_0002	 Page: 2	 Title: Abstract
Node ID: 2508_21069v1_0005	